In [5]:
# Step 1: Install required packages
!pip install -q seaborn ipywidgets plotly pandas numpy ibm-watson-machine-learning

# Step 2: Set up environment variables for IBM Watson
import os
os.environ["IBM_API_KEY"] = "RyIKeWjkIi-jx4F2uerWwUuInh6d9hoIJal8frtzKIMk"
os.environ["IBM_URL"] = "https://us-south.ml.cloud.ibm.com"
os.environ["IBM_PROJECT_ID"] = "5d59376b-9172-474f-ac91-c33d1f16bb7b"

# Step 3: Import libraries
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown, HTML
import base64
from io import BytesIO
import plotly.graph_objects as go
import time
import json
import re
import requests
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.wml_client_error import ApiRequestFailure, WMLClientError
import logging

# Enable Colab widget support
from google.colab import output
output.enable_custom_widget_manager()

# ======================
# LOGGING CONFIGURATION
# ======================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger('SideChannelSentinel')

# ======================
# IBM WATSONX CONFIGURATION
# ======================
class WatsonxConfig:
    @staticmethod
    def get_model(model_id, max_tokens=500, temperature=0.2):
        """Initialize IBM Granite model with credentials and retry logic"""
        for attempt in range(3):
            try:
                model = Model(
                    model_id=model_id,
                    credentials={
                        "apikey": os.environ["IBM_API_KEY"],
                        "url": os.environ["IBM_URL"]
                    },
                    project_id=os.environ["IBM_PROJECT_ID"],
                    params={
                        GenParams.DECODING_METHOD: "greedy",
                        GenParams.MAX_NEW_TOKENS: max_tokens,
                        GenParams.MIN_NEW_TOKENS: 50,
                        GenParams.TEMPERATURE: temperature,
                        GenParams.TOP_P: 0.9,
                        GenParams.TOP_K: 50
                    }
                )
                return model
            except (ApiRequestFailure, WMLClientError, requests.RequestException) as e:
                wait_time = (2 ** attempt) + 1
                logger.error(f"IBM API error (attempt {attempt+1}/3): {str(e)}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
            except Exception as e:
                logger.error(f"Unexpected error initializing model: {str(e)}")
                return None
        logger.error("Failed to initialize model after 3 attempts")
        return None

# ======================
# ENHANCED SIGNAL PROCESSING
# ======================
class AdvancedSignalAnalyzer:
    @staticmethod
    def detect_leakage_points(df, sensitivity=7, confidence=0.85):
        """Advanced leakage detection with multiple techniques"""
        logger.info("Starting leakage detection analysis")
        
        # 1. Statistical Analysis with Adaptive Thresholding
        window_size = max(10, int(len(df) * 0.02))  # Dynamic window size
        df['MA'] = df['Power Consumption (mA)'].rolling(window=window_size).mean()
        df['STD'] = df['Power Consumption (mA)'].rolling(window=window_size).std()
        
        # Adaptive threshold based on sensitivity
        sensitivity_multiplier = 0.5 + (sensitivity / 10) * 4.5  # 0.5 to 5.0
        df['Threshold'] = df['STD'].rolling(window=100, min_periods=1).median() * sensitivity_multiplier
        
        # 2. Simulated Correlation Power Analysis (CPA)
        # Generate hypothetical power model (Hamming weight model)
        hypothetical_power = AdvancedSignalAnalyzer.simulate_hamming_model(df)
        df['CPA_Correlation'] = df['Power Consumption (mA)'].rolling(window=window_size).corr(hypothetical_power)
        
        # 3. Combined detection logic
        high_deviation = df[(df['STD'] > df['Threshold']) | 
                            (df['CPA_Correlation'].abs() > confidence)]
        
        leakage_points = []
        if not high_deviation.empty:
            # Group consecutive points
            high_deviation['Group'] = (high_deviation.index.to_series().diff() > 1).cumsum()
            
            for _, group in high_deviation.groupby('Group'):
                peak_idx = group['STD'].idxmax()
                peak_time = df.loc[peak_idx, 'Time (ms)']
                peak_value = df.loc[peak_idx, 'Power Consumption (mA)']
                
                # Calculate feature vector for LLM
                features = {
                    'duration': group['Time (ms)'].max() - group['Time (ms)'].min(),
                    'avg_power': group['Power Consumption (mA)'].mean(),
                    'max_power': group['Power Consumption (mA)'].max(),
                    'cpa_corr': group['CPA_Correlation'].abs().mean(),
                    'std_dev': group['STD'].mean()
                }
                
                # Confidence score based on multiple factors
                conf_score = min(0.95, (features['std_dev'] / df['Threshold'].mean() * 0.4 + 
                                        features['cpa_corr'] * 0.6))
                
                leakage_points.append({
                    'time_ms': peak_time,
                    'power_value': peak_value,
                    'features': features,
                    'confidence': conf_score
                })
        
        logger.info(f"Detected {len(leakage_points)} potential leakage points")
        return leakage_points

    @staticmethod
    def simulate_hamming_model(df):
        """Simulate CPA Hamming weight model for demonstration"""
        # In real CPA, this would be based on intermediate values
        # Here we simulate with a synthetic pattern
        np.random.seed(42)
        base = np.sin(2 * np.pi * df['Time (ms)'] * 5) * 0.3
        spikes = np.zeros(len(df))
        
        # Add spikes at random positions to simulate operation points
        for _ in range(5):
            idx = np.random.randint(0, len(df))
            width = np.random.uniform(0.05, 0.2)
            spike = np.exp(-(df['Time (ms)'] - df.loc[idx, 'Time (ms)'])**2 / (2 * width**2))
            spikes += spike * np.random.uniform(0.5, 1.5)
        
        return base + spikes

    @staticmethod
    def calculate_correlation(leakage_points):
        """Calculate correlation metrics for leakage points"""
        if not leakage_points:
            return 0.0, "Low"
        
        # Weighted average of multiple factors
        total_risk = 0
        weights = {'std_dev': 0.4, 'cpa_corr': 0.5, 'duration': 0.1}
        
        for point in leakage_points:
            features = point['features']
            risk_score = (features['std_dev'] * weights['std_dev'] +
                          features['cpa_corr'] * weights['cpa_corr'] +
                          features['duration'] * weights['duration'])
            total_risk += risk_score * point['confidence']
        
        overall_risk = min(1.0, total_risk / len(leakage_points))
        
        # Determine risk level
        if overall_risk > 0.7:
            risk_level = "High"
        elif overall_risk > 0.5:
            risk_level = "Medium"
        else:
            risk_level = "Low"
            
        return overall_risk, risk_level

# ======================
# ENHANCED GRANITE MODELS INTEGRATION
# ======================
class EnhancedGraniteModels:
    @staticmethod
    def analyze_results(leakage_points, overall_risk, risk_level, model_choice="granite-4.0-tiny"):
        """Enhanced analysis with dynamic model selection"""
        model_id_map = {
            "tiny": "ibm/granite-4.0-tiny",
            "medium": "ibm/granite-4.0-medium",
            "instruct": "ibm/granite-8b-instruct"
        }
        
        model_id = model_id_map.get(model_choice.lower(), "ibm/granite-4.0-tiny")
        logger.info(f"Using model: {model_id} for vulnerability analysis")
        
        try:
            model = WatsonxConfig.get_model(model_id, max_tokens=600)
            if not model:
                return EnhancedGraniteModels.simulate_analysis_summary(leakage_points, overall_risk)
            
            # Prepare detailed leakage points description
            leakage_desc = "\n".join(
                f"- At {point['time_ms']:.2f} ms: "
                f"Power={point['power_value']:.2f}mA, "
                f"Duration={point['features']['duration']:.4f}ms, "
                f"CPA_Corr={point['features']['cpa_corr']:.2f}, "
                f"Confidence={point['confidence']:.2f}"
                for point in leakage_points
            ) if leakage_points else "No significant leakage points detected"
            
            # Enhanced prompt with more context
            prompt = f"""
            [INST] <<SYS>>
            You are a hardware security expert analyzing side-channel power traces. 
            Provide a technical assessment of these findings:

            DETECTION RESULTS:
            {leakage_desc}

            OVERALL RISK: {overall_risk:.2f} ({risk_level})
            
            ANALYSIS TASKS:
            1. Vulnerability summary (1-2 sentences)
            2. Identify cryptographic operations affected by leakage
            3. Assess exploit difficulty (Low/Medium/High)
            4. Business impact assessment
            5. Return response in JSON format:
            {{
                "vulnerability_summary": "Summary text",
                "affected_operations": ["operation1", "operation2"],
                "exploit_difficulty": "Low/Medium/High",
                "business_impact": "Impact explanation",
                "confidence_score": 0.0-1.0
            }}
            <</SYS>>
            [/INST]
            """
            
            # Generate analysis with retry
            response = EnhancedGraniteModels.retry_api_call(model.generate, prompt=prompt)
            analysis = response['results'][0]['generated_text']
            
            # Extract JSON safely
            result = EnhancedGraniteModels.extract_json(analysis)
            result['model_used'] = model_id.split('/')[-1]
            return result
        except Exception as e:
            logger.error(f"Analysis error: {str(e)}")
            return EnhancedGraniteModels.simulate_analysis_summary(leakage_points, overall_risk)
    
    @staticmethod
    def retry_api_call(func, *args, **kwargs):
        """Retry API call with exponential backoff"""
        for attempt in range(3):
            try:
                return func(*args, **kwargs)
            except (ApiRequestFailure, requests.RequestException) as e:
                wait_time = (2 ** attempt) + 1
                logger.warning(f"API call failed (attempt {attempt+1}/3): {str(e)}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
        raise Exception("API call failed after 3 attempts")

    @staticmethod
    def extract_json(text):
        """Robust JSON extraction from LLM response"""
        try:
            # Improved pattern to find JSON objects
            json_match = re.search(r'\{[\s\S]*\}', text)
            if not json_match:
                raise ValueError("No JSON found in response")
                
            json_str = json_match.group(0)
            return json.loads(json_str)
        except (ValueError, json.JSONDecodeError) as e:
            logger.error(f"JSON extraction error: {str(e)}")
            return {
                "vulnerability_summary": "Analysis summary unavailable",
                "affected_operations": [],
                "business_impact": "Unable to generate business impact assessment",
                "exploit_difficulty": "Unknown",
                "confidence_score": 0.0
            }
    
    @staticmethod
    def simulate_analysis_summary(leakage_points, overall_risk):
        """Fallback simulation if API fails"""
        summary = "Significant correlation detected between power consumption and secret data"
        operations = ["S-Box substitution", "Key schedule"]
        
        if overall_risk > 0.7:
            impact = "High risk of key extraction leading to complete system compromise"
            difficulty = "Medium (requires specialized equipment)"
        elif overall_risk > 0.5:
            impact = "Moderate risk of partial key recovery"
            difficulty = "High (requires advanced expertise)"
        else:
            impact = "Low risk in practical scenarios"
            difficulty = "Very High (theoretical risk only)"
        
        return {
            "vulnerability_summary": summary,
            "affected_operations": operations,
            "business_impact": impact,
            "exploit_difficulty": difficulty,
            "confidence_score": min(0.9, overall_risk + 0.2)
        }
    
    @staticmethod
    def generate_countermeasures(analysis, model_choice="granite-8b-instruct"):
        """Generate targeted countermeasures with model choice"""
        model_id_map = {
            "instruct": "ibm/granite-8b-instruct",
            "medium": "ibm/granite-4.0-medium",
            "tiny": "ibm/granite-4.0-tiny"
        }
        
        model_id = model_id_map.get(model_choice.lower(), "ibm/granite-8b-instruct")
        logger.info(f"Using model: {model_id} for countermeasures")
        
        try:
            model = WatsonxConfig.get_model(model_id, max_tokens=800, temperature=0.3)
            if not model:
                return EnhancedGraniteModels.simulate_countermeasures()
            
            # Prepare prompt with targeted operations
            operations = analysis.get('affected_operations', [])
            targeted_ops = f"Specifically target: {', '.join(operations)}" if operations else ""
            
            prompt = f"""
            [INST] <<SYS>>
            As a hardware security specialist, suggest countermeasures for these vulnerabilities:
            
            {analysis.get('vulnerability_summary', 'Vulnerability analysis unavailable')}
            
            Affected Operations: {', '.join(operations) or 'Not specified'}
            Exploit Difficulty: {analysis.get('exploit_difficulty', 'Unknown')}
            
            Requirements:
            1. Provide 5 hardware-focused countermeasures
            2. For each: 
               - Brief description
               - Implementation approach
               - Estimated effectiveness (High/Medium/Low)
            3. Format: 
               "1. [Description] 
                Implementation: [Details] 
                Effectiveness: [Rating]"
            4. Prioritize countermeasures addressing the specific operations mentioned
            {targeted_ops}
            <</SYS>>
            [/INST]
            """
            
            # Generate countermeasures
            response = EnhancedGraniteModels.retry_api_call(model.generate, prompt=prompt)
            text = response['results'][0]['generated_text']
            
            # Parse with enhanced regex
            return EnhancedGraniteModels.parse_countermeasures(text)
        except Exception as e:
            logger.error(f"Countermeasures error: {str(e)}")
            return EnhancedGraniteModels.simulate_countermeasures()
    
    @staticmethod
    def parse_countermeasures(text):
        """Robust parsing of countermeasures with effectiveness"""
        pattern = r"(\d+)\.\s*([^\n]+?)\n\s*Implementation:\s*(.*?)\n\s*Effectiveness:\s*(High|Medium|Low)"
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
        
        countermeasures = []
        implementations = []
        effectiveness = []
        
        for num, desc, impl, eff in matches[:5]:
            countermeasures.append(desc.strip())
            implementations.append(impl.strip())
            effectiveness.append(eff.strip())
        
        # Fill any missing with simulated
        sim_cm, sim_imp, sim_eff = EnhancedGraniteModels.simulate_countermeasures()
        for i in range(5 - len(countermeasures)):
            countermeasures.append(sim_cm[i])
            implementations.append(sim_imp[i])
            effectiveness.append(sim_eff[i])
            
        intro = f"Generated by IBM Granite model"
        return countermeasures, implementations, effectiveness, intro
    
    @staticmethod
    def simulate_countermeasures():
        """Fallback simulation if API fails"""
        countermeasures = [
            "Implement Boolean masking for S-Box operations",
            "Add dynamic noise generation circuits",
            "Use constant-time implementations for all operations",
            "Implement operation shuffling with PRNG",
            "Add random delays between cryptographic operations"
        ]
        
        implementations = [
            "Apply additive/multiplicative masking to intermediate values",
            "Use LFSR-based noise generator with adjustable amplitude",
            "Ensure all branches and memory accesses take fixed time",
            "Randomize processing order using hardware PRNG",
            "Insert delays with Poisson distribution between rounds"
        ]
        
        effectiveness = ["High", "Medium", "High", "Medium", "Low"]
        
        return countermeasures, implementations, effectiveness

# ======================
# ENHANCED VISUALIZATION
# ======================
def generate_sample_data():
    """Create realistic power trace simulation data with CPA features"""
    np.random.seed(42)
    time_points = np.linspace(0, 10, 1000)
    base_signal = np.sin(2 * np.pi * time_points) * 0.5
    
    # Add cryptographic operation signatures
    operation_points = [
        {"time_ms": 1.5, "magnitude": 1.8, "width": 0.2, "operation": "S-Box substitution"},
        {"time_ms": 3.2, "magnitude": 1.5, "width": 0.15, "operation": "Key schedule"},
        {"time_ms": 5.0, "magnitude": 0.8, "width": 0.1, "operation": "AddRoundKey"},
        {"time_ms": 7.8, "magnitude": 1.2, "width": 0.25, "operation": "Final round"}
    ]
    
    for point in operation_points:
        idx = np.abs(time_points - point['time_ms']).argmin()
        gaussian = point['magnitude'] * np.exp(-(time_points - point['time_ms'])**2 / (2 * point['width']**2))
        base_signal += gaussian
    
    # Add noise and high-frequency components
    base_signal += np.random.normal(0, 0.15, len(time_points))
    base_signal += 0.3 * np.sin(15 * np.pi * time_points)
    base_signal += 0.2 * np.random.random(len(time_points))
    
    return pd.DataFrame({
        "Time (ms)": time_points,
        "Power Consumption (mA)": base_signal
    }), operation_points

def plot_power_traces(df, leakage_points=None, operation_points=None):
    """Enhanced plot with interactive leakage points"""
    fig = go.Figure()
    
    # Main power trace
    fig.add_trace(go.Scatter(
        x=df["Time (ms)"],
        y=df["Power Consumption (mA)"],
        mode='lines',
        name='Power Consumption',
        line=dict(color='#0068c9', width=2),
        fill='tozeroy',
        fillcolor='rgba(0, 104, 201, 0.1)',
        hovertemplate='Time: %{x:.2f} ms<br>Power: %{y:.2f} mA<extra></extra>'
    ))
    
    # Known operation points
    if operation_points:
        for point in operation_points:
            fig.add_vline(
                x=point['time_ms'],
                line=dict(color="#888", width=1.5, dash="dot"),
                annotation_text=point["operation"],
                annotation_position="top right",
                annotation_font_size=10
            )
    
    # Detected leakage points
    if leakage_points:
        leak_x = [point['time_ms'] for point in leakage_points]
        leak_y = [point['power_value'] for point in leakage_points]
        leak_text = [
            f"<b>Leakage Point</b><br>Time: {p['time_ms']:.2f} ms<br>Power: {p['power_value']:.2f} mA"
            f"<br>Confidence: {p['confidence']:.2f}<br>CPA Corr: {p['features']['cpa_corr']:.2f}"
            for p in leakage_points
        ]
        
        fig.add_trace(go.Scatter(
            x=leak_x,
            y=leak_y,
            mode='markers',
            name='Leakage Detected',
            marker=dict(size=12, color='red', symbol='x'),
            hovertemplate='%{text}<extra></extra>',
            text=leak_text
        ))
    
    fig.update_layout(
        title='Power Consumption Analysis',
        xaxis_title='Time (ms)',
        yaxis_title='Power Consumption (mA)',
        template='plotly_white',
        height=450,
        hovermode="x unified",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    return fig

def risk_meter(value):
    """Create a visual risk meter with improved design"""
    colors = ['#00c853', '#ffd600', '#ff9100', '#ff3d00']
    value = min(100, max(0, value))
    level = min(3, int(value // 25))
    
    html = f"""
    <div style="background: #f8f9fa; border-radius: 10px; padding: 15px; margin: 10px 0; position: relative; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
        <div style="position: absolute; top: 15px; right: 15px; font-weight: bold; font-size: 24px; color: {colors[level]}">
            {value:.1f}%
        </div>
        <div style="margin-top: 10px;">
            <div style="display: flex; justify-content: space-between; margin-bottom: 5px; font-size: 12px;">
                <span>Low</span>
                <span>Medium</span>
                <span>High</span>
                <span>Critical</span>
            </div>
            <div style="background: #e0e0e0; height: 20px; border-radius: 10px; overflow: hidden;">
                <div style="background: linear-gradient(to right, {', '.join(colors)}); 
                            width: {value}%; height: 100%; 
                            position: relative;">
                </div>
            </div>
        </div>
        <div style="margin-top: 15px; font-size: 14px; color: #666;">
            <i class='fa fa-info-circle'></i> Risk level indicates likelihood of key extraction
        </div>
    </div>
    """
    return html

def create_download_link(df, filename="power_traces.csv"):
    """Generate a download link for CSV file"""
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    return f'<a href="data:file/csv;base64,{b64}" download="{filename}" style="color: #0068c9; text-decoration: none; border: 1px solid #0068c9; border-radius: 4px; padding: 6px 12px; display: inline-block; margin: 10px 0;">Download CSV File</a>'

# ======================
# ENHANCED UI MODULE
# ======================
# Create widgets
use_sample_data = widgets.Checkbox(
    value=True,
    description='Use sample data for demonstration',
    indent=False,
    style={'description_width': 'initial'}
)

upload_widget = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Upload CSV',
    layout=widgets.Layout(width='300px'),
    tooltip="CSV should contain 'Time (ms)' and 'Power Consumption (mA)' columns"
)

sensitivity_slider = widgets.IntSlider(
    value=5,  # Default changed to 5 (maps to 2.5 threshold multiplier)
    min=1,
    max=10,
    step=1,
    description='Detection Sensitivity:',
    style={'description_width': 'initial'},
    continuous_update=False,
    tooltip="Higher values detect smaller leaks but may increase false positives"
)

confidence_slider = widgets.IntSlider(
    value=85,
    min=50,
    max=100,
    step=5,
    description='Min Confidence Threshold:',
    style={'description_width': 'initial'},
    continuous_update=False,
    tooltip="Minimum confidence score to consider a leakage point valid"
)

model_selector = widgets.Dropdown(
    options=[('Granite 4.0 Tiny', 'tiny'), 
             ('Granite 4.0 Medium', 'medium'),
             ('Granite 8B Instruct', 'instruct')],
    value='tiny',
    description='Analysis Model:',
    style={'description_width': 'initial'},
    tooltip="Select IBM Granite model for vulnerability analysis"
)

analyze_button = widgets.Button(
    description='Analyze Power Traces',
    button_style='success',
    icon='search',
    layout=widgets.Layout(width='200px', height='40px')
)

export_button = widgets.Button(
    description='Export Full Report',
    button_style='info',
    icon='file-download',
    layout=widgets.Layout(width='200px', height='40px'),
    disabled=True
)

# Create output areas
power_trace_output = widgets.Output()
analysis_output = widgets.Output()
countermeasure_output = widgets.Output()

# ======================
# MAIN FUNCTIONALITY
# ======================
# Global state for report generation
analysis_results = {}
report_data = {}

def on_analyze_button_clicked(b):
    global analysis_results, report_data
    # Reset outputs and state
    power_trace_output.clear_output()
    analysis_output.clear_output()
    countermeasure_output.clear_output()
    analysis_results = {}
    report_data = {}
    export_button.disabled = True
    
    df = None
    operation_points = None
    file_uploaded = False
    
    with power_trace_output:
        # Show detailed progress
        display(widgets.HTML("<div class='progress-step'><i class='fa fa-spinner fa-spin'></i> <b>Step 1/4:</b> Processing input data...</div>"))
        
        # Handle data input
        if upload_widget.value:
            try:
                uploaded = list(upload_widget.value.values())[0]
                content = uploaded['content'].decode('utf-8')
                df = pd.read_csv(BytesIO(content.encode()))
                
                # Validate data types
                if not pd.api.types.is_numeric_dtype(df['Time (ms)']):
                    raise ValueError("Time column must be numeric")
                if not pd.api.types.is_numeric_dtype(df['Power Consumption (mA)']):
                    raise ValueError("Power column must be numeric")
                
                # Check data ranges
                if df['Time (ms)'].min() < 0 or df['Time (ms)'].max() > 10000:
                    raise ValueError("Time values out of expected range (0-10000 ms)")
                if df['Power Consumption (mA)'].abs().max() > 1000:
                    raise ValueError("Power values exceed reasonable range (±1000 mA)")
                
                file_uploaded = True
                display(widgets.HTML("<div class='success-message'>CSV file successfully uploaded and validated!</div>"))
            except Exception as e:
                display(widgets.HTML(f"<div class='error-message'><b>Data Validation Error:</b> {str(e)}</div>"))
                return
        elif use_sample_data.value:
            df, operation_points = generate_sample_data()
        
        # Validate data
        if df is None or df.empty:
            display(widgets.HTML("<div class='error-message'>No data available. Please upload a file or use sample data.</div>"))
            return
        
        # Check required columns
        required_columns = ["Time (ms)", "Power Consumption (mA)"]
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            display(widgets.HTML(
                f"<div class='error-message'><b>Missing columns:</b> {', '.join(missing)}<br>"
                f"Required columns: {', '.join(required_columns)}</div>"
            ))
            return
        
        # Update progress
        power_trace_output.clear_output()
        display(widgets.HTML("<div class='progress-step'><i class='fa fa-check-circle'></i> <b>Step 1/4:</b> Data processed successfully</div>"))
        display(widgets.HTML("<div class='progress-step'><i class='fa fa-spinner fa-spin'></i> <b>Step 2/4:</b> Visualizing power trace...</div>"))
        
        display(widgets.HTML("<h3 style='color:#0068c9; border-bottom: 1px solid #e0e0e0; padding-bottom: 10px;'>Power Trace Data</h3>"))
        
        if use_sample_data.value:
            display(widgets.HTML("<p style='color: #666;'>Simulated AES-256 cryptographic operation data</p>"))
            display(widgets.HTML(create_download_link(df)))
        
        # Plot power traces
        fig = plot_power_traces(df, operation_points=operation_points)
        display(fig)
        
        # Show data summary
        display(widgets.HTML(f"""
        <div class='summary-box'>
            <h4 style='color:#0068c9; margin-top:0;'>Data Summary</h4>
            <ul>
                <li><b>Time Points:</b> {len(df)}</li>
                <li><b>Duration:</b> {df['Time (ms)'].iloc[-1]:.2f} ms</li>
                <li><b>Avg Power:</b> {df['Power Consumption (mA)'].mean():.2f} mA</li>
                <li><b>Max Power:</b> {df['Power Consumption (mA)'].max():.2f} mA</li>
                <li><b>Data Source:</b> {"User Upload" if file_uploaded else "Sample Data"}</li>
            </ul>
        </div>
        """))
        
        # Store for report
        report_data['df'] = df
        report_data['operation_points'] = operation_points
        report_data['file_uploaded'] = file_uploaded
    
    with analysis_output:
        display(widgets.HTML("<div class='progress-step'><i class='fa fa-spinner fa-spin'></i> <b>Step 3/4:</b> Analyzing power traces...</div>"))
        
        # Run signal analysis
        leakage_points = AdvancedSignalAnalyzer.detect_leakage_points(
            df, 
            sensitivity=sensitivity_slider.value,
            confidence=confidence_slider.value/100
        )
        overall_risk, risk_level = AdvancedSignalAnalyzer.calculate_correlation(leakage_points)
        
        # Update progress
        analysis_output.clear_output()
        display(widgets.HTML("<div class='progress-step'><i class='fa fa-check-circle'></i> <b>Step 3/4:</b> Signal analysis completed</div>"))
        display(widgets.HTML("<div class='progress-step'><i class='fa fa-spinner fa-spin'></i> <b>Step 4/4:</b> Synthesizing results with IBM Granite...</div>"))
        
        # Use Granite for interpretation
        analysis = EnhancedGraniteModels.analyze_results(
            leakage_points, 
            overall_risk, 
            risk_level,
            model_choice=model_selector.value
        )
        
        # Store results for report
        analysis_results = {
            'leakage_points': leakage_points,
            'overall_risk': overall_risk,
            'risk_level': risk_level,
            'analysis': analysis
        }
        
        # Clear loading and show results
        analysis_output.clear_output()
        
        display(widgets.HTML(f"""
        <h3 style='color:#0068c9; border-bottom: 1px solid #e0e0e0; padding-bottom: 10px;'>
            Vulnerability Analysis Results
            <span style='font-size: 0.6em; color: #666; float: right;'>
                Sens: {sensitivity_slider.value}/10 | Conf: {confidence_slider.value}% | Model: {analysis.get('model_used', 'Granite')}
            </span>
        </h3>
        """))
        
        # Risk assessment
        risk_score = overall_risk * 100
        risk_color = "#ff3d00" if risk_level == "High" else "#ff9100" if risk_level == "Medium" else "#00c853"
        
        display(widgets.HTML(f"""
        <div class='risk-box'>
            <h4 style='color:#0068c9; margin-top:0;'>Risk Assessment</h4>
            {risk_meter(risk_score)}
            <div class='risk-details'>
                <table>
                    <tr>
                        <td><b>Risk Level:</b></td>
                        <td style='color:{risk_color}; font-weight:bold;'>{risk_level}</td>
                        <td><b>Analysis Model:</b></td>
                        <td>{analysis.get('model_used', 'IBM Granite')}</td>
                    </tr>
                    <tr>
                        <td><b>Exploit Difficulty:</b></td>
                        <td>{analysis.get('exploit_difficulty', 'Unknown')}</td>
                        <td><b>Confidence Score:</b></td>
                        <td>{analysis.get('confidence_score', 0.0):.2f}</td>
                    </tr>
                </table>
            </div>
        </div>
        """))
        
        # Vulnerability summary
        display(widgets.HTML(f"""
        <div class='vulnerability-summary'>
            <h4 style='margin-top:0; color:#ff8f00;'>Vulnerability Summary</h4>
            <p>{analysis.get("vulnerability_summary", "Summary unavailable")}</p>
        </div>
        """))
        
        # Affected operations
        if analysis.get("affected_operations"):
            display(widgets.HTML("<h4 style='color:#0068c9;'>Affected Cryptographic Operations</h4>"))
            ops_html = "<ul>"
            for op in analysis["affected_operations"]:
                ops_html += f"<li>{op}</li>"
            ops_html += "</ul>"
            display(widgets.HTML(ops_html))
        
        # Business risk
        display(widgets.HTML("<h4 style='color:#0068c9; margin-top:20px;'>Business Impact</h4>"))
        display(widgets.HTML(f"<p>{analysis.get('business_impact', 'Risk assessment unavailable')}</p>"))
        
        # Enable export button
        export_button.disabled = False
    
    with countermeasure_output:
        display(widgets.HTML("<div class='progress-step'><i class='fa fa-spinner fa-spin'></i> Generating security recommendations...</div>"))
        
        # Generate countermeasures
        countermeasures, implementations, effectiveness, intro = EnhancedGraniteModels.generate_countermeasures(
            analysis,
            model_choice=model_selector.value
        )
        
        # Store for report
        analysis_results['countermeasures'] = countermeasures
        analysis_results['implementations'] = implementations
        analysis_results['effectiveness'] = effectiveness
        
        # Clear loading and show results
        countermeasure_output.clear_output()
        
        display(widgets.HTML(f"""
        <h3 style='color:#0068c9; border-bottom: 1px solid #e0e0e0; padding-bottom: 10px;'>
            Security Recommendations
            <span style='font-size: 0.6em; color: #666; float: right;'>
                {intro}
            </span>
        </h3>
        """))
        
        display(widgets.HTML(f"""
        <div class='model-info'>
            <p><b>Countermeasure model:</b> {model_selector.label}</p>
        </div>
        """))
        
        # Countermeasures
        display(widgets.HTML("<h4 style='color:#0068c9;'>Recommended Countermeasures</h4>"))
        
        cm_html = "<div class='countermeasures'>"
        for i, (cm, imp, eff) in enumerate(zip(countermeasures, implementations, effectiveness), 1):
            eff_color = "#4caf50" if eff == "High" else "#ff9800" if eff == "Medium" else "#f44336"
            cm_html += f"""
            <div class='countermeasure-box'>
                <div class='cm-header'>
                    <span class='cm-number'>#{i}</span>
                    <span class='cm-title'>{cm}</span>
                    <span class='cm-effectiveness' style='background-color:{eff_color};'>{eff}</span>
                </div>
                <div class='cm-implementation'>
                    <i class='fa fa-cog'></i> <b>Implementation:</b> {imp}
                </div>
            </div>
            """
        cm_html += "</div>"
        
        display(widgets.HTML(cm_html))
        
        # Implementation guidance
        display(widgets.HTML("""
        <h4 style='color:#0068c9; margin-top:25px;'>Implementation Roadmap</h4>
        <div class='roadmap-box'>
            <table>
                <tr class='header-row'>
                    <th>Phase</th>
                    <th>Activities</th>
                    <th>Timeline</th>
                </tr>
                <tr>
                    <td><b>Phase 1</b></td>
                    <td>Implement masking and shuffling countermeasures</td>
                    <td style='text-align:center;'>2-4 weeks</td>
                </tr>
                <tr class='alt-row'>
                    <td><b>Phase 2</b></td>
                    <td>Add noise generation circuits and constant-time implementations</td>
                    <td style='text-align:center;'>3-5 weeks</td>
                </tr>
                <tr>
                    <td><b>Phase 3</b></td>
                    <td>Verification and penetration testing</td>
                    <td style='text-align:center;'>2-3 weeks</td>
                </tr>
            </table>
            
            <div class='verification-box'>
                <b>Verification Steps:</b>
                <ol>
                    <li>Re-run power analysis after each mitigation</li>
                    <li>Perform correlation coefficient validation</li>
                    <li>Conformal equivalence checking</li>
                    <li>100% coverage testing of vulnerable operations</li>
                    <li>Penetration testing with real-world attack scenarios</li>
                </ol>
            </div>
        </div>
        """))
        
        # Add export button
        display(export_button)

def on_export_button_clicked(b):
    """Generate comprehensive HTML report"""
    if not analysis_results or not report_data:
        with countermeasure_output:
            display(widgets.HTML("<div class='error-message'>No analysis data available for export</div>"))
        return
    
    try:
        # Generate HTML report
        html_content = generate_html_report(report_data, analysis_results)
        
        # Create download link
        b64 = base64.b64encode(html_content.encode()).decode()
        url = f'data:text/html;base64,{b64}'
        
        with countermeasure_output:
            display(widgets.HTML(
                f"<div class='success-message'>Report generated successfully!</div>"
                f"<a href='{url}' download='sidechannel_report.html' class='download-link'>"
                "<i class='fa fa-download'></i> Download Full Report"
                "</a>"
            ))
    except Exception as e:
        with countermeasure_output:
            display(widgets.HTML(f"<div class='error-message'>Report generation failed: {str(e)}</div>"))

def generate_html_report(report_data, analysis_results):
    """Generate comprehensive HTML report"""
    # Create plot image
    fig = plot_power_traces(
        report_data['df'], 
        leakage_points=analysis_results.get('leakage_points', []),
        operation_points=report_data.get('operation_points', [])
    )
    plot_html = fig.to_html(full_html=False, include_plotlyjs='cdn')
    
    # Risk meter
    risk_score = analysis_results.get('overall_risk', 0) * 100
    risk_level = analysis_results.get('risk_level', 'Low')
    
    # Countermeasures table
    cm_table = ""
    for i, (cm, imp, eff) in enumerate(zip(
        analysis_results.get('countermeasures', []),
        analysis_results.get('implementations', []),
        analysis_results.get('effectiveness', [])
    ), 1):
        eff_color = "green" if eff == "High" else "orange" if eff == "Medium" else "red"
        cm_table += f"""
        <tr>
            <td>{i}</td>
            <td>{cm}</td>
            <td>{imp}</td>
            <td style="color:{eff_color}">{eff}</td>
        </tr>
        """
    
    # Create HTML structure
    return f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>SideChannel Sentinel Report</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; }}
            .container {{ max-width: 1200px; margin: 0 auto; padding: 20px; }}
            .header {{ background: #0068c9; color: white; padding: 30px; text-align: center; border-radius: 8px; }}
            .section {{ margin: 30px 0; padding: 20px; background: white; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.05); }}
            .risk-meter {{ background: #f8f9fa; padding: 20px; border-radius: 8px; }}
            table {{ width: 100%; border-collapse: collapse; }}
            th, td {{ padding: 12px 15px; text-align: left; border-bottom: 1px solid #ddd; }}
            th {{ background-color: #f5f5f5; }}
            .countermeasure-row:nth-child(even) {{ background-color: #f9f9f9; }}
            .plot-container {{ margin: 20px 0; }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>SideChannel Sentinel Report</h1>
                <p>Comprehensive Side-Channel Vulnerability Analysis</p>
                <p>Generated on {time.strftime("%Y-%m-%d %H:%M:%S")}</p>
            </div>
            
            <div class="section">
                <h2>Executive Summary</h2>
                <p><b>Overall Risk:</b> <span style="color:{"#ff3d00" if risk_level=="High" else "#ff9100" if risk_level=="Medium" else "#00c853"}">{risk_level}</span> ({risk_score:.1f}%)</p>
                <p><b>Vulnerability Summary:</b> {analysis_results.get('analysis', {}).get('vulnerability_summary', '')}</p>
                <p><b>Business Impact:</b> {analysis_results.get('analysis', {}).get('business_impact', '')}</p>
            </div>
            
            <div class="section">
                <h2>Power Trace Analysis</h2>
                <div class="plot-container">
                    {plot_html}
                </div>
            </div>
            
            <div class="section">
                <h2>Detailed Vulnerability Assessment</h2>
                <p><b>Affected Operations:</b> {', '.join(analysis_results.get('analysis', {}).get('affected_operations', [])) or 'None detected'}</p>
                <p><b>Exploit Difficulty:</b> {analysis_results.get('analysis', {}).get('exploit_difficulty', 'Unknown')}</p>
                <p><b>Analysis Confidence:</b> {analysis_results.get('analysis', {}).get('confidence_score', 0.0)*100:.1f}%</p>
            </div>
            
            <div class="section">
                <h2>Security Recommendations</h2>
                <table>
                    <tr>
                        <th>#</th>
                        <th>Countermeasure</th>
                        <th>Implementation</th>
                        <th>Effectiveness</th>
                    </tr>
                    {cm_table}
                </table>
            </div>
            
            <div class="section">
                <h2>Implementation Roadmap</h2>
                <table>
                    <tr>
                        <th>Phase</th>
                        <th>Activities</th>
                        <th>Timeline</th>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 1</td>
                        <td>Implement critical countermeasures</td>
                        <td>2-4 weeks</td>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 2</td>
                        <td>Hardware modifications and verification</td>
                        <td>3-5 weeks</td>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 3</td>
                        <td>Penetration testing and validation</td>
                        <td>2-3 weeks</td>
                    </tr>
                </table>
            </div>
        </div>
    </body>
    </html>
    """

# Bind button clicks
analyze_button.on_click(on_analyze_button_clicked)
export_button.on_click(on_export_button_clicked)

# ======================
# STYLES AND UX ENHANCEMENTS
# ======================
styles = """
<style>
    .progress-step {
        background: #e3f2fd;
        padding: 10px 15px;
        border-radius: 5px;
        margin: 10px 0;
        border-left: 4px solid #0068c9;
    }
    .success-message {
        color: #2e7d32;
        background: #e8f5e9;
        padding: 10px;
        border-radius: 5px;
        border-left: 4px solid #4caf50;
        margin: 10px 0;
    }
    .error-message {
        color: #c62828;
        background: #ffebee;
        padding: 15px;
        border-radius: 5px;
        border-left: 4px solid #f44336;
        margin: 15px 0;
    }
    .summary-box {
        background: #f8f9fa;
        padding: 15px;
        border-radius: 8px;
        border-left: 4px solid #0068c9;
        margin: 15px 0;
    }
    .risk-box {
        background: #f0f2f6;
        border-radius: 10px;
        padding: 15px;
        margin-bottom: 20px;
    }
    .risk-details {
        margin-top: 15px;
        background: white;
        padding: 10px;
        border-radius: 5px;
    }
    .vulnerability-summary {
        background: #fff8e1;
        border-left: 4px solid #ffc107;
        padding: 15px;
        margin: 20px 0;
        border-radius: 0 8px 8px 0;
    }
    .model-info {
        background: #e8f5e9;
        border-left: 4px solid #4caf50;
        padding: 15px;
        margin: 10px 0;
        border-radius: 0 8px 8px 0;
    }
    .countermeasures {
        margin-left: 10px;
    }
    .countermeasure-box {
        margin-bottom: 15px;
        padding: 15px;
        background: white;
        border-radius: 8px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.05);
    }
    .cm-header {
        display: flex;
        align-items: center;
        margin-bottom: 10px;
    }
    .cm-number {
        background: #0068c9;
        color: white;
        width: 28px;
        height: 28px;
        border-radius: 50%;
        display: flex;
        align-items: center;
        justify-content: center;
        margin-right: 10px;
        font-weight: bold;
    }
    .cm-title {
        flex-grow: 1;
        font-weight: bold;
        color: #0068c9;
    }
    .cm-effectiveness {
        padding: 4px 10px;
        border-radius: 20px;
        font-size: 0.8em;
        color: white;
    }
    .cm-implementation {
        margin-top: 8px;
        padding-left: 15px;
        border-left: 2px solid #e0e0e0;
        color: #555;
    }
    .roadmap-box {
        background: #f8f9fa;
        border-radius: 8px;
        padding: 15px;
        border: 1px solid #e0e0e0;
    }
    .verification-box {
        margin-top: 15px;
        padding: 15px;
        background: #e3f2fd;
        border-radius: 5px;
    }
    .download-link {
        display: inline-block;
        background: #0068c9;
        color: white;
        text-decoration: none;
        padding: 12px 24px;
        border-radius: 5px;
        font-weight: bold;
        margin: 15px 0;
        transition: background 0.3s;
    }
    .download-link:hover {
        background: #0056b3;
    }
    .fa {{
        font-family: FontAwesome;
    }
</style>
"""

# ======================
# UI LAYOUT
# ======================
display(widgets.HTML(styles))

display(widgets.VBox([
    widgets.HTML(
        "<div style='background:#0068c9; color:white; padding:20px; border-radius:8px 8px 0 0;'>"
        "<h1 style='text-align:center; margin:0;'>🔒 SideChannel Sentinel Pro</h1>"
        "<h3 style='text-align:center; margin:0; font-weight:normal;'>Advanced Side-Channel Vulnerability Detection</h3>"
        "</div>"
    ),
    
    widgets.HTML(
        "<div style='padding:15px; background:#f0f5ff; border-radius:0 0 8px 8px; margin-bottom:20px;'>"
        "<p style='text-align:center; margin:0;'>Signal Processing + CPA + IBM Granite for Comprehensive Hardware Security</p>"
        "</div>"
    ),
    
    widgets.VBox([
        widgets.HTML("<h3 style='color:#0068c9;'><i class='fa fa-database'></i> Data Source</h3>"),
        widgets.HBox([use_sample_data, upload_widget])
    ], layout=widgets.Layout(padding='10px', border='1px solid #e0e0e0', borderRadius='8px')),
    
    widgets.VBox([
        widgets.HTML("<h3 style='color:#0068c9;'><i class='fa fa-sliders'></i> Analysis Configuration</h3>"),
        sensitivity_slider,
        confidence_slider,
        model_selector
    ], layout=widgets.Layout(padding='10px', border='1px solid #e0e0e0', borderRadius='8px', margin='15px 0')),
    
    widgets.HTML(
        "<div style='background:#e8f5e9; padding:15px; border-radius:8px; margin:10px 0;'>"
        "<h4 style='color:#0068c9; margin-top:0;'><i class='fa fa-key'></i> IBM Watsonx Configuration</h4>"
        "<p>Using IBM credentials for Granite models:</p>"
        "<ul>"
        f"<li><b>Region:</b> us-south</li>"
        f"<li><b>Project ID:</b> {os.environ['IBM_PROJECT_ID']}</li>"
        f"<li><b>Available Models:</b> Granite 4.0 Tiny/Medium, Granite 8B Instruct</li>"
        "</ul>"
        "</div>"
    ),
    
    widgets.HBox([analyze_button], layout=widgets.Layout(justify_content='center', margin='10px 0')),
    
    power_trace_output,
    analysis_output,
    countermeasure_output
]))

# Automatically run with sample data
on_analyze_button_clicked(analyze_button)


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'google.colab'

In [7]:
# Step 1: Install required packages (run this in your terminal before executing the script)
# pip install seaborn ipywidgets plotly pandas numpy ibm-watson-machine-learning

# Step 2: Set up environment variables for IBM Watson
import os
os.environ["IBM_API_KEY"] = "RyIKeWjkIi-jx4F2uerWwUuInh6d9hoIJal8frtzKIMk"
os.environ["IBM_URL"] = "https://us-south.ml.cloud.ibm.com"
os.environ["IBM_PROJECT_ID"] = "5d59376b-9172-474f-ac91-c33d1f16bb7b"

# Step 3: Import libraries
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown, HTML
import base64
from io import BytesIO
import plotly.graph_objects as go
import time
import json
import re
import requests
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.wml_client_error import ApiRequestFailure, WMLClientError
import logging

# ======================
# LOGGING CONFIGURATION
# ======================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger('SideChannelSentinel')

# ======================
# IBM WATSONX CONFIGURATION
# ======================
class WatsonxConfig:
    @staticmethod
    def get_model(model_id, max_tokens=500, temperature=0.2):
        """Initialize IBM Granite model with credentials and retry logic"""
        for attempt in range(3):
            try:
                model = Model(
                    model_id=model_id,
                    credentials={
                        "apikey": os.environ["IBM_API_KEY"],
                        "url": os.environ["IBM_URL"]
                    },
                    project_id=os.environ["IBM_PROJECT_ID"],
                    params={
                        GenParams.DECODING_METHOD: "greedy",
                        GenParams.MAX_NEW_TOKENS: max_tokens,
                        GenParams.MIN_NEW_TOKENS: 50,
                        GenParams.TEMPERATURE: temperature,
                        GenParams.TOP_P: 0.9,
                        GenParams.TOP_K: 50
                    }
                )
                return model
            except (ApiRequestFailure, WMLClientError, requests.RequestException) as e:
                wait_time = (2 ** attempt) + 1
                logger.error(f"IBM API error (attempt {attempt+1}/3): {str(e)}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
            except Exception as e:
                logger.error(f"Unexpected error initializing model: {str(e)}")
                return None
        logger.error("Failed to initialize model after 3 attempts")
        return None

# ======================
# ENHANCED SIGNAL PROCESSING
# ======================
class AdvancedSignalAnalyzer:
    @staticmethod
    def detect_leakage_points(df, sensitivity=7, confidence=0.85):
        """Advanced leakage detection with multiple techniques"""
        logger.info("Starting leakage detection analysis")
        
        # 1. Statistical Analysis with Adaptive Thresholding
        window_size = max(10, int(len(df) * 0.02))  # Dynamic window size
        df['MA'] = df['Power Consumption (mA)'].rolling(window=window_size).mean()
        df['STD'] = df['Power Consumption (mA)'].rolling(window=window_size).std()
        
        # Adaptive threshold based on sensitivity
        sensitivity_multiplier = 0.5 + (sensitivity / 10) * 4.5  # 0.5 to 5.0
        df['Threshold'] = df['STD'].rolling(window=100, min_periods=1).median() * sensitivity_multiplier
        
        # 2. Simulated Correlation Power Analysis (CPA)
        # Generate hypothetical power model (Hamming weight model)
        hypothetical_power = AdvancedSignalAnalyzer.simulate_hamming_model(df)
        df['CPA_Correlation'] = df['Power Consumption (mA)'].rolling(window=window_size).corr(hypothetical_power)
        
        # 3. Combined detection logic
        high_deviation = df[(df['STD'] > df['Threshold']) | 
                            (df['CPA_Correlation'].abs() > confidence)]
        
        leakage_points = []
        if not high_deviation.empty:
            # Group consecutive points
            high_deviation['Group'] = (high_deviation.index.to_series().diff() > 1).cumsum()
            
            for _, group in high_deviation.groupby('Group'):
                peak_idx = group['STD'].idxmax()
                peak_time = df.loc[peak_idx, 'Time (ms)']
                peak_value = df.loc[peak_idx, 'Power Consumption (mA)']
                
                # Calculate feature vector for LLM
                features = {
                    'duration': group['Time (ms)'].max() - group['Time (ms)'].min(),
                    'avg_power': group['Power Consumption (mA)'].mean(),
                    'max_power': group['Power Consumption (mA)'].max(),
                    'cpa_corr': group['CPA_Correlation'].abs().mean(),
                    'std_dev': group['STD'].mean()
                }
                
                # Confidence score based on multiple factors
                conf_score = min(0.95, (features['std_dev'] / df['Threshold'].mean() * 0.4 + 
                                        features['cpa_corr'] * 0.6))
                
                leakage_points.append({
                    'time_ms': peak_time,
                    'power_value': peak_value,
                    'features': features,
                    'confidence': conf_score
                })
        
        logger.info(f"Detected {len(leakage_points)} potential leakage points")
        return leakage_points

    @staticmethod
    def simulate_hamming_model(df):
        """Simulate CPA Hamming weight model for demonstration"""
        # In real CPA, this would be based on intermediate values
        # Here we simulate with a synthetic pattern
        np.random.seed(42)
        base = np.sin(2 * np.pi * df['Time (ms)'] * 5) * 0.3
        spikes = np.zeros(len(df))
        
        # Add spikes at random positions to simulate operation points
        for _ in range(5):
            idx = np.random.randint(0, len(df))
            width = np.random.uniform(0.05, 0.2)
            spike = np.exp(-(df['Time (ms)'] - df.loc[idx, 'Time (ms)'])**2 / (2 * width**2))
            spikes += spike * np.random.uniform(0.5, 1.5)
        
        return base + spikes

    @staticmethod
    def calculate_correlation(leakage_points):
        """Calculate correlation metrics for leakage points"""
        if not leakage_points:
            return 0.0, "Low"
        
        # Weighted average of multiple factors
        total_risk = 0
        weights = {'std_dev': 0.4, 'cpa_corr': 0.5, 'duration': 0.1}
        
        for point in leakage_points:
            features = point['features']
            risk_score = (features['std_dev'] * weights['std_dev'] +
                          features['cpa_corr'] * weights['cpa_corr'] +
                          features['duration'] * weights['duration'])
            total_risk += risk_score * point['confidence']
        
        overall_risk = min(1.0, total_risk / len(leakage_points))
        
        # Determine risk level
        if overall_risk > 0.7:
            risk_level = "High"
        elif overall_risk > 0.5:
            risk_level = "Medium"
        else:
            risk_level = "Low"
            
        return overall_risk, risk_level

# ======================
# ENHANCED GRANITE MODELS INTEGRATION
# ======================
class EnhancedGraniteModels:
    @staticmethod
    def analyze_results(leakage_points, overall_risk, risk_level, model_choice="granite-4.0-tiny"):
        """Enhanced analysis with dynamic model selection"""
        model_id_map = {
            "tiny": "ibm/granite-4.0-tiny",
            "medium": "ibm/granite-4.0-medium",
            "instruct": "ibm/granite-8b-instruct"
        }
        
        model_id = model_id_map.get(model_choice.lower(), "ibm/granite-4.0-tiny")
        logger.info(f"Using model: {model_id} for vulnerability analysis")
        
        try:
            model = WatsonxConfig.get_model(model_id, max_tokens=600)
            if not model:
                return EnhancedGraniteModels.simulate_analysis_summary(leakage_points, overall_risk)
            
            # Prepare detailed leakage points description
            leakage_desc = "\n".join(
                f"- At {point['time_ms']:.2f} ms: "
                f"Power={point['power_value']:.2f}mA, "
                f"Duration={point['features']['duration']:.4f}ms, "
                f"CPA_Corr={point['features']['cpa_corr']:.2f}, "
                f"Confidence={point['confidence']:.2f}"
                for point in leakage_points
            ) if leakage_points else "No significant leakage points detected"
            
            # Enhanced prompt with more context
            prompt = f"""
            [INST] <<SYS>>
            You are a hardware security expert analyzing side-channel power traces. 
            Provide a technical assessment of these findings:

            DETECTION RESULTS:
            {leakage_desc}

            OVERALL RISK: {overall_risk:.2f} ({risk_level})
            
            ANALYSIS TASKS:
            1. Vulnerability summary (1-2 sentences)
            2. Identify cryptographic operations affected by leakage
            3. Assess exploit difficulty (Low/Medium/High)
            4. Business impact assessment
            5. Return response in JSON format:
            {{
                "vulnerability_summary": "Summary text",
                "affected_operations": ["operation1", "operation2"],
                "exploit_difficulty": "Low/Medium/High",
                "business_impact": "Impact explanation",
                "confidence_score": 0.0-1.0
            }}
            <</SYS>>
            [/INST]
            """
            
            # Generate analysis with retry
            response = EnhancedGraniteModels.retry_api_call(model.generate, prompt=prompt)
            analysis = response['results'][0]['generated_text']
            
            # Extract JSON safely
            result = EnhancedGraniteModels.extract_json(analysis)
            result['model_used'] = model_id.split('/')[-1]
            return result
        except Exception as e:
            logger.error(f"Analysis error: {str(e)}")
            return EnhancedGraniteModels.simulate_analysis_summary(leakage_points, overall_risk)
    
    @staticmethod
    def retry_api_call(func, *args, **kwargs):
        """Retry API call with exponential backoff"""
        for attempt in range(3):
            try:
                return func(*args, **kwargs)
            except (ApiRequestFailure, requests.RequestException) as e:
                wait_time = (2 ** attempt) + 1
                logger.warning(f"API call failed (attempt {attempt+1}/3): {str(e)}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
        raise Exception("API call failed after 3 attempts")

    @staticmethod
    def extract_json(text):
        """Robust JSON extraction from LLM response"""
        try:
            # Improved pattern to find JSON objects
            json_match = re.search(r'\{[\s\S]*\}', text)
            if not json_match:
                raise ValueError("No JSON found in response")
                
            json_str = json_match.group(0)
            return json.loads(json_str)
        except (ValueError, json.JSONDecodeError) as e:
            logger.error(f"JSON extraction error: {str(e)}")
            return {
                "vulnerability_summary": "Analysis summary unavailable",
                "affected_operations": [],
                "business_impact": "Unable to generate business impact assessment",
                "exploit_difficulty": "Unknown",
                "confidence_score": 0.0
            }
    
    @staticmethod
    def simulate_analysis_summary(leakage_points, overall_risk):
        """Fallback simulation if API fails"""
        summary = "Significant correlation detected between power consumption and secret data"
        operations = ["S-Box substitution", "Key schedule"]
        
        if overall_risk > 0.7:
            impact = "High risk of key extraction leading to complete system compromise"
            difficulty = "Medium (requires specialized equipment)"
        elif overall_risk > 0.5:
            impact = "Moderate risk of partial key recovery"
            difficulty = "High (requires advanced expertise)"
        else:
            impact = "Low risk in practical scenarios"
            difficulty = "Very High (theoretical risk only)"
        
        return {
            "vulnerability_summary": summary,
            "affected_operations": operations,
            "business_impact": impact,
            "exploit_difficulty": difficulty,
            "confidence_score": min(0.9, overall_risk + 0.2)
        }
    
    @staticmethod
    def generate_countermeasures(analysis, model_choice="granite-8b-instruct"):
        """Generate targeted countermeasures with model choice"""
        model_id_map = {
            "instruct": "ibm/granite-8b-instruct",
            "medium": "ibm/granite-4.0-medium",
            "tiny": "ibm/granite-4.0-tiny"
        }
        
        model_id = model_id_map.get(model_choice.lower(), "ibm/granite-8b-instruct")
        logger.info(f"Using model: {model_id} for countermeasures")
        
        try:
            model = WatsonxConfig.get_model(model_id, max_tokens=800, temperature=0.3)
            if not model:
                return EnhancedGraniteModels.simulate_countermeasures()
            
            # Prepare prompt with targeted operations
            operations = analysis.get('affected_operations', [])
            targeted_ops = f"Specifically target: {', '.join(operations)}" if operations else ""
            
            prompt = f"""
            [INST] <<SYS>>
            As a hardware security specialist, suggest countermeasures for these vulnerabilities:
            
            {analysis.get('vulnerability_summary', 'Vulnerability analysis unavailable')}
            
            Affected Operations: {', '.join(operations) or 'Not specified'}
            Exploit Difficulty: {analysis.get('exploit_difficulty', 'Unknown')}
            
            Requirements:
            1. Provide 5 hardware-focused countermeasures
            2. For each: 
               - Brief description
               - Implementation approach
               - Estimated effectiveness (High/Medium/Low)
            3. Format: 
               "1. [Description] 
                Implementation: [Details] 
                Effectiveness: [Rating]"
            4. Prioritize countermeasures addressing the specific operations mentioned
            {targeted_ops}
            <</SYS>>
            [/INST]
            """
            
            # Generate countermeasures
            response = EnhancedGraniteModels.retry_api_call(model.generate, prompt=prompt)
            text = response['results'][0]['generated_text']
            
            # Parse with enhanced regex
            return EnhancedGraniteModels.parse_countermeasures(text)
        except Exception as e:
            logger.error(f"Countermeasures error: {str(e)}")
            return EnhancedGraniteModels.simulate_countermeasures()
    
    @staticmethod
    def parse_countermeasures(text):
        """Robust parsing of countermeasures with effectiveness"""
        pattern = r"(\d+)\.\s*([^\n]+?)\n\s*Implementation:\s*(.*?)\n\s*Effectiveness:\s*(High|Medium|Low)"
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
        
        countermeasures = []
        implementations = []
        effectiveness = []
        
        for num, desc, impl, eff in matches[:5]:
            countermeasures.append(desc.strip())
            implementations.append(impl.strip())
            effectiveness.append(eff.strip())
        
        # Fill any missing with simulated
        sim_cm, sim_imp, sim_eff = EnhancedGraniteModels.simulate_countermeasures()
        for i in range(5 - len(countermeasures)):
            countermeasures.append(sim_cm[i])
            implementations.append(sim_imp[i])
            effectiveness.append(sim_eff[i])
            
        intro = f"Generated by IBM Granite model"
        return countermeasures, implementations, effectiveness, intro
    
    @staticmethod
    def simulate_countermeasures():
        """Fallback simulation if API fails"""
        countermeasures = [
            "Implement Boolean masking for S-Box operations",
            "Add dynamic noise generation circuits",
            "Use constant-time implementations for all operations",
            "Implement operation shuffling with PRNG",
            "Add random delays between cryptographic operations"
        ]
        
        implementations = [
            "Apply additive/multiplicative masking to intermediate values",
            "Use LFSR-based noise generator with adjustable amplitude",
            "Ensure all branches and memory accesses take fixed time",
            "Randomize processing order using hardware PRNG",
            "Insert delays with Poisson distribution between rounds"
        ]
        
        effectiveness = ["High", "Medium", "High", "Medium", "Low"]
        
        return countermeasures, implementations, effectiveness

# ======================
# ENHANCED VISUALIZATION
# ======================
def generate_sample_data():
    """Create realistic power trace simulation data with CPA features"""
    np.random.seed(42)
    time_points = np.linspace(0, 10, 1000)
    base_signal = np.sin(2 * np.pi * time_points) * 0.5
    
    # Add cryptographic operation signatures
    operation_points = [
        {"time_ms": 1.5, "magnitude": 1.8, "width": 0.2, "operation": "S-Box substitution"},
        {"time_ms": 3.2, "magnitude": 1.5, "width": 0.15, "operation": "Key schedule"},
        {"time_ms": 5.0, "magnitude": 0.8, "width": 0.1, "operation": "AddRoundKey"},
        {"time_ms": 7.8, "magnitude": 1.2, "width": 0.25, "operation": "Final round"}
    ]
    
    for point in operation_points:
        idx = np.abs(time_points - point['time_ms']).argmin()
        gaussian = point['magnitude'] * np.exp(-(time_points - point['time_ms'])**2 / (2 * point['width']**2))
        base_signal += gaussian
    
    # Add noise and high-frequency components
    base_signal += np.random.normal(0, 0.15, len(time_points))
    base_signal += 0.3 * np.sin(15 * np.pi * time_points)
    base_signal += 0.2 * np.random.random(len(time_points))
    
    return pd.DataFrame({
        "Time (ms)": time_points,
        "Power Consumption (mA)": base_signal
    }), operation_points

def plot_power_traces(df, leakage_points=None, operation_points=None):
    """Enhanced plot with interactive leakage points"""
    fig = go.Figure()
    
    # Main power trace
    fig.add_trace(go.Scatter(
        x=df["Time (ms)"],
        y=df["Power Consumption (mA)"],
        mode='lines',
        name='Power Consumption',
        line=dict(color='#0068c9', width=2),
        fill='tozeroy',
        fillcolor='rgba(0, 104, 201, 0.1)',
        hovertemplate='Time: %{x:.2f} ms<br>Power: %{y:.2f} mA<extra></extra>'
    ))
    
    # Known operation points
    if operation_points:
        for point in operation_points:
            fig.add_vline(
                x=point['time_ms'],
                line=dict(color="#888", width=1.5, dash="dot"),
                annotation_text=point["operation"],
                annotation_position="top right",
                annotation_font_size=10
            )
    
    # Detected leakage points
    if leakage_points:
        leak_x = [point['time_ms'] for point in leakage_points]
        leak_y = [point['power_value'] for point in leakage_points]
        leak_text = [
            f"<b>Leakage Point</b><br>Time: {p['time_ms']:.2f} ms<br>Power: {p['power_value']:.2f} mA"
            f"<br>Confidence: {p['confidence']:.2f}<br>CPA Corr: {p['features']['cpa_corr']:.2f}"
            for p in leakage_points
        ]
        
        fig.add_trace(go.Scatter(
            x=leak_x,
            y=leak_y,
            mode='markers',
            name='Leakage Detected',
            marker=dict(size=12, color='red', symbol='x'),
            hovertemplate='%{text}<extra></extra>',
            text=leak_text
        ))
    
    fig.update_layout(
        title='Power Consumption Analysis',
        xaxis_title='Time (ms)',
        yaxis_title='Power Consumption (mA)',
        template='plotly_white',
        height=450,
        hovermode="x unified",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    return fig

def risk_meter(value):
    """Create a visual risk meter with improved design"""
    value = min(100, max(0, value))
    level = min(3, int(value // 25))
    colors = ['#00c853', '#ffd600', '#ff9100', '#ff3d00']
    
    html = f"""
    <div style="background: #f8f9fa; border-radius: 10px; padding: 15px; margin: 10px 0; position: relative; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
        <div style="position: absolute; top: 15px; right: 15px; font-weight: bold; font-size: 24px; color: {colors[level]}">
            {value:.1f}%
        </div>
        <div style="margin-top: 10px;">
            <div style="display: flex; justify-content: space-between; margin-bottom: 5px; font-size: 12px;">
                <span>Low</span>
                <span>Medium</span>
                <span>High</span>
                <span>Critical</span>
            </div>
            <div style="background: #e0e0e0; height: 20px; border-radius: 10px; overflow: hidden;">
                <div style="background: linear-gradient(to right, {', '.join(colors)}); 
                            width: {value}%; height: 100%; 
                            position: relative;">
                </div>
            </div>
        </div>
        <div style="margin-top: 15px; font-size: 14px; color: #666;">
            Risk level indicates likelihood of key extraction
        </div>
    </div>
    """
    return html

def create_download_link(df, filename="power_traces.csv"):
    """Generate a download link for CSV file"""
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    return f'<a href="data:file/csv;base64,{b64}" download="{filename}" style="color: #0068c9; text-decoration: none; border: 1px solid #0068c9; border-radius: 4px; padding: 6px 12px; display: inline-block; margin: 10px 0;">Download CSV File</a>'

# ======================
# ENHANCED UI MODULE
# ======================
# Create widgets
use_sample_data = widgets.Checkbox(
    value=True,
    description='Use sample data for demonstration',
    indent=False,
    style={'description_width': 'initial'}
)

upload_widget = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Upload CSV',
    layout=widgets.Layout(width='300px'),
    tooltip="CSV should contain 'Time (ms)' and 'Power Consumption (mA)' columns"
)

sensitivity_slider = widgets.IntSlider(
    value=5,  # Default changed to 5 (maps to 2.5 threshold multiplier)
    min=1,
    max=10,
    step=1,
    description='Detection Sensitivity:',
    style={'description_width': 'initial'},
    continuous_update=False,
    tooltip="Higher values detect smaller leaks but may increase false positives"
)

confidence_slider = widgets.IntSlider(
    value=85,
    min=50,
    max=100,
    step=5,
    description='Min Confidence Threshold:',
    style={'description_width': 'initial'},
    continuous_update=False,
    tooltip="Minimum confidence score to consider a leakage point valid"
)

model_selector = widgets.Dropdown(
    options=[('Granite 4.0 Tiny', 'tiny'), 
             ('Granite 4.0 Medium', 'medium'),
             ('Granite 8B Instruct', 'instruct')],
    value='tiny',
    description='Analysis Model:',
    style={'description_width': 'initial'},
    tooltip="Select IBM Granite model for vulnerability analysis"
)

analyze_button = widgets.Button(
    description='Analyze Power Traces',
    button_style='success',
    icon='search',
    layout=widgets.Layout(width='200px', height='40px')
)

export_button = widgets.Button(
    description='Export Full Report',
    button_style='info',
    icon='file-download',
    layout=widgets.Layout(width='200px', height='40px'),
    disabled=True
)

# Create output areas
power_trace_output = widgets.Output()
analysis_output = widgets.Output()
countermeasure_output = widgets.Output()

# ======================
# MAIN FUNCTIONALITY
# ======================
# Global state for report generation
analysis_results = {}
report_data = {}

def on_analyze_button_clicked(b):
    global analysis_results, report_data
    # Reset outputs and state
    power_trace_output.clear_output()
    analysis_output.clear_output()
    countermeasure_output.clear_output()
    analysis_results = {}
    report_data = {}
    export_button.disabled = True
    
    df = None
    operation_points = None
    file_uploaded = False
    
    with power_trace_output:
        # Show detailed progress
        display(widgets.HTML("<div class='progress-step'><b>Step 1/4:</b> Processing input data...</div>"))
        
        # Handle data input
        if upload_widget.value:
            try:
                uploaded = list(upload_widget.value.values())[0]
                content = uploaded['content'].decode('utf-8')
                df = pd.read_csv(BytesIO(content.encode()))
                
                # Validate data types
                if not pd.api.types.is_numeric_dtype(df['Time (ms)']):
                    raise ValueError("Time column must be numeric")
                if not pd.api.types.is_numeric_dtype(df['Power Consumption (mA)']):
                    raise ValueError("Power column must be numeric")
                
                # Check data ranges
                if df['Time (ms)'].min() < 0 or df['Time (ms)'].max() > 10000:
                    raise ValueError("Time values out of expected range (0-10000 ms)")
                if df['Power Consumption (mA)'].abs().max() > 1000:
                    raise ValueError("Power values exceed reasonable range (±1000 mA)")
                
                file_uploaded = True
                display(widgets.HTML("<div class='success-message'>CSV file successfully uploaded and validated!</div>"))
            except Exception as e:
                display(widgets.HTML(f"<div class='error-message'><b>Data Validation Error:</b> {str(e)}</div>"))
                return
        elif use_sample_data.value:
            df, operation_points = generate_sample_data()
        
        # Validate data
        if df is None or df.empty:
            display(widgets.HTML("<div class='error-message'>No data available. Please upload a file or use sample data.</div>"))
            return
        
        # Check required columns
        required_columns = ["Time (ms)", "Power Consumption (mA)"]
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            display(widgets.HTML(
                f"<div class='error-message'><b>Missing columns:</b> {', '.join(missing)}<br>"
                f"Required columns: {', '.join(required_columns)}</div>"
            ))
            return
        
        # Update progress
        power_trace_output.clear_output()
        display(widgets.HTML("<div class='success-message'>Data processed successfully</div>"))
        display(widgets.HTML("<h3 style='color:#0068c9; border-bottom: 1px solid #e0e0e0; padding-bottom: 10px;'>Power Trace Data</h3>"))
        
        if use_sample_data.value:
            display(widgets.HTML("<p style='color: #666;'>Simulated AES-256 cryptographic operation data</p>"))
            display(widgets.HTML(create_download_link(df)))
        
        # Plot power traces
        fig = plot_power_traces(df, operation_points=operation_points)
        display(fig)
        
        # Show data summary
        display(widgets.HTML(f"""
        <div class='summary-box'>
            <h4 style='color:#0068c9; margin-top:0;'>Data Summary</h4>
            <ul>
                <li><b>Time Points:</b> {len(df)}</li>
                <li><b>Duration:</b> {df['Time (ms)'].iloc[-1]:.2f} ms</li>
                <li><b>Avg Power:</b> {df['Power Consumption (mA)'].mean():.2f} mA</li>
                <li><b>Max Power:</b> {df['Power Consumption (mA)'].max():.2f} mA</li>
                <li><b>Data Source:</b> {"User Upload" if file_uploaded else "Sample Data"}</li>
            </ul>
        </div>
        """))
        
        # Store for report
        report_data['df'] = df
        report_data['operation_points'] = operation_points
        report_data['file_uploaded'] = file_uploaded
    
    with analysis_output:
        display(widgets.HTML("<div class='progress-step'><b>Step 3/4:</b> Analyzing power traces...</div>"))
        
        # Run signal analysis
        leakage_points = AdvancedSignalAnalyzer.detect_leakage_points(
            df, 
            sensitivity=sensitivity_slider.value,
            confidence=confidence_slider.value/100
        )
        overall_risk, risk_level = AdvancedSignalAnalyzer.calculate_correlation(leakage_points)
        
        # Update progress
        analysis_output.clear_output()
        display(widgets.HTML("<div class='success-message'>Signal analysis completed</div>"))
        display(widgets.HTML("<div class='progress-step'><b>Step 4/4:</b> Synthesizing results with IBM Granite...</div>"))
        
        # Use Granite for interpretation
        analysis = EnhancedGraniteModels.analyze_results(
            leakage_points, 
            overall_risk, 
            risk_level,
            model_choice=model_selector.value
        )
        
        # Store results for report
        analysis_results = {
            'leakage_points': leakage_points,
            'overall_risk': overall_risk,
            'risk_level': risk_level,
            'analysis': analysis
        }
        
        # Clear loading and show results
        analysis_output.clear_output()
        
        display(widgets.HTML(f"""
        <h3 style='color:#0068c9; border-bottom: 1px solid #e0e0e0; padding-bottom: 10px;'>
            Vulnerability Analysis Results
            <span style='font-size: 0.6em; color: #666; float: right;'>
                Sens: {sensitivity_slider.value}/10 | Conf: {confidence_slider.value}% | Model: {analysis.get('model_used', 'Granite')}
            </span>
        </h3>
        """))
        
        # Risk assessment
        risk_score = overall_risk * 100
        risk_color = "#ff3d00" if risk_level == "High" else "#ff9100" if risk_level == "Medium" else "#00c853"
        
        display(widgets.HTML(f"""
        <div class='risk-box'>
            <h4 style='color:#0068c9; margin-top:0;'>Risk Assessment</h4>
            {risk_meter(risk_score)}
            <div class='risk-details'>
                <table>
                    <tr>
                        <td><b>Risk Level:</b></td>
                        <td style='color:{risk_color}; font-weight:bold;'>{risk_level}</td>
                        <td><b>Analysis Model:</b></td>
                        <td>{analysis.get('model_used', 'IBM Granite')}</td>
                    </tr>
                    <tr>
                        <td><b>Exploit Difficulty:</b></td>
                        <td>{analysis.get('exploit_difficulty', 'Unknown')}</td>
                        <td><b>Confidence Score:</b></td>
                        <td>{analysis.get('confidence_score', 0.0):.2f}</td>
                    </tr>
                </table>
            </div>
        </div>
        """))
        
        # Vulnerability summary
        display(widgets.HTML(f"""
        <div class='vulnerability-summary'>
            <h4 style='margin-top:0; color:#ff8f00;'>Vulnerability Summary</h4>
            <p>{analysis.get("vulnerability_summary", "Summary unavailable")}</p>
        </div>
        """))
        
        # Affected operations
        if analysis.get("affected_operations"):
            display(widgets.HTML("<h4 style='color:#0068c9;'>Affected Cryptographic Operations</h4>"))
            ops_html = "<ul>"
            for op in analysis["affected_operations"]:
                ops_html += f"<li>{op}</li>"
            ops_html += "</ul>"
            display(widgets.HTML(ops_html))
        
        # Business risk
        display(widgets.HTML("<h4 style='color:#0068c9; margin-top:20px;'>Business Impact</h4>"))
        display(widgets.HTML(f"<p>{analysis.get('business_impact', 'Risk assessment unavailable')}</p>"))
        
        # Enable export button
        export_button.disabled = False
    
    with countermeasure_output:
        display(widgets.HTML("<div class='progress-step'>Generating security recommendations...</div>"))
        
        # Generate countermeasures
        countermeasures, implementations, effectiveness, intro = EnhancedGraniteModels.generate_countermeasures(
            analysis,
            model_choice=model_selector.value
        )
        
        # Store for report
        analysis_results['countermeasures'] = countermeasures
        analysis_results['implementations'] = implementations
        analysis_results['effectiveness'] = effectiveness
        
        # Clear loading and show results
        countermeasure_output.clear_output()
        
        display(widgets.HTML(f"""
        <h3 style='color:#0068c9; border-bottom: 1px solid #e0e0e0; padding-bottom: 10px;'>
            Security Recommendations
            <span style='font-size: 0.6em; color: #666; float: right;'>
                {intro}
            </span>
        </h3>
        """))
        
        display(widgets.HTML(f"""
        <div class='model-info'>
            <p><b>Countermeasure model:</b> {model_selector.label}</p>
        </div>
        """))
        
        # Countermeasures
        display(widgets.HTML("<h4 style='color:#0068c9;'>Recommended Countermeasures</h4>"))
        
        cm_html = "<div class='countermeasures'>"
        for i, (cm, imp, eff) in enumerate(zip(countermeasures, implementations, effectiveness), 1):
            eff_color = "#4caf50" if eff == "High" else "#ff9800" if eff == "Medium" else "#f44336"
            cm_html += f"""
            <div class='countermeasure-box'>
                <div class='cm-header'>
                    <span class='cm-number'>#{i}</span>
                    <span class='cm-title'>{cm}</span>
                    <span class='cm-effectiveness' style='background-color:{eff_color};'>{eff}</span>
                </div>
                <div class='cm-implementation'>
                    <b>Implementation:</b> {imp}
                </div>
            </div>
            """
        cm_html += "</div>"
        
        display(widgets.HTML(cm_html))
        
        # Implementation guidance
        display(widgets.HTML("""
        <h4 style='color:#0068c9; margin-top:25px;'>Implementation Roadmap</h4>
        <div class='roadmap-box'>
            <table>
                <tr class='header-row'>
                    <th>Phase</th>
                    <th>Activities</th>
                    <th>Timeline</th>
                </tr>
                <tr>
                    <td><b>Phase 1</b></td>
                    <td>Implement masking and shuffling countermeasures</td>
                    <td style='text-align:center;'>2-4 weeks</td>
                </tr>
                <tr class='alt-row'>
                    <td><b>Phase 2</b></td>
                    <td>Add noise generation circuits and constant-time implementations</td>
                    <td style='text-align:center;'>3-5 weeks</td>
                </tr>
                <tr>
                    <td><b>Phase 3</b></td>
                    <td>Verification and penetration testing</td>
                    <td style='text-align:center;'>2-3 weeks</td>
                </tr>
            </table>
            
            <div class='verification-box'>
                <b>Verification Steps:</b>
                <ol>
                    <li>Re-run power analysis after each mitigation</li>
                    <li>Perform correlation coefficient validation</li>
                    <li>Conformal equivalence checking</li>
                    <li>100% coverage testing of vulnerable operations</li>
                    <li>Penetration testing with real-world attack scenarios</li>
                </ol>
            </div>
        </div>
        """))
        
        # Add export button
        display(export_button)

def on_export_button_clicked(b):
    """Generate comprehensive HTML report"""
    if not analysis_results or not report_data:
        with countermeasure_output:
            display(widgets.HTML("<div class='error-message'>No analysis data available for export</div>"))
        return
    
    try:
        # Generate HTML report
        html_content = generate_html_report(report_data, analysis_results)
        
        # Create download link
        b64 = base64.b64encode(html_content.encode()).decode()
        url = f'data:text/html;base64,{b64}'
        
        with countermeasure_output:
            display(widgets.HTML(
                f"<div class='success-message'>Report generated successfully!</div>"
                f"<a href='{url}' download='sidechannel_report.html' class='download-link'>"
                "Download Full Report"
                "</a>"
            ))
    except Exception as e:
        with countermeasure_output:
            display(widgets.HTML(f"<div class='error-message'>Report generation failed: {str(e)}</div>"))

def generate_html_report(report_data, analysis_results):
    """Generate comprehensive HTML report"""
    # Create plot image
    fig = plot_power_traces(
        report_data['df'], 
        leakage_points=analysis_results.get('leakage_points', []),
        operation_points=report_data.get('operation_points', [])
    )
    plot_html = fig.to_html(full_html=False, include_plotlyjs='cdn')
    
    # Risk meter
    risk_score = analysis_results.get('overall_risk', 0) * 100
    risk_level = analysis_results.get('risk_level', 'Low')
    
    # Countermeasures table
    cm_table = ""
    for i, (cm, imp, eff) in enumerate(zip(
        analysis_results.get('countermeasures', []),
        analysis_results.get('implementations', []),
        analysis_results.get('effectiveness', [])
    ), 1):
        eff_color = "green" if eff == "High" else "orange" if eff == "Medium" else "red"
        cm_table += f"""
        <tr>
            <td>{i}</td>
            <td>{cm}</td>
            <td>{imp}</td>
            <td style="color:{eff_color}">{eff}</td>
        </tr>
        """
    
    # Create HTML structure
    return f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>SideChannel Sentinel Report</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; }}
            .container {{ max-width: 1200px; margin: 0 auto; padding: 20px; }}
            .header {{ background: #0068c9; color: white; padding: 30px; text-align: center; border-radius: 8px; }}
            .section {{ margin: 30px 0; padding: 20px; background: white; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.05); }}
            .risk-meter {{ background: #f8f9fa; padding: 20px; border-radius: 8px; }}
            table {{ width: 100%; border-collapse: collapse; }}
            th, td {{ padding: 12px 15px; text-align: left; border-bottom: 1px solid #ddd; }}
            th {{ background-color: #f5f5f5; }}
            .countermeasure-row:nth-child(even) {{ background-color: #f9f9f9; }}
            .plot-container {{ margin: 20px 0; }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>SideChannel Sentinel Report</h1>
                <p>Comprehensive Side-Channel Vulnerability Analysis</p>
                <p>Generated on {time.strftime("%Y-%m-%d %H:%M:%S")}</p>
            </div>
            
            <div class="section">
                <h2>Executive Summary</h2>
                <p><b>Overall Risk:</b> <span style="color:{"#ff3d00" if risk_level=="High" else "#ff9100" if risk_level=="Medium" else "#00c853"}">{risk_level}</span> ({risk_score:.1f}%)</p>
                <p><b>Vulnerability Summary:</b> {analysis_results.get('analysis', {}).get('vulnerability_summary', '')}</p>
                <p><b>Business Impact:</b> {analysis_results.get('analysis', {}).get('business_impact', '')}</p>
            </div>
            
            <div class="section">
                <h2>Power Trace Analysis</h2>
                <div class="plot-container">
                    {plot_html}
                </div>
            </div>
            
            <div class="section">
                <h2>Detailed Vulnerability Assessment</h2>
                <p><b>Affected Operations:</b> {', '.join(analysis_results.get('analysis', {}).get('affected_operations', [])) or 'None detected'}</p>
                <p><b>Exploit Difficulty:</b> {analysis_results.get('analysis', {}).get('exploit_difficulty', 'Unknown')}</p>
                <p><b>Analysis Confidence:</b> {analysis_results.get('analysis', {}).get('confidence_score', 0.0)*100:.1f}%</p>
            </div>
            
            <div class="section">
                <h2>Security Recommendations</h2>
                <table>
                    <tr>
                        <th>#</th>
                        <th>Countermeasure</th>
                        <th>Implementation</th>
                        <th>Effectiveness</th>
                    </tr>
                    {cm_table}
                </table>
            </div>
            
            <div class="section">
                <h2>Implementation Roadmap</h2>
                <table>
                    <tr>
                        <th>Phase</th>
                        <th>Activities</th>
                        <th>Timeline</th>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 1</td>
                        <td>Implement critical countermeasures</td>
                        <td>2-4 weeks</td>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 2</td>
                        <td>Hardware modifications and verification</td>
                        <td>3-5 weeks</td>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 3</td>
                        <td>Penetration testing and validation</td>
                        <td>2-3 weeks</td>
                    </tr>
                </table>
            </div>
        </div>
    </body>
    </html>
    """

# Bind button clicks
analyze_button.on_click(on_analyze_button_clicked)
export_button.on_click(on_export_button_clicked)

# ======================
# STYLES AND UX ENHANCEMENTS
# ======================
styles = """
<style>
    .progress-step {
        background: #e3f2fd;
        padding: 10px 15px;
        border-radius: 5px;
        margin: 10px 0;
        border-left: 4px solid #0068c9;
    }
    .success-message {
        color: #2e7d32;
        background: #e8f5e9;
        padding: 10px;
        border-radius: 5px;
        border-left: 4px solid #4caf50;
        margin: 10px 0;
    }
    .error-message {
        color: #c62828;
        background: #ffebee;
        padding: 15px;
        border-radius: 5px;
        border-left: 4px solid #f44336;
        margin: 15px 0;
    }
    .summary-box {
        background: #f8f9fa;
        padding: 15px;
        border-radius: 8px;
        border-left: 4px solid #0068c9;
        margin: 15px 0;
    }
    .risk-box {
        background: #f0f2f6;
        border-radius: 10px;
        padding: 15px;
        margin-bottom: 20px;
    }
    .risk-details {
        margin-top: 15px;
        background: white;
        padding: 10px;
        border-radius: 5px;
    }
    .vulnerability-summary {
        background: #fff8e1;
        border-left: 4px solid #ffc107;
        padding: 15px;
        margin: 20px 0;
        border-radius: 0 8px 8px 0;
    }
    .model-info {
        background: #e8f5e9;
        border-left: 4px solid #4caf50;
        padding: 15px;
        margin: 10px 0;
        border-radius: 0 8px 8px 0;
    }
    .countermeasures {
        margin-left: 10px;
    }
    .countermeasure-box {
        margin-bottom: 15px;
        padding: 15px;
        background: white;
        border-radius: 8px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.05);
    }
    .cm-header {
        display: flex;
        align-items: center;
        margin-bottom: 10px;
    }
    .cm-number {
        background: #0068c9;
        color: white;
        width: 28px;
        height: 28px;
        border-radius: 50%;
        display: flex;
        align-items: center;
        justify-content: center;
        margin-right: 10px;
        font-weight: bold;
    }
    .cm-title {
        flex-grow: 1;
        font-weight: bold;
        color: #0068c9;
    }
    .cm-effectiveness {
        padding: 4px 10px;
        border-radius: 20px;
        font-size: 0.8em;
        color: white;
    }
    .cm-implementation {
        margin-top: 8px;
        padding-left: 15px;
        border-left: 2px solid #e0e0e0;
        color: #555;
    }
    .roadmap-box {
        background: #f8f9fa;
        border-radius: 8px;
        padding: 15px;
        border: 1px solid #e0e0e0;
    }
    .verification-box {
        margin-top: 15px;
        padding: 15px;
        background: #e3f2fd;
        border-radius: 5px;
    }
    .download-link {
        display: inline-block;
        background: #0068c9;
        color: white;
        text-decoration: none;
        padding: 12px 24px;
        border-radius: 5px;
        font-weight: bold;
        margin: 15px 0;
        transition: background 0.3s;
    }
    .download-link:hover {
        background: #0056b3;
    }
</style>
"""

# ======================
# UI LAYOUT
# ======================
display(widgets.HTML(styles))

display(widgets.VBox([
    widgets.HTML(
        "<div style='background:#0068c9; color:white; padding:20px; border-radius:8px 8px 0 0;'>"
        "<h1 style='text-align:center; margin:0;'>🔒 SideChannel Sentinel Pro</h1>"
        "<h3 style='text-align:center; margin:0; font-weight:normal;'>Advanced Side-Channel Vulnerability Detection</h3>"
        "</div>"
    ),
    
    widgets.HTML(
        "<div style='padding:15px; background:#f0f5ff; border-radius:0 0 8px 8px; margin-bottom:20px;'>"
        "<p style='text-align:center; margin:0;'>Signal Processing + CPA + IBM Granite for Comprehensive Hardware Security</p>"
        "</div>"
    ),
    
    widgets.VBox([
        widgets.HTML("<h3 style='color:#0068c9;'>Data Source</h3>"),
        widgets.HBox([use_sample_data, upload_widget])
    ], layout=widgets.Layout(padding='10px', border='1px solid #e0e0e0', borderRadius='8px')),
    
    widgets.VBox([
        widgets.HTML("<h3 style='color:#0068c9;'>Analysis Configuration</h3>"),
        sensitivity_slider,
        confidence_slider,
        model_selector
    ], layout=widgets.Layout(padding='10px', border='1px solid #e0e0e0', borderRadius='8px', margin='15px 0')),
    
    widgets.HTML(
        "<div style='background:#e8f5e9; padding:15px; border-radius:8px; margin:10px 0;'>"
        "<h4 style='color:#0068c9; margin-top:0;'>IBM Watsonx Configuration</h4>"
        "<p>Using IBM credentials for Granite models:</p>"
        "<ul>"
        f"<li><b>Region:</b> us-south</li>"
        f"<li><b>Project ID:</b> {os.environ['IBM_PROJECT_ID']}</li>"
        f"<li><b>Available Models:</b> Granite 4.0 Tiny/Medium, Granite 8B Instruct</li>"
        "</ul>"
        "</div>"
    ),
    
    widgets.HBox([analyze_button], layout=widgets.Layout(justify_content='center', margin='10px 0')),
    
    power_trace_output,
    analysis_output,
    countermeasure_output
]))

# Automatically run with sample data
on_analyze_button_clicked(analyze_button)

HTML(value="\n        <h3 style='color:#0068c9; border-bottom: 1px solid #e0e0e0; padding-bottom: 10px;'>\n   …

HTML(value='\n        <div class=\'risk-box\'>\n            <h4 style=\'color:#0068c9; margin-top:0;\'>Risk As…

HTML(value="\n        <div class='vulnerability-summary'>\n            <h4 style='margin-top:0; color:#ff8f00;…

HTML(value="<h4 style='color:#0068c9;'>Affected Cryptographic Operations</h4>")

HTML(value='<ul><li>S-Box substitution</li><li>Key schedule</li></ul>')

HTML(value="<h4 style='color:#0068c9; margin-top:20px;'>Business Impact</h4>")

HTML(value='<p>Low risk in practical scenarios</p>')

HTML(value="<div class='progress-step'>Generating security recommendations...</div>")

2025-06-28 10:48:36,788 - SideChannelSentinel - INFO - Using model: ibm/granite-4.0-tiny for countermeasures
2025-06-28 10:48:38,194 - ibm_watson_machine_learning.client - INFO - Client successfully initialized
2025-06-28 10:48:39,694 - ibm_watson_machine_learning.wml_client_error - WARNING - Model 'ibm/granite-4.0-tiny' is not supported for this environment. Supported models: ['cross-encoder/ms-marco-minilm-l-12-v2', 'google/flan-t5-xl', 'google/flan-t5-xxl', 'google/flan-ul2', 'ibm/granite-13b-instruct-v2', 'ibm/granite-20b-code-instruct', 'ibm/granite-3-1-8b-base', 'ibm/granite-3-2-8b-instruct', 'ibm/granite-3-2b-instruct', 'ibm/granite-3-3-8b-instruct', 'ibm/granite-3-8b-instruct', 'ibm/granite-34b-code-instruct', 'ibm/granite-3b-code-instruct', 'ibm/granite-8b-code-instruct', 'ibm/granite-embedding-107m-multilingual', 'ibm/granite-embedding-278m-multilingual', 'ibm/granite-guardian-3-2b', 'ibm/granite-guardian-3-8b', 'ibm/granite-ttm-1024-96-r2', 'ibm/granite-ttm-1536-96-r2', 'ibm

ValueError: not enough values to unpack (expected 4, got 3)

In [9]:
# Step 1: Install required packages (run this in your terminal before executing the script)
# pip install seaborn ipywidgets plotly pandas numpy ibm-watson-machine-learning

# Step 2: Set up environment variables for IBM Watson
import os
os.environ["IBM_API_KEY"] = "RyIKeWjkIi-jx4F2uerWwUuInh6d9hoIJal8frtzKIMk"
os.environ["IBM_URL"] = "https://us-south.ml.cloud.ibm.com"
os.environ["IBM_PROJECT_ID"] = "5d59376b-9172-474f-ac91-c33d1f16bb7b"

# Step 3: Import libraries
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown, HTML
import base64
from io import BytesIO
import plotly.graph_objects as go
import time
import json
import re
import requests
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.wml_client_error import ApiRequestFailure, WMLClientError
import logging

# ======================
# LOGGING CONFIGURATION
# ======================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger('SideChannelSentinel')

# ======================
# IBM WATSONX CONFIGURATION
# ======================
class WatsonxConfig:
    @staticmethod
    def get_model(model_id, max_tokens=500, temperature=0.2):
        """Initialize IBM Granite model with credentials and retry logic"""
        for attempt in range(3):
            try:
                model = Model(
                    model_id=model_id,
                    credentials={
                        "apikey": os.environ["IBM_API_KEY"],
                        "url": os.environ["IBM_URL"]
                    },
                    project_id=os.environ["IBM_PROJECT_ID"],
                    params={
                        GenParams.DECODING_METHOD: "greedy",
                        GenParams.MAX_NEW_TOKENS: max_tokens,
                        GenParams.MIN_NEW_TOKENS: 50,
                        GenParams.TEMPERATURE: temperature,
                        GenParams.TOP_P: 0.9,
                        GenParams.TOP_K: 50
                    }
                )
                return model
            except (ApiRequestFailure, WMLClientError, requests.RequestException) as e:
                wait_time = (2 ** attempt) + 1
                logger.error(f"IBM API error (attempt {attempt+1}/3): {str(e)}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
            except Exception as e:
                logger.error(f"Unexpected error initializing model: {str(e)}")
                return None
        logger.error("Failed to initialize model after 3 attempts")
        return None

# ======================
# ENHANCED SIGNAL PROCESSING
# ======================
class AdvancedSignalAnalyzer:
    @staticmethod
    def detect_leakage_points(df, sensitivity=7, confidence=0.85):
        """Advanced leakage detection with multiple techniques"""
        logger.info("Starting leakage detection analysis")
        
        # 1. Statistical Analysis with Adaptive Thresholding
        window_size = max(10, int(len(df) * 0.02))  # Dynamic window size
        df['MA'] = df['Power Consumption (mA)'].rolling(window=window_size).mean()
        df['STD'] = df['Power Consumption (mA)'].rolling(window=window_size).std()
        
        # Adaptive threshold based on sensitivity
        sensitivity_multiplier = 0.5 + (sensitivity / 10) * 4.5  # 0.5 to 5.0
        df['Threshold'] = df['STD'].rolling(window=100, min_periods=1).median() * sensitivity_multiplier
        
        # 2. Simulated Correlation Power Analysis (CPA)
        # Generate hypothetical power model (Hamming weight model)
        hypothetical_power = AdvancedSignalAnalyzer.simulate_hamming_model(df)
        df['CPA_Correlation'] = df['Power Consumption (mA)'].rolling(window=window_size).corr(hypothetical_power)
        
        # 3. Combined detection logic
        high_deviation = df[(df['STD'] > df['Threshold']) | 
                            (df['CPA_Correlation'].abs() > confidence)]
        
        leakage_points = []
        if not high_deviation.empty:
            # Group consecutive points
            high_deviation['Group'] = (high_deviation.index.to_series().diff() > 1).cumsum()
            
            for _, group in high_deviation.groupby('Group'):
                peak_idx = group['STD'].idxmax()
                peak_time = df.loc[peak_idx, 'Time (ms)']
                peak_value = df.loc[peak_idx, 'Power Consumption (mA)']
                
                # Calculate feature vector for LLM
                features = {
                    'duration': group['Time (ms)'].max() - group['Time (ms)'].min(),
                    'avg_power': group['Power Consumption (mA)'].mean(),
                    'max_power': group['Power Consumption (mA)'].max(),
                    'cpa_corr': group['CPA_Correlation'].abs().mean(),
                    'std_dev': group['STD'].mean()
                }
                
                # Confidence score based on multiple factors
                conf_score = min(0.95, (features['std_dev'] / df['Threshold'].mean() * 0.4 + 
                                        features['cpa_corr'] * 0.6))
                
                leakage_points.append({
                    'time_ms': peak_time,
                    'power_value': peak_value,
                    'features': features,
                    'confidence': conf_score
                })
        
        logger.info(f"Detected {len(leakage_points)} potential leakage points")
        return leakage_points

    @staticmethod
    def simulate_hamming_model(df):
        """Simulate CPA Hamming weight model for demonstration"""
        # In real CPA, this would be based on intermediate values
        # Here we simulate with a synthetic pattern
        np.random.seed(42)
        base = np.sin(2 * np.pi * df['Time (ms)'] * 5) * 0.3
        spikes = np.zeros(len(df))
        
        # Add spikes at random positions to simulate operation points
        for _ in range(5):
            idx = np.random.randint(0, len(df))
            width = np.random.uniform(0.05, 0.2)
            spike = np.exp(-(df['Time (ms)'] - df.loc[idx, 'Time (ms)'])**2 / (2 * width**2))
            spikes += spike * np.random.uniform(0.5, 1.5)
        
        return base + spikes

    @staticmethod
    def calculate_correlation(leakage_points):
        """Calculate correlation metrics for leakage points"""
        if not leakage_points:
            return 0.0, "Low"
        
        # Weighted average of multiple factors
        total_risk = 0
        weights = {'std_dev': 0.4, 'cpa_corr': 0.5, 'duration': 0.1}
        
        for point in leakage_points:
            features = point['features']
            risk_score = (features['std_dev'] * weights['std_dev'] +
                          features['cpa_corr'] * weights['cpa_corr'] +
                          features['duration'] * weights['duration'])
            total_risk += risk_score * point['confidence']
        
        overall_risk = min(1.0, total_risk / len(leakage_points))
        
        # Determine risk level
        if overall_risk > 0.7:
            risk_level = "High"
        elif overall_risk > 0.5:
            risk_level = "Medium"
        else:
            risk_level = "Low"
            
        return overall_risk, risk_level

# ======================
# ENHANCED GRANITE MODELS INTEGRATION
# ======================
class EnhancedGraniteModels:
    @staticmethod
    def analyze_results(leakage_points, overall_risk, risk_level, model_choice="tiny"):
        """Enhanced analysis with dynamic model selection"""
        model_id_map = {
            "tiny": "ibm/granite-3-2b-instruct",
            "medium": "ibm/granite-3-8b-instruct",
            "instruct": "ibm/granite-13b-instruct-v2"
        }
        
        model_id = model_id_map.get(model_choice.lower(), "ibm/granite-3-2b-instruct")
        logger.info(f"Using model: {model_id} for vulnerability analysis")
        
        try:
            model = WatsonxConfig.get_model(model_id, max_tokens=600)
            if not model:
                return EnhancedGraniteModels.simulate_analysis_summary(leakage_points, overall_risk)
            
            # Prepare detailed leakage points description
            leakage_desc = "\n".join(
                f"- At {point['time_ms']:.2f} ms: "
                f"Power={point['power_value']:.2f}mA, "
                f"Duration={point['features']['duration']:.4f}ms, "
                f"CPA_Corr={point['features']['cpa_corr']:.2f}, "
                f"Confidence={point['confidence']:.2f}"
                for point in leakage_points
            ) if leakage_points else "No significant leakage points detected"
            
            # Enhanced prompt with more context
            prompt = f"""
            [INST] <<SYS>>
            You are a hardware security expert analyzing side-channel power traces. 
            Provide a technical assessment of these findings:

            DETECTION RESULTS:
            {leakage_desc}

            OVERALL RISK: {overall_risk:.2f} ({risk_level})
            
            ANALYSIS TASKS:
            1. Vulnerability summary (1-2 sentences)
            2. Identify cryptographic operations affected by leakage
            3. Assess exploit difficulty (Low/Medium/High)
            4. Business impact assessment
            5. Return response in JSON format:
            {{
                "vulnerability_summary": "Summary text",
                "affected_operations": ["operation1", "operation2"],
                "exploit_difficulty": "Low/Medium/High",
                "business_impact": "Impact explanation",
                "confidence_score": 0.0-1.0
            }}
            <</SYS>>
            [/INST]
            """
            
            # Generate analysis with retry
            response = EnhancedGraniteModels.retry_api_call(model.generate, prompt=prompt)
            analysis = response['results'][0]['generated_text']
            
            # Extract JSON safely
            result = EnhancedGraniteModels.extract_json(analysis)
            result['model_used'] = model_id.split('/')[-1]
            return result
        except Exception as e:
            logger.error(f"Analysis error: {str(e)}")
            return EnhancedGraniteModels.simulate_analysis_summary(leakage_points, overall_risk)
    
    @staticmethod
    def retry_api_call(func, *args, **kwargs):
        """Retry API call with exponential backoff"""
        for attempt in range(3):
            try:
                return func(*args, **kwargs)
            except (ApiRequestFailure, requests.RequestException) as e:
                wait_time = (2 ** attempt) + 1
                logger.warning(f"API call failed (attempt {attempt+1}/3): {str(e)}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
        raise Exception("API call failed after 3 attempts")

    @staticmethod
    def extract_json(text):
        """Robust JSON extraction from LLM response"""
        try:
            # Improved pattern to find JSON objects
            json_match = re.search(r'\{[\s\S]*\}', text)
            if not json_match:
                raise ValueError("No JSON found in response")
                
            json_str = json_match.group(0)
            return json.loads(json_str)
        except (ValueError, json.JSONDecodeError) as e:
            logger.error(f"JSON extraction error: {str(e)}")
            return {
                "vulnerability_summary": "Analysis summary unavailable",
                "affected_operations": [],
                "business_impact": "Unable to generate business impact assessment",
                "exploit_difficulty": "Unknown",
                "confidence_score": 0.0
            }
    
    @staticmethod
    def simulate_analysis_summary(leakage_points, overall_risk):
        """Fallback simulation if API fails"""
        summary = "Significant correlation detected between power consumption and secret data"
        operations = ["S-Box substitution", "Key schedule"]
        
        if overall_risk > 0.7:
            impact = "High risk of key extraction leading to complete system compromise"
            difficulty = "Medium (requires specialized equipment)"
        elif overall_risk > 0.5:
            impact = "Moderate risk of partial key recovery"
            difficulty = "High (requires advanced expertise)"
        else:
            impact = "Low risk in practical scenarios"
            difficulty = "Very High (theoretical risk only)"
        
        return {
            "vulnerability_summary": summary,
            "affected_operations": operations,
            "business_impact": impact,
            "exploit_difficulty": difficulty,
            "confidence_score": min(0.9, overall_risk + 0.2)
        }
    
    @staticmethod
    def generate_countermeasures(analysis, model_choice="instruct"):
        """Generate targeted countermeasures with model choice"""
        model_id_map = {
            "instruct": "ibm/granite-13b-instruct-v2",
            "medium": "ibm/granite-3-8b-instruct",
            "tiny": "ibm/granite-3-2b-instruct"
        }
        
        model_id = model_id_map.get(model_choice.lower(), "ibm/granite-13b-instruct-v2")
        logger.info(f"Using model: {model_id} for countermeasures")
        
        try:
            model = WatsonxConfig.get_model(model_id, max_tokens=800, temperature=0.3)
            if not model:
                return EnhancedGraniteModels.simulate_countermeasures()
            
            # Prepare prompt with targeted operations
            operations = analysis.get('affected_operations', [])
            targeted_ops = f"Specifically target: {', '.join(operations)}" if operations else ""
            
            prompt = f"""
            [INST] <<SYS>>
            As a hardware security specialist, suggest countermeasures for these vulnerabilities:
            
            {analysis.get('vulnerability_summary', 'Vulnerability analysis unavailable')}
            
            Affected Operations: {', '.join(operations) or 'Not specified'}
            Exploit Difficulty: {analysis.get('exploit_difficulty', 'Unknown')}
            
            Requirements:
            1. Provide 5 hardware-focused countermeasures
            2. For each: 
               - Brief description
               - Implementation approach
               - Estimated effectiveness (High/Medium/Low)
            3. Format: 
               "1. [Description] 
                Implementation: [Details] 
                Effectiveness: [Rating]"
            4. Prioritize countermeasures addressing the specific operations mentioned
            {targeted_ops}
            <</SYS>>
            [/INST]
            """
            
            # Generate countermeasures
            response = EnhancedGraniteModels.retry_api_call(model.generate, prompt=prompt)
            text = response['results'][0]['generated_text']
            
            # Parse with enhanced regex
            return EnhancedGraniteModels.parse_countermeasures(text)
        except Exception as e:
            logger.error(f"Countermeasures error: {str(e)}")
            return EnhancedGraniteModels.simulate_countermeasures()
    
    @staticmethod
    def parse_countermeasures(text):
        """Robust parsing of countermeasures with effectiveness"""
        pattern = r"(\d+)\.\s*([^\n]+?)\n\s*Implementation:\s*(.*?)\n\s*Effectiveness:\s*(High|Medium|Low)"
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
        
        countermeasures = []
        implementations = []
        effectiveness = []
        
        for num, desc, impl, eff in matches[:5]:
            countermeasures.append(desc.strip())
            implementations.append(impl.strip())
            effectiveness.append(eff.strip())
        
        # Fill any missing with simulated
        sim_cm, sim_imp, sim_eff, _ = EnhancedGraniteModels.simulate_countermeasures()
        for i in range(5 - len(countermeasures)):
            countermeasures.append(sim_cm[i])
            implementations.append(sim_imp[i])
            effectiveness.append(sim_eff[i])
            
        intro = f"Generated by IBM Granite model"
        return countermeasures, implementations, effectiveness, intro
    
    @staticmethod
    def simulate_countermeasures():
        """Fallback simulation if API fails"""
        countermeasures = [
            "Implement Boolean masking for S-Box operations",
            "Add dynamic noise generation circuits",
            "Use constant-time implementations for all operations",
            "Implement operation shuffling with PRNG",
            "Add random delays between cryptographic operations"
        ]
        
        implementations = [
            "Apply additive/multiplicative masking to intermediate values",
            "Use LFSR-based noise generator with adjustable amplitude",
            "Ensure all branches and memory accesses take fixed time",
            "Randomize processing order using hardware PRNG",
            "Insert delays with Poisson distribution between rounds"
        ]
        
        effectiveness = ["High", "Medium", "High", "Medium", "Low"]
        
        intro = "Simulated countermeasures (API unavailable)"
        
        return countermeasures, implementations, effectiveness, intro

# ======================
# ENHANCED VISUALIZATION
# ======================
def generate_sample_data():
    """Create realistic power trace simulation data with CPA features"""
    np.random.seed(42)
    time_points = np.linspace(0, 10, 1000)
    base_signal = np.sin(2 * np.pi * time_points) * 0.5
    
    # Add cryptographic operation signatures
    operation_points = [
        {"time_ms": 1.5, "magnitude": 1.8, "width": 0.2, "operation": "S-Box substitution"},
        {"time_ms": 3.2, "magnitude": 1.5, "width": 0.15, "operation": "Key schedule"},
        {"time_ms": 5.0, "magnitude": 0.8, "width": 0.1, "operation": "AddRoundKey"},
        {"time_ms": 7.8, "magnitude": 1.2, "width": 0.25, "operation": "Final round"}
    ]
    
    for point in operation_points:
        idx = np.abs(time_points - point['time_ms']).argmin()
        gaussian = point['magnitude'] * np.exp(-(time_points - point['time_ms'])**2 / (2 * point['width']**2))
        base_signal += gaussian
    
    # Add noise and high-frequency components
    base_signal += np.random.normal(0, 0.15, len(time_points))
    base_signal += 0.3 * np.sin(15 * np.pi * time_points)
    base_signal += 0.2 * np.random.random(len(time_points))
    
    return pd.DataFrame({
        "Time (ms)": time_points,
        "Power Consumption (mA)": base_signal
    }), operation_points

def plot_power_traces(df, leakage_points=None, operation_points=None):
    """Enhanced plot with interactive leakage points"""
    fig = go.Figure()
    
    # Main power trace
    fig.add_trace(go.Scatter(
        x=df["Time (ms)"],
        y=df["Power Consumption (mA)"],
        mode='lines',
        name='Power Consumption',
        line=dict(color='#0068c9', width=2),
        fill='tozeroy',
        fillcolor='rgba(0, 104, 201, 0.1)',
        hovertemplate='Time: %{x:.2f} ms<br>Power: %{y:.2f} mA<extra></extra>'
    ))
    
    # Known operation points
    if operation_points:
        for point in operation_points:
            fig.add_vline(
                x=point['time_ms'],
                line=dict(color="#888", width=1.5, dash="dot"),
                annotation_text=point["operation"],
                annotation_position="top right",
                annotation_font_size=10
            )
    
    # Detected leakage points
    if leakage_points:
        leak_x = [point['time_ms'] for point in leakage_points]
        leak_y = [point['power_value'] for point in leakage_points]
        leak_text = [
            f"<b>Leakage Point</b><br>Time: {p['time_ms']:.2f} ms<br>Power: {p['power_value']:.2f} mA"
            f"<br>Confidence: {p['confidence']:.2f}<br>CPA Corr: {p['features']['cpa_corr']:.2f}"
            for p in leakage_points
        ]
        
        fig.add_trace(go.Scatter(
            x=leak_x,
            y=leak_y,
            mode='markers',
            name='Leakage Detected',
            marker=dict(size=12, color='red', symbol='x'),
            hovertemplate='%{text}<extra></extra>',
            text=leak_text
        ))
    
    fig.update_layout(
        title='Power Consumption Analysis',
        xaxis_title='Time (ms)',
        yaxis_title='Power Consumption (mA)',
        template='plotly_white',
        height=450,
        hovermode="x unified",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    return fig

def risk_meter(value):
    """Create a visual risk meter with improved design"""
    value = min(100, max(0, value))
    level = min(3, int(value // 25))
    colors = ['#00c853', '#ffd600', '#ff9100', '#ff3d00']
    
    html = f"""
    <div style="background: #f8f9fa; border-radius: 10px; padding: 15px; margin: 10px 0; position: relative; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
        <div style="position: absolute; top: 15px; right: 15px; font-weight: bold; font-size: 24px; color: {colors[level]}">
            {value:.1f}%
        </div>
        <div style="margin-top: 10px;">
            <div style="display: flex; justify-content: space-between; margin-bottom: 5px; font-size: 12px;">
                <span>Low</span>
                <span>Medium</span>
                <span>High</span>
                <span>Critical</span>
            </div>
            <div style="background: #e0e0e0; height: 20px; border-radius: 10px; overflow: hidden;">
                <div style="background: linear-gradient(to right, {', '.join(colors)}); 
                            width: {value}%; height: 100%; 
                            position: relative;">
                </div>
            </div>
        </div>
        <div style="margin-top: 15px; font-size: 14px; color: #666;">
            Risk level indicates likelihood of key extraction
        </div>
    </div>
    """
    return html

def create_download_link(df, filename="power_traces.csv"):
    """Generate a download link for CSV file"""
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    return f'<a href="data:file/csv;base64,{b64}" download="{filename}" style="color: #0068c9; text-decoration: none; border: 1px solid #0068c9; border-radius: 4px; padding: 6px 12px; display: inline-block; margin: 10px 0;">Download CSV File</a>'

# ======================
# ENHANCED UI MODULE
# ======================
# Create widgets
use_sample_data = widgets.Checkbox(
    value=True,
    description='Use sample data for demonstration',
    indent=False,
    style={'description_width': 'initial'}
)

upload_widget = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Upload CSV',
    layout=widgets.Layout(width='300px'),
    tooltip="CSV should contain 'Time (ms)' and 'Power Consumption (mA)' columns"
)

sensitivity_slider = widgets.IntSlider(
    value=5,  # Default changed to 5 (maps to 2.5 threshold multiplier)
    min=1,
    max=10,
    step=1,
    description='Detection Sensitivity:',
    style={'description_width': 'initial'},
    continuous_update=False,
    tooltip="Higher values detect smaller leaks but may increase false positives"
)

confidence_slider = widgets.IntSlider(
    value=85,
    min=50,
    max=100,
    step=5,
    description='Min Confidence Threshold:',
    style={'description_width': 'initial'},
    continuous_update=False,
    tooltip="Minimum confidence score to consider a leakage point valid"
)

model_selector = widgets.Dropdown(
    options=[('Granite 3.2B Tiny', 'tiny'), 
             ('Granite 3.8B Medium', 'medium'),
             ('Granite 13B Instruct', 'instruct')],
    value='tiny',
    description='Analysis Model:',
    style={'description_width': 'initial'},
    tooltip="Select IBM Granite model for vulnerability analysis"
)

analyze_button = widgets.Button(
    description='Analyze Power Traces',
    button_style='success',
    icon='search',
    layout=widgets.Layout(width='200px', height='40px')
)

export_button = widgets.Button(
    description='Export Full Report',
    button_style='info',
    icon='file-download',
    layout=widgets.Layout(width='200px', height='40px'),
    disabled=True
)

# Create output areas
power_trace_output = widgets.Output()
analysis_output = widgets.Output()
countermeasure_output = widgets.Output()

# ======================
# MAIN FUNCTIONALITY
# ======================
# Global state for report generation
analysis_results = {}
report_data = {}

def on_analyze_button_clicked(b):
    global analysis_results, report_data
    # Reset outputs and state
    power_trace_output.clear_output()
    analysis_output.clear_output()
    countermeasure_output.clear_output()
    analysis_results = {}
    report_data = {}
    export_button.disabled = True
    
    df = None
    operation_points = None
    file_uploaded = False
    
    with power_trace_output:
        # Show detailed progress
        display(widgets.HTML("<div class='progress-step'><b>Step 1/4:</b> Processing input data...</div>"))
        
        # Handle data input
        if upload_widget.value:
            try:
                uploaded = list(upload_widget.value.values())[0]
                content = uploaded['content'].decode('utf-8')
                df = pd.read_csv(BytesIO(content.encode()))
                
                # Validate data types
                if not pd.api.types.is_numeric_dtype(df['Time (ms)']):
                    raise ValueError("Time column must be numeric")
                if not pd.api.types.is_numeric_dtype(df['Power Consumption (mA)']):
                    raise ValueError("Power column must be numeric")
                
                # Check data ranges
                if df['Time (ms)'].min() < 0 or df['Time (ms)'].max() > 10000:
                    raise ValueError("Time values out of expected range (0-10000 ms)")
                if df['Power Consumption (mA)'].abs().max() > 1000:
                    raise ValueError("Power values exceed reasonable range (±1000 mA)")
                
                file_uploaded = True
                display(widgets.HTML("<div class='success-message'>CSV file successfully uploaded and validated!</div>"))
            except Exception as e:
                display(widgets.HTML(f"<div class='error-message'><b>Data Validation Error:</b> {str(e)}</div>"))
                return
        elif use_sample_data.value:
            df, operation_points = generate_sample_data()
        
        # Validate data
        if df is None or df.empty:
            display(widgets.HTML("<div class='error-message'>No data available. Please upload a file or use sample data.</div>"))
            return
        
        # Check required columns
        required_columns = ["Time (ms)", "Power Consumption (mA)"]
        if not all(col in df.columns for col in required_columns):
            missing = [col for col in required_columns if col not in df.columns]
            display(widgets.HTML(
                f"<div class='error-message'><b>Missing columns:</b> {', '.join(missing)}<br>"
                f"Required columns: {', '.join(required_columns)}</div>"
            ))
            return
        
        # Update progress
        power_trace_output.clear_output()
        display(widgets.HTML("<div class='success-message'>Data processed successfully</div>"))
        display(widgets.HTML("<h3 style='color:#0068c9; border-bottom: 1px solid #e0e0e0; padding-bottom: 10px;'>Power Trace Data</h3>"))
        
        if use_sample_data.value:
            display(widgets.HTML("<p style='color: #666;'>Simulated AES-256 cryptographic operation data</p>"))
            display(widgets.HTML(create_download_link(df)))
        
        # Plot power traces
        fig = plot_power_traces(df, operation_points=operation_points)
        display(fig)
        
        # Show data summary
        display(widgets.HTML(f"""
        <div class='summary-box'>
            <h4 style='color:#0068c9; margin-top:0;'>Data Summary</h4>
            <ul>
                <li><b>Time Points:</b> {len(df)}</li>
                <li><b>Duration:</b> {df['Time (ms)'].iloc[-1]:.2f} ms</li>
                <li><b>Avg Power:</b> {df['Power Consumption (mA)'].mean():.2f} mA</li>
                <li><b>Max Power:</b> {df['Power Consumption (mA)'].max():.2f} mA</li>
                <li><b>Data Source:</b> {"User Upload" if file_uploaded else "Sample Data"}</li>
            </ul>
        </div>
        """))
        
        # Store for report
        report_data['df'] = df
        report_data['operation_points'] = operation_points
        report_data['file_uploaded'] = file_uploaded
    
    with analysis_output:
        display(widgets.HTML("<div class='progress-step'><b>Step 3/4:</b> Analyzing power traces...</div>"))
        
        # Run signal analysis
        leakage_points = AdvancedSignalAnalyzer.detect_leakage_points(
            df, 
            sensitivity=sensitivity_slider.value,
            confidence=confidence_slider.value/100
        )
        overall_risk, risk_level = AdvancedSignalAnalyzer.calculate_correlation(leakage_points)
        
        # Update progress
        analysis_output.clear_output()
        display(widgets.HTML("<div class='success-message'>Signal analysis completed</div>"))
        display(widgets.HTML("<div class='progress-step'><b>Step 4/4:</b> Synthesizing results with IBM Granite...</div>"))
        
        # Use Granite for interpretation
        analysis = EnhancedGraniteModels.analyze_results(
            leakage_points, 
            overall_risk, 
            risk_level,
            model_choice=model_selector.value
        )
        
        # Store results for report
        analysis_results = {
            'leakage_points': leakage_points,
            'overall_risk': overall_risk,
            'risk_level': risk_level,
            'analysis': analysis
        }
        
        # Clear loading and show results
        analysis_output.clear_output()
        
        display(widgets.HTML(f"""
        <h3 style='color:#0068c9; border-bottom: 1px solid #e0e0e0; padding-bottom: 10px;'>
            Vulnerability Analysis Results
            <span style='font-size: 0.6em; color: #666; float: right;'>
                Sens: {sensitivity_slider.value}/10 | Conf: {confidence_slider.value}% | Model: {analysis.get('model_used', 'Granite')}
            </span>
        </h3>
        """))
        
        # Risk assessment
        risk_score = overall_risk * 100
        risk_color = "#ff3d00" if risk_level == "High" else "#ff9100" if risk_level == "Medium" else "#00c853"
        
        display(widgets.HTML(f"""
        <div class='risk-box'>
            <h4 style='color:#0068c9; margin-top:0;'>Risk Assessment</h4>
            {risk_meter(risk_score)}
            <div class='risk-details'>
                <table>
                    <tr>
                        <td><b>Risk Level:</b></td>
                        <td style='color:{risk_color}; font-weight:bold;'>{risk_level}</td>
                        <td><b>Analysis Model:</b></td>
                        <td>{analysis.get('model_used', 'IBM Granite')}</td>
                    </tr>
                    <tr>
                        <td><b>Exploit Difficulty:</b></td>
                        <td>{analysis.get('exploit_difficulty', 'Unknown')}</td>
                        <td><b>Confidence Score:</b></td>
                        <td>{analysis.get('confidence_score', 0.0):.2f}</td>
                    </tr>
                </table>
            </div>
        </div>
        """))
        
        # Vulnerability summary
        display(widgets.HTML(f"""
        <div class='vulnerability-summary'>
            <h4 style='margin-top:0; color:#ff8f00;'>Vulnerability Summary</h4>
            <p>{analysis.get("vulnerability_summary", "Summary unavailable")}</p>
        </div>
        """))
        
        # Affected operations
        if analysis.get("affected_operations"):
            display(widgets.HTML("<h4 style='color:#0068c9;'>Affected Cryptographic Operations</h4>"))
            ops_html = "<ul>"
            for op in analysis["affected_operations"]:
                ops_html += f"<li>{op}</li>"
            ops_html += "</ul>"
            display(widgets.HTML(ops_html))
        
        # Business risk
        display(widgets.HTML("<h4 style='color:#0068c9; margin-top:20px;'>Business Impact</h4>"))
        display(widgets.HTML(f"<p>{analysis.get('business_impact', 'Risk assessment unavailable')}</p>"))
        
        # Enable export button
        export_button.disabled = False
    
    with countermeasure_output:
        display(widgets.HTML("<div class='progress-step'>Generating security recommendations...</div>"))
        
        # Generate countermeasures
        countermeasures, implementations, effectiveness, intro = EnhancedGraniteModels.generate_countermeasures(
            analysis,
            model_choice=model_selector.value
        )
        
        # Store for report
        analysis_results['countermeasures'] = countermeasures
        analysis_results['implementations'] = implementations
        analysis_results['effectiveness'] = effectiveness
        
        # Clear loading and show results
        countermeasure_output.clear_output()
        
        display(widgets.HTML(f"""
        <h3 style='color:#0068c9; border-bottom: 1px solid #e0e0e0; padding-bottom: 10px;'>
            Security Recommendations
            <span style='font-size: 0.6em; color: #666; float: right;'>
                {intro}
            </span>
        </h3>
        """))
        
        display(widgets.HTML(f"""
        <div class='model-info'>
            <p><b>Countermeasure model:</b> {model_selector.label}</p>
        </div>
        """))
        
        # Countermeasures
        display(widgets.HTML("<h4 style='color:#0068c9;'>Recommended Countermeasures</h4>"))
        
        cm_html = "<div class='countermeasures'>"
        for i, (cm, imp, eff) in enumerate(zip(countermeasures, implementations, effectiveness), 1):
            eff_color = "#4caf50" if eff == "High" else "#ff9800" if eff == "Medium" else "#f44336"
            cm_html += f"""
            <div class='countermeasure-box'>
                <div class='cm-header'>
                    <span class='cm-number'>#{i}</span>
                    <span class='cm-title'>{cm}</span>
                    <span class='cm-effectiveness' style='background-color:{eff_color};'>{eff}</span>
                </div>
                <div class='cm-implementation'>
                    <b>Implementation:</b> {imp}
                </div>
            </div>
            """
        cm_html += "</div>"
        
        display(widgets.HTML(cm_html))
        
        # Implementation guidance
        display(widgets.HTML("""
        <h4 style='color:#0068c9; margin-top:25px;'>Implementation Roadmap</h4>
        <div class='roadmap-box'>
            <table>
                <tr class='header-row'>
                    <th>Phase</th>
                    <th>Activities</th>
                    <th>Timeline</th>
                </tr>
                <tr>
                    <td><b>Phase 1</b></td>
                    <td>Implement masking and shuffling countermeasures</td>
                    <td style='text-align:center;'>2-4 weeks</td>
                </tr>
                <tr class='alt-row'>
                    <td><b>Phase 2</b></td>
                    <td>Add noise generation circuits and constant-time implementations</td>
                    <td style='text-align:center;'>3-5 weeks</td>
                </tr>
                <tr>
                    <td><b>Phase 3</b></td>
                    <td>Verification and penetration testing</td>
                    <td style='text-align:center;'>2-3 weeks</td>
                </tr>
            </table>
            
            <div class='verification-box'>
                <b>Verification Steps:</b>
                <ol>
                    <li>Re-run power analysis after each mitigation</li>
                    <li>Perform correlation coefficient validation</li>
                    <li>Conformal equivalence checking</li>
                    <li>100% coverage testing of vulnerable operations</li>
                    <li>Penetration testing with real-world attack scenarios</li>
                </ol>
            </div>
        </div>
        """))
        
        # Add export button
        display(export_button)

def on_export_button_clicked(b):
    """Generate comprehensive HTML report"""
    if not analysis_results or not report_data:
        with countermeasure_output:
            display(widgets.HTML("<div class='error-message'>No analysis data available for export</div>"))
        return
    
    try:
        # Generate HTML report
        html_content = generate_html_report(report_data, analysis_results)
        
        # Create download link
        b64 = base64.b64encode(html_content.encode()).decode()
        url = f'data:text/html;base64,{b64}'
        
        with countermeasure_output:
            display(widgets.HTML(
                f"<div class='success-message'>Report generated successfully!</div>"
                f"<a href='{url}' download='sidechannel_report.html' class='download-link'>"
                "Download Full Report"
                "</a>"
            ))
    except Exception as e:
        with countermeasure_output:
            display(widgets.HTML(f"<div class='error-message'>Report generation failed: {str(e)}</div>"))

def generate_html_report(report_data, analysis_results):
    """Generate comprehensive HTML report"""
    # Create plot image
    fig = plot_power_traces(
        report_data['df'], 
        leakage_points=analysis_results.get('leakage_points', []),
        operation_points=report_data.get('operation_points', [])
    )
    plot_html = fig.to_html(full_html=False, include_plotlyjs='cdn')
    
    # Risk meter
    risk_score = analysis_results.get('overall_risk', 0) * 100
    risk_level = analysis_results.get('risk_level', 'Low')
    
    # Countermeasures table
    cm_table = ""
    for i, (cm, imp, eff) in enumerate(zip(
        analysis_results.get('countermeasures', []),
        analysis_results.get('implementations', []),
        analysis_results.get('effectiveness', [])
    ), 1):
        eff_color = "green" if eff == "High" else "orange" if eff == "Medium" else "red"
        cm_table += f"""
        <tr>
            <td>{i}</td>
            <td>{cm}</td>
            <td>{imp}</td>
            <td style="color:{eff_color}">{eff}</td>
        </tr>
        """
    
    # Create HTML structure
    return f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>SideChannel Sentinel Report</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; }}
            .container {{ max-width: 1200px; margin: 0 auto; padding: 20px; }}
            .header {{ background: #0068c9; color: white; padding: 30px; text-align: center; border-radius: 8px; }}
            .section {{ margin: 30px 0; padding: 20px; background: white; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.05); }}
            .risk-meter {{ background: #f8f9fa; padding: 20px; border-radius: 8px; }}
            table {{ width: 100%; border-collapse: collapse; }}
            th, td {{ padding: 12px 15px; text-align: left; border-bottom: 1px solid #ddd; }}
            th {{ background-color: #f5f5f5; }}
            .countermeasure-row:nth-child(even) {{ background-color: #f9f9f9; }}
            .plot-container {{ margin: 20px 0; }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>SideChannel Sentinel Report</h1>
                <p>Comprehensive Side-Channel Vulnerability Analysis</p>
                <p>Generated on {time.strftime("%Y-%m-%d %H:%M:%S")}</p>
            </div>
            
            <div class="section">
                <h2>Executive Summary</h2>
                <p><b>Overall Risk:</b> <span style="color:{"#ff3d00" if risk_level=="High" else "#ff9100" if risk_level=="Medium" else "#00c853"}">{risk_level}</span> ({risk_score:.1f}%)</p>
                <p><b>Vulnerability Summary:</b> {analysis_results.get('analysis', {}).get('vulnerability_summary', '')}</p>
                <p><b>Business Impact:</b> {analysis_results.get('analysis', {}).get('business_impact', '')}</p>
            </div>
            
            <div class="section">
                <h2>Power Trace Analysis</h2>
                <div class="plot-container">
                    {plot_html}
                </div>
            </div>
            
            <div class="section">
                <h2>Detailed Vulnerability Assessment</h2>
                <p><b>Affected Operations:</b> {', '.join(analysis_results.get('analysis', {}).get('affected_operations', [])) or 'None detected'}</p>
                <p><b>Exploit Difficulty:</b> {analysis_results.get('analysis', {}).get('exploit_difficulty', 'Unknown')}</p>
                <p><b>Analysis Confidence:</b> {analysis_results.get('analysis', {}).get('confidence_score', 0.0)*100:.1f}%</p>
            </div>
            
            <div class="section">
                <h2>Security Recommendations</h2>
                <table>
                    <tr>
                        <th>#</th>
                        <th>Countermeasure</th>
                        <th>Implementation</th>
                        <th>Effectiveness</th>
                    </tr>
                    {cm_table}
                </table>
            </div>
            
            <div class="section">
                <h2>Implementation Roadmap</h2>
                <table>
                    <tr>
                        <th>Phase</th>
                        <th>Activities</th>
                        <th>Timeline</th>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 1</td>
                        <td>Implement critical countermeasures</td>
                        <td>2-4 weeks</td>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 2</td>
                        <td>Hardware modifications and verification</td>
                        <td>3-5 weeks</td>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 3</td>
                        <td>Penetration testing and validation</td>
                        <td>2-3 weeks</td>
                    </tr>
                </table>
            </div>
        </div>
    </body>
    </html>
    """

# Bind button clicks
analyze_button.on_click(on_analyze_button_clicked)
export_button.on_click(on_export_button_clicked)

# ======================
# STYLES AND UX ENHANCEMENTS
# ======================
styles = """
<style>
    .progress-step {
        background: #e3f2fd;
        padding: 10px 15px;
        border-radius: 5px;
        margin: 10px 0;
        border-left: 4px solid #0068c9;
    }
    .success-message {
        color: #2e7d32;
        background: #e8f5e9;
        padding: 10px;
        border-radius: 5px;
        border-left: 4px solid #4caf50;
        margin: 10px 0;
    }
    .error-message {
        color: #c62828;
        background: #ffebee;
        padding: 15px;
        border-radius: 5px;
        border-left: 4px solid #f44336;
        margin: 15px 0;
    }
    .summary-box {
        background: #f8f9fa;
        padding: 15px;
        border-radius: 8px;
        border-left: 4px solid #0068c9;
        margin: 15px 0;
    }
    .risk-box {
        background: #f0f2f6;
        border-radius: 10px;
        padding: 15px;
        margin-bottom: 20px;
    }
    .risk-details {
        margin-top: 15px;
        background: white;
        padding: 10px;
        border-radius: 5px;
    }
    .vulnerability-summary {
        background: #fff8e1;
        border-left: 4px solid #ffc107;
        padding: 15px;
        margin: 20px 0;
        border-radius: 0 8px 8px 0;
    }
    .model-info {
        background: #e8f5e9;
        border-left: 4px solid #4caf50;
        padding: 15px;
        margin: 10px 0;
        border-radius: 0 8px 8px 0;
    }
    .countermeasures {
        margin-left: 10px;
    }
    .countermeasure-box {
        margin-bottom: 15px;
        padding: 15px;
        background: white;
        border-radius: 8px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.05);
    }
    .cm-header {
        display: flex;
        align-items: center;
        margin-bottom: 10px;
    }
    .cm-number {
        background: #0068c9;
        color: white;
        width: 28px;
        height: 28px;
        border-radius: 50%;
        display: flex;
        align-items: center;
        justify-content: center;
        margin-right: 10px;
        font-weight: bold;
    }
    .cm-title {
        flex-grow: 1;
        font-weight: bold;
        color: #0068c9;
    }
    .cm-effectiveness {
        padding: 4px 10px;
        border-radius: 20px;
        font-size: 0.8em;
        color: white;
    }
    .cm-implementation {
        margin-top: 8px;
        padding-left: 15px;
        border-left: 2px solid #e0e0e0;
        color: #555;
    }
    .roadmap-box {
        background: #f8f9fa;
        border-radius: 8px;
        padding: 15px;
        border: 1px solid #e0e0e0;
    }
    .verification-box {
        margin-top: 15px;
        padding: 15px;
        background: #e3f2fd;
        border-radius: 5px;
    }
    .download-link {
        display: inline-block;
        background: #0068c9;
        color: white;
        text-decoration: none;
        padding: 12px 24px;
        border-radius: 5px;
        font-weight: bold;
        margin: 15px 0;
        transition: background 0.3s;
    }
    .download-link:hover {
        background: #0056b3;
    }
</style>
"""

# ======================
# UI LAYOUT
# ======================
display(widgets.HTML(styles))

display(widgets.VBox([
    widgets.HTML(
        "<div style='background:#0068c9; color:white; padding:20px; border-radius:8px 8px 0 0;'>"
        "<h1 style='text-align:center; margin:0;'>🔒 SideChannel Sentinel Pro</h1>"
        "<h3 style='text-align:center; margin:0; font-weight:normal;'>Advanced Side-Channel Vulnerability Detection</h3>"
        "</div>"
    ),
    
    widgets.HTML(
        "<div style='padding:15px; background:#f0f5ff; border-radius:0 0 8px 8px; margin-bottom:20px;'>"
        "<p style='text-align:center; margin:0;'>Signal Processing + CPA + IBM Granite for Comprehensive Hardware Security</p>"
        "</div>"
    ),
    
    widgets.VBox([
        widgets.HTML("<h3 style='color:#0068c9;'>Data Source</h3>"),
        widgets.HBox([use_sample_data, upload_widget])
    ], layout=widgets.Layout(padding='10px', border='1px solid #e0e0e0', borderRadius='8px')),
    
    widgets.VBox([
        widgets.HTML("<h3 style='color:#0068c9;'>Analysis Configuration</h3>"),
        sensitivity_slider,
        confidence_slider,
        model_selector
    ], layout=widgets.Layout(padding='10px', border='1px solid #e0e0e0', borderRadius='8px', margin='15px 0')),
    
    widgets.HTML(
        "<div style='background:#e8f5e9; padding:15px; border-radius:8px; margin:10px 0;'>"
        "<h4 style='color:#0068c9; margin-top:0;'>IBM Watsonx Configuration</h4>"
        "<p>Using IBM credentials for Granite models:</p>"
        "<ul>"
        f"<li><b>Region:</b> us-south</li>"
        f"<li><b>Project ID:</b> {os.environ['IBM_PROJECT_ID']}</li>"
        f"<li><b>Available Models:</b> Granite 3.2B Tiny, Granite 3.8B Medium, Granite 13B Instruct</li>"
        "</ul>"
        "</div>"
    ),
    
    widgets.HBox([analyze_button], layout=widgets.Layout(justify_content='center', margin='10px 0')),
    
    power_trace_output,
    analysis_output,
    countermeasure_output
]))

# Automatically run with sample data
on_analyze_button_clicked(analyze_button)

HTML(value="\n        <h3 style='color:#0068c9; border-bottom: 1px solid #e0e0e0; padding-bottom: 10px;'>\n   …

HTML(value="\n        <div class='model-info'>\n            <p><b>Countermeasure model:</b> Granite 3.2B Tiny<…

HTML(value="<h4 style='color:#0068c9;'>Recommended Countermeasures</h4>")

HTML(value="<div class='countermeasures'>\n            <div class='countermeasure-box'>\n                <div …

HTML(value="\n        <h4 style='color:#0068c9; margin-top:25px;'>Implementation Roadmap</h4>\n        <div cl…

Button(button_style='info', description='Export Full Report', icon='file-download', layout=Layout(height='40px…

In [10]:
pip install streamlit streamlit_jupyter ibm-watson-machine-learning plotly pandas numpy requests

  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.1 MB 1.3 MB/s eta 0:00:08
   --- ------------------------------------ 0.8/10.1 


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import base64
from io import BytesIO
import plotly.graph_objects as go
import time
import json
import re
import requests
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.wml_client_error import ApiRequestFailure, WMLClientError
import logging
import os

# ======================
# LOGGING CONFIGURATION
# ======================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger('SideChannelSentinel')

# ======================
# IBM WATSONX CONFIGURATION
# ======================
class WatsonxConfig:
    @staticmethod
    def get_model(model_id, max_tokens=500, temperature=0.2):
        """Initialize IBM Granite model with credentials and retry logic"""
        for attempt in range(3):
            try:
                model = Model(
                    model_id=model_id,
                    credentials={
                        "apikey": os.environ["IBM_API_KEY"],
                        "url": os.environ["IBM_URL"]
                    },
                    project_id=os.environ["IBM_PROJECT_ID"],
                    params={
                        GenParams.DECODING_METHOD: "greedy",
                        GenParams.MAX_NEW_TOKENS: max_tokens,
                        GenParams.MIN_NEW_TOKENS: 50,
                        GenParams.TEMPERATURE: temperature,
                        GenParams.TOP_P: 0.9,
                        GenParams.TOP_K: 50
                    }
                )
                return model
            except (ApiRequestFailure, WMLClientError, requests.RequestException) as e:
                wait_time = (2 ** attempt) + 1
                logger.error(f"IBM API error (attempt {attempt+1}/3): {str(e)}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
            except Exception as e:
                logger.error(f"Unexpected error initializing model: {str(e)}")
                return None
        logger.error("Failed to initialize model after 3 attempts")
        return None

# ======================
# ENHANCED SIGNAL PROCESSING
# ======================
class AdvancedSignalAnalyzer:
    @staticmethod
    def detect_leakage_points(df, sensitivity=7, confidence=0.85):
        """Advanced leakage detection with multiple techniques"""
        logger.info("Starting leakage detection analysis")
        
        # 1. Statistical Analysis with Adaptive Thresholding
        window_size = max(10, int(len(df) * 0.02))  # Dynamic window size
        df['MA'] = df['Power Consumption (mA)'].rolling(window=window_size).mean()
        df['STD'] = df['Power Consumption (mA)'].rolling(window=window_size).std()
        
        # Adaptive threshold based on sensitivity
        sensitivity_multiplier = 0.5 + (sensitivity / 10) * 4.5  # 0.5 to 5.0
        df['Threshold'] = df['STD'].rolling(window=100, min_periods=1).median() * sensitivity_multiplier
        
        # 2. Simulated Correlation Power Analysis (CPA)
        # Generate hypothetical power model (Hamming weight model)
        hypothetical_power = AdvancedSignalAnalyzer.simulate_hamming_model(df)
        df['CPA_Correlation'] = df['Power Consumption (mA)'].rolling(window=window_size).corr(hypothetical_power)
        
        # 3. Combined detection logic
        high_deviation = df[(df['STD'] > df['Threshold']) | 
                            (df['CPA_Correlation'].abs() > confidence)]
        
        leakage_points = []
        if not high_deviation.empty:
            # Group consecutive points
            high_deviation['Group'] = (high_deviation.index.to_series().diff() > 1).cumsum()
            
            for _, group in high_deviation.groupby('Group'):
                peak_idx = group['STD'].idxmax()
                peak_time = df.loc[peak_idx, 'Time (ms)']
                peak_value = df.loc[peak_idx, 'Power Consumption (mA)']
                
                # Calculate feature vector for LLM
                features = {
                    'duration': group['Time (ms)'].max() - group['Time (ms)'].min(),
                    'avg_power': group['Power Consumption (mA)'].mean(),
                    'max_power': group['Power Consumption (mA)'].max(),
                    'cpa_corr': group['CPA_Correlation'].abs().mean(),
                    'std_dev': group['STD'].mean()
                }
                
                # Confidence score based on multiple factors
                conf_score = min(0.95, (features['std_dev'] / df['Threshold'].mean() * 0.4 + 
                                        features['cpa_corr'] * 0.6))
                
                leakage_points.append({
                    'time_ms': peak_time,
                    'power_value': peak_value,
                    'features': features,
                    'confidence': conf_score
                })
        
        logger.info(f"Detected {len(leakage_points)} potential leakage points")
        return leakage_points

    @staticmethod
    def simulate_hamming_model(df):
        """Simulate CPA Hamming weight model for demonstration"""
        # In real CPA, this would be based on intermediate values
        # Here we simulate with a synthetic pattern
        np.random.seed(42)
        base = np.sin(2 * np.pi * df['Time (ms)'] * 5) * 0.3
        spikes = np.zeros(len(df))
        
        # Add spikes at random positions to simulate operation points
        for _ in range(5):
            idx = np.random.randint(0, len(df))
            width = np.random.uniform(0.05, 0.2)
            spike = np.exp(-(df['Time (ms)'] - df.loc[idx, 'Time (ms)'])**2 / (2 * width**2))
            spikes += spike * np.random.uniform(0.5, 1.5)
        
        return base + spikes

    @staticmethod
    def calculate_correlation(leakage_points):
        """Calculate correlation metrics for leakage points"""
        if not leakage_points:
            return 0.0, "Low"
        
        # Weighted average of multiple factors
        total_risk = 0
        weights = {'std_dev': 0.4, 'cpa_corr': 0.5, 'duration': 0.1}
        
        for point in leakage_points:
            features = point['features']
            risk_score = (features['std_dev'] * weights['std_dev'] +
                          features['cpa_corr'] * weights['cpa_corr'] +
                          features['duration'] * weights['duration'])
            total_risk += risk_score * point['confidence']
        
        overall_risk = min(1.0, total_risk / len(leakage_points))
        
        # Determine risk level
        if overall_risk > 0.7:
            risk_level = "High"
        elif overall_risk > 0.5:
            risk_level = "Medium"
        else:
            risk_level = "Low"
            
        return overall_risk, risk_level

# ======================
# ENHANCED GRANITE MODELS INTEGRATION
# ======================
class EnhancedGraniteModels:
    @staticmethod
    def analyze_results(leakage_points, overall_risk, risk_level, model_choice="tiny"):
        """Enhanced analysis with dynamic model selection"""
        model_id_map = {
            "tiny": "ibm/granite-3-2b-instruct",
            "medium": "ibm/granite-3-8b-instruct",
            "instruct": "ibm/granite-13b-instruct-v2"
        }
        
        model_id = model_id_map.get(model_choice.lower(), "ibm/granite-3-2b-instruct")
        logger.info(f"Using model: {model_id} for vulnerability analysis")
        
        try:
            model = WatsonxConfig.get_model(model_id, max_tokens=600)
            if not model:
                return EnhancedGraniteModels.simulate_analysis_summary(leakage_points, overall_risk)
            
            # Prepare detailed leakage points description
            leakage_desc = "\n".join(
                f"- At {point['time_ms']:.2f} ms: "
                f"Power={point['power_value']:.2f}mA, "
                f"Duration={point['features']['duration']:.4f}ms, "
                f"CPA_Corr={point['features']['cpa_corr']:.2f}, "
                f"Confidence={point['confidence']:.2f}"
                for point in leakage_points
            ) if leakage_points else "No significant leakage points detected"
            
            # Enhanced prompt with more context
            prompt = f"""
            [INST] <<SYS>>
            You are a hardware security expert analyzing side-channel power traces. 
            Provide a technical assessment of these findings:

            DETECTION RESULTS:
            {leakage_desc}

            OVERALL RISK: {overall_risk:.2f} ({risk_level})
            
            ANALYSIS TASKS:
            1. Vulnerability summary (1-2 sentences)
            2. Identify cryptographic operations affected by leakage
            3. Assess exploit difficulty (Low/Medium/High)
            4. Business impact assessment
            5. Return response in JSON format:
            {{
                "vulnerability_summary": "Summary text",
                "affected_operations": ["operation1", "operation2"],
                "exploit_difficulty": "Low/Medium/High",
                "business_impact": "Impact explanation",
                "confidence_score": 0.0-1.0
            }}
            <</SYS>>
            [/INST]
            """
            
            # Generate analysis with retry
            response = EnhancedGraniteModels.retry_api_call(model.generate, prompt=prompt)
            analysis = response['results'][0]['generated_text']
            
            # Extract JSON safely
            result = EnhancedGraniteModels.extract_json(analysis)
            result['model_used'] = model_id.split('/')[-1]
            return result
        except Exception as e:
            logger.error(f"Analysis error: {str(e)}")
            return EnhancedGraniteModels.simulate_analysis_summary(leakage_points, overall_risk)
    
    @staticmethod
    def retry_api_call(func, *args, **kwargs):
        """Retry API call with exponential backoff"""
        for attempt in range(3):
            try:
                return func(*args, **kwargs)
            except (ApiRequestFailure, requests.RequestException) as e:
                wait_time = (2 ** attempt) + 1
                logger.warning(f"API call failed (attempt {attempt+1}/3): {str(e)}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
        raise Exception("API call failed after 3 attempts")

    @staticmethod
    def extract_json(text):
        """Robust JSON extraction from LLM response"""
        try:
            # Improved pattern to find JSON objects
            json_match = re.search(r'\{[\s\S]*\}', text)
            if not json_match:
                raise ValueError("No JSON found in response")
                
            json_str = json_match.group(0)
            return json.loads(json_str)
        except (ValueError, json.JSONDecodeError) as e:
            logger.error(f"JSON extraction error: {str(e)}")
            return {
                "vulnerability_summary": "Analysis summary unavailable",
                "affected_operations": [],
                "business_impact": "Unable to generate business impact assessment",
                "exploit_difficulty": "Unknown",
                "confidence_score": 0.0
            }
    
    @staticmethod
    def simulate_analysis_summary(leakage_points, overall_risk):
        """Fallback simulation if API fails"""
        summary = "Significant correlation detected between power consumption and secret data"
        operations = ["S-Box substitution", "Key schedule"]
        
        if overall_risk > 0.7:
            impact = "High risk of key extraction leading to complete system compromise"
            difficulty = "Medium (requires specialized equipment)"
        elif overall_risk > 0.5:
            impact = "Moderate risk of partial key recovery"
            difficulty = "High (requires advanced expertise)"
        else:
            impact = "Low risk in practical scenarios"
            difficulty = "Very High (theoretical risk only)"
        
        return {
            "vulnerability_summary": summary,
            "affected_operations": operations,
            "business_impact": impact,
            "exploit_difficulty": difficulty,
            "confidence_score": min(0.9, overall_risk + 0.2)
        }
    
    @staticmethod
    def generate_countermeasures(analysis, model_choice="instruct"):
        """Generate targeted countermeasures with model choice"""
        model_id_map = {
            "instruct": "ibm/granite-13b-instruct-v2",
            "medium": "ibm/granite-3-8b-instruct",
            "tiny": "ibm/granite-3-2b-instruct"
        }
        
        model_id = model_id_map.get(model_choice.lower(), "ibm/granite-13b-instruct-v2")
        logger.info(f"Using model: {model_id} for countermeasures")
        
        try:
            model = WatsonxConfig.get_model(model_id, max_tokens=800, temperature=0.3)
            if not model:
                return EnhancedGraniteModels.simulate_countermeasures()
            
            # Prepare prompt with targeted operations
            operations = analysis.get('affected_operations', [])
            targeted_ops = f"Specifically target: {', '.join(operations)}" if operations else ""
            
            prompt = f"""
            [INST] <<SYS>>
            As a hardware security specialist, suggest countermeasures for these vulnerabilities:
            
            {analysis.get('vulnerability_summary', 'Vulnerability analysis unavailable')}
            
            Affected Operations: {', '.join(operations) or 'Not specified'}
            Exploit Difficulty: {analysis.get('exploit_difficulty', 'Unknown')}
            
            Requirements:
            1. Provide 5 hardware-focused countermeasures
            2. For each: 
               - Brief description
               - Implementation approach
               - Estimated effectiveness (High/Medium/Low)
            3. Format: 
               "1. [Description] 
                Implementation: [Details] 
                Effectiveness: [Rating]"
            4. Prioritize countermeasures addressing the specific operations mentioned
            {targeted_ops}
            <</SYS>>
            [/INST]
            """
            
            # Generate countermeasures
            response = EnhancedGraniteModels.retry_api_call(model.generate, prompt=prompt)
            text = response['results'][0]['generated_text']
            
            # Parse with enhanced regex
            return EnhancedGraniteModels.parse_countermeasures(text)
        except Exception as e:
            logger.error(f"Countermeasures error: {str(e)}")
            return EnhancedGraniteModels.simulate_countermeasures()
    
    @staticmethod
    def parse_countermeasures(text):
        """Robust parsing of countermeasures with effectiveness"""
        pattern = r"(\d+)\.\s*([^\n]+?)\n\s*Implementation:\s*(.*?)\n\s*Effectiveness:\s*(High|Medium|Low)"
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
        
        countermeasures = []
        implementations = []
        effectiveness = []
        
        for num, desc, impl, eff in matches[:5]:
            countermeasures.append(desc.strip())
            implementations.append(impl.strip())
            effectiveness.append(eff.strip())
        
        # Fill any missing with simulated
        sim_cm, sim_imp, sim_eff, _ = EnhancedGraniteModels.simulate_countermeasures()
        for i in range(5 - len(countermeasures)):
            countermeasures.append(sim_cm[i])
            implementations.append(sim_imp[i])
            effectiveness.append(sim_eff[i])
            
        intro = f"Generated by IBM Granite model"
        return countermeasures, implementations, effectiveness, intro
    
    @staticmethod
    def simulate_countermeasures():
        """Fallback simulation if API fails"""
        countermeasures = [
            "Implement Boolean masking for S-Box operations",
            "Add dynamic noise generation circuits",
            "Use constant-time implementations for all operations",
            "Implement operation shuffling with PRNG",
            "Add random delays between cryptographic operations"
        ]
        
        implementations = [
            "Apply additive/multiplicative masking to intermediate values",
            "Use LFSR-based noise generator with adjustable amplitude",
            "Ensure all branches and memory accesses take fixed time",
            "Randomize processing order using hardware PRNG",
            "Insert delays with Poisson distribution between rounds"
        ]
        
        effectiveness = ["High", "Medium", "High", "Medium", "Low"]
        
        intro = "Simulated countermeasures (API unavailable)"
        
        return countermeasures, implementations, effectiveness, intro

# ======================
# ENHANCED VISUALIZATION
# ======================
def generate_sample_data():
    """Create realistic power trace simulation data with CPA features"""
    np.random.seed(42)
    time_points = np.linspace(0, 10, 1000)
    base_signal = np.sin(2 * np.pi * time_points) * 0.5
    
    # Add cryptographic operation signatures
    operation_points = [
        {"time_ms": 1.5, "magnitude": 1.8, "width": 0.2, "operation": "S-Box substitution"},
        {"time_ms": 3.2, "magnitude": 1.5, "width": 0.15, "operation": "Key schedule"},
        {"time_ms": 5.0, "magnitude": 0.8, "width": 0.1, "operation": "AddRoundKey"},
        {"time_ms": 7.8, "magnitude": 1.2, "width": 0.25, "operation": "Final round"}
    ]
    
    for point in operation_points:
        idx = np.abs(time_points - point['time_ms']).argmin()
        gaussian = point['magnitude'] * np.exp(-(time_points - point['time_ms'])**2 / (2 * point['width']**2))
        base_signal += gaussian
    
    # Add noise and high-frequency components
    base_signal += np.random.normal(0, 0.15, len(time_points))
    base_signal += 0.3 * np.sin(15 * np.pi * time_points)
    base_signal += 0.2 * np.random.random(len(time_points))
    
    return pd.DataFrame({
        "Time (ms)": time_points,
        "Power Consumption (mA)": base_signal
    }), operation_points

def plot_power_traces(df, leakage_points=None, operation_points=None):
    """Enhanced plot with interactive leakage points"""
    fig = go.Figure()
    
    # Main power trace
    fig.add_trace(go.Scatter(
        x=df["Time (ms)"],
        y=df["Power Consumption (mA)"],
        mode='lines',
        name='Power Consumption',
        line=dict(color='#0068c9', width=2),
        fill='tozeroy',
        fillcolor='rgba(0, 104, 201, 0.1)',
        hovertemplate='Time: %{x:.2f} ms<br>Power: %{y:.2f} mA<extra></extra>'
    ))
    
    # Known operation points
    if operation_points:
        for point in operation_points:
            fig.add_vline(
                x=point['time_ms'],
                line=dict(color="#888", width=1.5, dash="dot"),
                annotation_text=point["operation"],
                annotation_position="top right",
                annotation_font_size=10
            )
    
    # Detected leakage points
    if leakage_points:
        leak_x = [point['time_ms'] for point in leakage_points]
        leak_y = [point['power_value'] for point in leakage_points]
        leak_text = [
            f"<b>Leakage Point</b><br>Time: {p['time_ms']:.2f} ms<br>Power: {p['power_value']:.2f} mA"
            f"<br>Confidence: {p['confidence']:.2f}<br>CPA Corr: {p['features']['cpa_corr']:.2f}"
            for p in leakage_points
        ]
        
        fig.add_trace(go.Scatter(
            x=leak_x,
            y=leak_y,
            mode='markers',
            name='Leakage Detected',
            marker=dict(size=12, color='red', symbol='x'),
            hovertemplate='%{text}<extra></extra>',
            text=leak_text
        ))
    
    fig.update_layout(
        title='Power Consumption Analysis',
        xaxis_title='Time (ms)',
        yaxis_title='Power Consumption (mA)',
        template='plotly_white',
        height=450,
        hovermode="x unified",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    return fig

def risk_meter(value):
    """Create a visual risk meter with improved design"""
    value = min(100, max(0, value))
    level = min(3, int(value // 25))
    colors = ['#00c853', '#ffd600', '#ff9100', '#ff3d00']
    
    html = f"""
    <div style="background: #f8f9fa; border-radius: 10px; padding: 15px; margin: 10px 0; position: relative; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
        <div style="position: absolute; top: 15px; right: 15px; font-weight: bold; font-size: 24px; color: {colors[level]}">
            {value:.1f}%
        </div>
        <div style="margin-top: 10px;">
            <div style="display: flex; justify-content: space-between; margin-bottom: 5px; font-size: 12px;">
                <span>Low</span>
                <span>Medium</span>
                <span>High</span>
                <span>Critical</span>
            </div>
            <div style="background: #e0e0e0; height: 20px; border-radius: 10px; overflow: hidden;">
                <div style="background: linear-gradient(to right, {', '.join(colors)}); 
                            width: {value}%; height: 100%; 
                            position: relative;">
                </div>
            </div>
        </div>
        <div style="margin-top: 15px; font-size: 14px; color: #666;">
            Risk level indicates likelihood of key extraction
        </div>
    </div>
    """
    return html

def create_download_link(df, filename="power_traces.csv"):
    """Generate a download link for CSV file"""
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    return f'<a href="data:file/csv;base64,{b64}" download="{filename}" style="color: #0068c9; text-decoration: none; border: 1px solid #0068c9; border-radius: 4px; padding: 6px 12px; display: inline-block; margin: 10px 0;">Download CSV File</a>'

# ======================
# STREAMLIT UI
# ======================
def main():
    # Set up environment variables for IBM Watson
    os.environ["IBM_API_KEY"] = "RyIKeWjkIi-jx4F2uerWwUuInh6d9hoIJal8frtzKIMk"
    os.environ["IBM_URL"] = "https://us-south.ml.cloud.ibm.com"
    os.environ["IBM_PROJECT_ID"] = "5d59376b-9172-474f-ac91-c33d1f16bb7b"
    
    # Custom CSS styling
    st.markdown("""
    <style>
        .header {
            background: #0068c9;
            color: white;
            padding: 30px;
            text-align: center;
            border-radius: 8px;
            margin-bottom: 20px;
        }
        .section {
            margin: 20px 0;
            padding: 20px;
            background: white;
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.05);
        }
        .risk-box {
            background: #f0f2f6;
            border-radius: 10px;
            padding: 15px;
            margin-bottom: 20px;
        }
        .risk-details {
            margin-top: 15px;
            background: white;
            padding: 10px;
            border-radius: 5px;
        }
        .vulnerability-summary {
            background: #fff8e1;
            border-left: 4px solid #ffc107;
            padding: 15px;
            margin: 20px 0;
            border-radius: 0 8px 8px 0;
        }
        .countermeasure-box {
            margin-bottom: 15px;
            padding: 15px;
            background: white;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.05);
        }
        .cm-header {
            display: flex;
            align-items: center;
            margin-bottom: 10px;
        }
        .cm-number {
            background: #0068c9;
            color: white;
            width: 28px;
            height: 28px;
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            margin-right: 10px;
            font-weight: bold;
        }
        .cm-title {
            flex-grow: 1;
            font-weight: bold;
            color: #0068c9;
        }
        .cm-effectiveness {
            padding: 4px 10px;
            border-radius: 20px;
            font-size: 0.8em;
            color: white;
        }
        .cm-implementation {
            margin-top: 8px;
            padding-left: 15px;
            border-left: 2px solid #e0e0e0;
            color: #555;
        }
        .progress-step {
            background: #e3f2fd;
            padding: 10px 15px;
            border-radius: 5px;
            margin: 10px 0;
            border-left: 4px solid #0068c9;
        }
        .success-message {
            color: #2e7d32;
            background: #e8f5e9;
            padding: 10px;
            border-radius: 5px;
            border-left: 4px solid #4caf50;
            margin: 10px 0;
        }
        .error-message {
            color: #c62828;
            background: #ffebee;
            padding: 15px;
            border-radius: 5px;
            border-left: 4px solid #f44336;
            margin: 15px 0;
        }
        .summary-box {
            background: #f8f9fa;
            padding: 15px;
            border-radius: 8px;
            border-left: 4px solid #0068c9;
            margin: 15px 0;
        }
        .download-link {
            display: inline-block;
            background: #0068c9;
            color: white;
            text-decoration: none;
            padding: 12px 24px;
            border-radius: 5px;
            font-weight: bold;
            margin: 15px 0;
            transition: background 0.3s;
        }
        .download-link:hover {
            background: #0056b3;
        }
    </style>
    """, unsafe_allow_html=True)
    
    # Initialize session state
    if 'analysis_results' not in st.session_state:
        st.session_state.analysis_results = {}
    if 'report_data' not in st.session_state:
        st.session_state.report_data = {}
    
    # Header
    st.markdown("""
    <div class="header">
        <h1>🔒 SideChannel Sentinel Pro</h1>
        <h3>Advanced Side-Channel Vulnerability Detection</h3>
        <p>Signal Processing + CPA + IBM Granite for Comprehensive Hardware Security</p>
    </div>
    """, unsafe_allow_html=True)
    
    # Data source section
    st.markdown("<h2>Data Source</h2>", unsafe_allow_html=True)
    use_sample_data = st.checkbox('Use sample data for demonstration', value=True)
    uploaded_file = st.file_uploader("Upload CSV file", type=['csv'], 
                                     help="CSV should contain 'Time (ms)' and 'Power Consumption (mA)' columns")
    
    # Analysis configuration
    st.markdown("<h2>Analysis Configuration</h2>", unsafe_allow_html=True)
    col1, col2 = st.columns(2)
    with col1:
        sensitivity = st.slider("Detection Sensitivity:", 1, 10, 5, 
                                help="Higher values detect smaller leaks but may increase false positives")
    with col2:
        confidence = st.slider("Min Confidence Threshold:", 50, 100, 85, 
                               help="Minimum confidence score to consider a leakage point valid")
    
    # FIXED: Corrected selectbox format function
    model_choice = st.selectbox(
        "Analysis Model:", 
        options=[('Granite 3.2B Tiny', 'tiny'), 
                 ('Granite 3.8B Medium', 'medium'),
                 ('Granite 13B Instruct', 'instruct')],
        format_func=lambda x: x[0],  # Use first element of tuple as display text
        index=0
    )
    
    # Extract the internal value from the selected tuple
    if isinstance(model_choice, tuple):
        model_choice = model_choice[1]
    
    # IBM Watsonx Configuration
    st.markdown("""
    <div class="section">
        <h3>IBM Watsonx Configuration</h3>
        <p>Using IBM credentials for Granite models:</p>
        <ul>
            <li><b>Region:</b> us-south</li>
            <li><b>Project ID:</b> 5d59376b-9172-474f-ac91-c33d1f16bb7b</li>
            <li><b>Available Models:</b> Granite 3.2B Tiny, Granite 3.8B Medium, Granite 13B Instruct</li>
        </ul>
    </div>
    """, unsafe_allow_html=True)
    
    # Analyze button
    if st.button("Analyze Power Traces", use_container_width=True, type="primary"):
        st.session_state.analysis_results = {}
        st.session_state.report_data = {}
        
        # Step 1: Process input data
        with st.status("Processing input data...", expanded=True) as status:
            df = None
            operation_points = None
            file_uploaded = False
            
            if uploaded_file is not None:
                try:
                    content = uploaded_file.getvalue().decode('utf-8')
                    df = pd.read_csv(BytesIO(content.encode()))
                    
                    # Validate data types
                    if not pd.api.types.is_numeric_dtype(df['Time (ms)']):
                        raise ValueError("Time column must be numeric")
                    if not pd.api.types.is_numeric_dtype(df['Power Consumption (mA)']):
                        raise ValueError("Power column must be numeric")
                    
                    # Check data ranges
                    if df['Time (ms)'].min() < 0 or df['Time (ms)'].max() > 10000:
                        raise ValueError("Time values out of expected range (0-10000 ms)")
                    if df['Power Consumption (mA)'].abs().max() > 1000:
                        raise ValueError("Power values exceed reasonable range (±1000 mA)")
                    
                    file_uploaded = True
                    st.success("CSV file successfully uploaded and validated!")
                except Exception as e:
                    st.error(f"Data Validation Error: {str(e)}")
                    status.update(label="Data processing failed", state="error", expanded=False)
                    return
            elif use_sample_data:
                df, operation_points = generate_sample_data()
                st.success("Using sample data for analysis")
            
            # Validate data
            if df is None or df.empty:
                st.error("No data available. Please upload a file or use sample data.")
                status.update(label="Data processing failed", state="error", expanded=False)
                return
            
            # Check required columns
            required_columns = ["Time (ms)", "Power Consumption (mA)"]
            if not all(col in df.columns for col in required_columns):
                missing = [col for col in required_columns if col not in df.columns]
                st.error(f"""
                **Missing columns:** {', '.join(missing)}  
                **Required columns:** {', '.join(required_columns)}
                """)
                status.update(label="Data processing failed", state="error", expanded=False)
                return
            
            # Store for report
            st.session_state.report_data = {
                'df': df,
                'operation_points': operation_points,
                'file_uploaded': file_uploaded
            }
            
            # Show data summary
            st.markdown("""
            <div class="summary-box">
                <h4>Data Summary</h4>
                <ul>
                    <li><b>Time Points:</b> {}</li>
                    <li><b>Duration:</b> {:.2f} ms</li>
                    <li><b>Avg Power:</b> {:.2f} mA</li>
                    <li><b>Max Power:</b> {:.2f} mA</li>
                    <li><b>Data Source:</b> {}</li>
                </ul>
            </div>
            """.format(
                len(df),
                df['Time (ms)'].iloc[-1],
                df['Power Consumption (mA)'].mean(),
                df['Power Consumption (mA)'].max(),
                "User Upload" if file_uploaded else "Sample Data"
            ), unsafe_allow_html=True)
            
            # Plot power traces
            st.subheader("Power Trace Data")
            if use_sample_data:
                st.markdown("<p style='color: #666;'>Simulated AES-256 cryptographic operation data</p>", unsafe_allow_html=True)
                st.markdown(create_download_link(df), unsafe_allow_html=True)
            
            fig = plot_power_traces(df, operation_points=operation_points)
            st.plotly_chart(fig, use_container_width=True)
            
            status.update(label="Data processed successfully", state="complete", expanded=False)
        
        # Step 2: Analyze power traces
        with st.status("Analyzing power traces...", expanded=True) as status:
            # Run signal analysis
            leakage_points = AdvancedSignalAnalyzer.detect_leakage_points(
                st.session_state.report_data['df'], 
                sensitivity=sensitivity,
                confidence=confidence/100
            )
            overall_risk, risk_level = AdvancedSignalAnalyzer.calculate_correlation(leakage_points)
            
            # Update plot with leakage points
            st.subheader("Analysis Results")
            fig = plot_power_traces(
                st.session_state.report_data['df'], 
                leakage_points=leakage_points,
                operation_points=st.session_state.report_data.get('operation_points', [])
            )
            st.plotly_chart(fig, use_container_width=True)
            
            # Use Granite for interpretation
            st.info("Synthesizing results with IBM Granite...")
            analysis = EnhancedGraniteModels.analyze_results(
                leakage_points, 
                overall_risk, 
                risk_level,
                model_choice=model_choice
            )
            
            # Store results for report
            st.session_state.analysis_results = {
                'leakage_points': leakage_points,
                'overall_risk': overall_risk,
                'risk_level': risk_level,
                'analysis': analysis
            }
            
            status.update(label="Analysis completed", state="complete", expanded=False)
        
        # Step 3: Display vulnerability analysis
        st.subheader("Vulnerability Analysis Results")
        st.caption(f"Sensitivity: {sensitivity}/10 | Confidence: {confidence}% | Model: {analysis.get('model_used', 'Granite')}")
        
        # Risk assessment
        risk_score = overall_risk * 100
        risk_color = "#ff3d00" if risk_level == "High" else "#ff9100" if risk_level == "Medium" else "#00c853"
        
        st.markdown(f"""
        <div class="risk-box">
            <h4>Risk Assessment</h4>
            {risk_meter(risk_score)}
            <div class="risk-details">
                <table>
                    <tr>
                        <td><b>Risk Level:</b></td>
                        <td style='color:{risk_color}; font-weight:bold;'>{risk_level}</td>
                        <td><b>Analysis Model:</b></td>
                        <td>{analysis.get('model_used', 'IBM Granite')}</td>
                    </tr>
                    <tr>
                        <td><b>Exploit Difficulty:</b></td>
                        <td>{analysis.get('exploit_difficulty', 'Unknown')}</td>
                        <td><b>Confidence Score:</b></td>
                        <td>{analysis.get('confidence_score', 0.0):.2f}</td>
                    </tr>
                </table>
            </div>
        </div>
        """, unsafe_allow_html=True)
        
        # Vulnerability summary
        st.markdown(f"""
        <div class="vulnerability-summary">
            <h4>Vulnerability Summary</h4>
            <p>{analysis.get("vulnerability_summary", "Summary unavailable")}</p>
        </div>
        """, unsafe_allow_html=True)
        
        # Affected operations
        if analysis.get("affected_operations"):
            st.subheader("Affected Cryptographic Operations")
            for op in analysis["affected_operations"]:
                st.markdown(f"- {op}")
        
        # Business risk
        st.subheader("Business Impact")
        st.markdown(analysis.get('business_impact', 'Risk assessment unavailable'))
        
        # Step 4: Generate countermeasures
        with st.status("Generating security recommendations...", expanded=True) as status:
            st.info("Generating countermeasures using IBM Granite...")
            countermeasures, implementations, effectiveness, intro = EnhancedGraniteModels.generate_countermeasures(
                analysis,
                model_choice=model_choice
            )
            
            # Store for report
            st.session_state.analysis_results['countermeasures'] = countermeasures
            st.session_state.analysis_results['implementations'] = implementations
            st.session_state.analysis_results['effectiveness'] = effectiveness
            
            status.update(label="Countermeasures generated", state="complete", expanded=False)
        
        # Display countermeasures
        st.subheader("Security Recommendations")
        st.caption(f"{intro} | Model: {model_choice}")
        
        for i, (cm, imp, eff) in enumerate(zip(countermeasures, implementations, effectiveness), 1):
            eff_color = "#4caf50" if eff == "High" else "#ff9800" if eff == "Medium" else "#f44336"
            st.markdown(f"""
            <div class="countermeasure-box">
                <div class="cm-header">
                    <span class="cm-number">{i}</span>
                    <span class="cm-title">{cm}</span>
                    <span class="cm-effectiveness" style='background-color:{eff_color};'>{eff}</span>
                </div>
                <div class="cm-implementation">
                    <b>Implementation:</b> {imp}
                </div>
            </div>
            """, unsafe_allow_html=True)
        
        # Implementation roadmap
        st.subheader("Implementation Roadmap")
        st.markdown("""
        <div class="section">
            <table>
                <tr style="background-color: #f5f5f5;">
                    <th>Phase</th>
                    <th>Activities</th>
                    <th>Timeline</th>
                </tr>
                <tr>
                    <td><b>Phase 1</b></td>
                    <td>Implement masking and shuffling countermeasures</td>
                    <td style="text-align:center;">2-4 weeks</td>
                </tr>
                <tr style="background-color: #f9f9f9;">
                    <td><b>Phase 2</b></td>
                    <td>Add noise generation circuits and constant-time implementations</td>
                    <td style="text-align:center;">3-5 weeks</td>
                </tr>
                <tr>
                    <td><b>Phase 3</b></td>
                    <td>Verification and penetration testing</td>
                    <td style="text-align:center;">2-3 weeks</td>
                </tr>
            </table>
            
            <div style="margin-top: 15px; padding: 15px; background: #e3f2fd; border-radius: 5px;">
                <b>Verification Steps:</b>
                <ol>
                    <li>Re-run power analysis after each mitigation</li>
                    <li>Perform correlation coefficient validation</li>
                    <li>Conformal equivalence checking</li>
                    <li>100% coverage testing of vulnerable operations</li>
                    <li>Penetration testing with real-world attack scenarios</li>
                </ol>
            </div>
        </div>
        """, unsafe_allow_html=True)
    
    # Export report button
    if st.session_state.analysis_results:
        if st.button("Export Full Report", use_container_width=True):
            # Generate HTML report
            try:
                html_content = generate_html_report(st.session_state.report_data, st.session_state.analysis_results)
                
                # Create download link
                b64 = base64.b64encode(html_content.encode()).decode()
                url = f'data:text/html;base64,{b64}'
                
                st.markdown(
                    f"<div class='success-message'>Report generated successfully!</div>"
                    f"<a href='{url}' download='sidechannel_report.html' class='download-link'>"
                    "Download Full Report"
                    "</a>", unsafe_allow_html=True)
            except Exception as e:
                st.error(f"Report generation failed: {str(e)}")

def generate_html_report(report_data, analysis_results):
    """Generate comprehensive HTML report"""
    # Create plot image
    fig = plot_power_traces(
        report_data['df'], 
        leakage_points=analysis_results.get('leakage_points', []),
        operation_points=report_data.get('operation_points', [])
    )
    plot_html = fig.to_html(full_html=False, include_plotlyjs='cdn')
    
    # Risk meter
    risk_score = analysis_results.get('overall_risk', 0) * 100
    risk_level = analysis_results.get('risk_level', 'Low')
    
    # Countermeasures table
    cm_table = ""
    for i, (cm, imp, eff) in enumerate(zip(
        analysis_results.get('countermeasures', []),
        analysis_results.get('implementations', []),
        analysis_results.get('effectiveness', [])
    ), 1):
        eff_color = "green" if eff == "High" else "orange" if eff == "Medium" else "red"
        cm_table += f"""
        <tr>
            <td>{i}</td>
            <td>{cm}</td>
            <td>{imp}</td>
            <td style="color:{eff_color}">{eff}</td>
        </tr>
        """
    
    # Create HTML structure
    return f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>SideChannel Sentinel Report</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; }}
            .container {{ max-width: 1200px; margin: 0 auto; padding: 20px; }}
            .header {{ background: #0068c9; color: white; padding: 30px; text-align: center; border-radius: 8px; }}
            .section {{ margin: 30px 0; padding: 20px; background: white; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.05); }}
            .risk-meter {{ background: #f8f9fa; padding: 20px; border-radius: 8px; }}
            table {{ width: 100%; border-collapse: collapse; }}
            th, td {{ padding: 12px 15px; text-align: left; border-bottom: 1px solid #ddd; }}
            th {{ background-color: #f5f5f5; }}
            .countermeasure-row:nth-child(even) {{ background-color: #f9f9f9; }}
            .plot-container {{ margin: 20px 0; }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>SideChannel Sentinel Report</h1>
                <p>Comprehensive Side-Channel Vulnerability Analysis</p>
                <p>Generated on {time.strftime("%Y-%m-%d %H:%M:%S")}</p>
            </div>
            
            <div class="section">
                <h2>Executive Summary</h2>
                <p><b>Overall Risk:</b> <span style="color:{"#ff3d00" if risk_level=="High" else "#ff9100" if risk_level=="Medium" else "#00c853"}">{risk_level}</span> ({risk_score:.1f}%)</p>
                <p><b>Vulnerability Summary:</b> {analysis_results.get('analysis', {}).get('vulnerability_summary', '')}</p>
                <p><b>Business Impact:</b> {analysis_results.get('analysis', {}).get('business_impact', '')}</p>
            </div>
            
            <div class="section">
                <h2>Power Trace Analysis</h2>
                <div class="plot-container">
                    {plot_html}
                </div>
            </div>
            
            <div class="section">
                <h2>Detailed Vulnerability Assessment</h2>
                <p><b>Affected Operations:</b> {', '.join(analysis_results.get('analysis', {}).get('affected_operations', [])) or 'None detected'}</p>
                <p><b>Exploit Difficulty:</b> {analysis_results.get('analysis', {}).get('exploit_difficulty', 'Unknown')}</p>
                <p><b>Analysis Confidence:</b> {analysis_results.get('analysis', {}).get('confidence_score', 0.0)*100:.1f}%</p>
            </div>
            
            <div class="section">
                <h2>Security Recommendations</h2>
                <table>
                    <tr>
                        <th>#</th>
                        <th>Countermeasure</th>
                        <th>Implementation</th>
                        <th>Effectiveness</th>
                    </tr>
                    {cm_table}
                </table>
            </div>
            
            <div class="section">
                <h2>Implementation Roadmap</h2>
                <table>
                    <tr>
                        <th>Phase</th>
                        <th>Activities</th>
                        <th>Timeline</th>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 1</td>
                        <td>Implement critical countermeasures</td>
                        <td>2-4 weeks</td>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 2</td>
                        <td>Hardware modifications and verification</td>
                        <td>3-5 weeks</td>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 3</td>
                        <td>Penetration testing and validation</td>
                        <td>2-3 weeks</td>
                    </tr>
                </table>
            </div>
        </div>
    </body>
    </html>
    """

if __name__ == "__main__":
    main()

Overwriting app.py


In [22]:
import subprocess
import threading
import time
import socket
from IPython.display import display, HTML

def find_free_port():
    """Find a free port to run Streamlit on"""
    with socket.socket() as s:
        s.bind(('', 0))
        return s.getsockname()[1]

def run_streamlit(port):
    """Run Streamlit app in a subprocess"""
    command = f"streamlit run app.py --server.port {port} --server.headless true"
    subprocess.run(command.split())

# Find a free port
port = find_free_port()

# Start Streamlit in a separate thread
thread = threading.Thread(target=run_streamlit, args=(port,))
thread.daemon = True
thread.start()

# Wait for the server to start
time.sleep(2)

# Display the app in an iframe
display(HTML(f"""
<iframe src="http://localhost:{port}" width="100%" height="800" style="border:none;"></iframe>
"""))

In [26]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import base64
from io import BytesIO
import plotly.graph_objects as go
import time
import json
import re
import requests
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.wml_client_error import ApiRequestFailure, WMLClientError
import logging
import os

# ======================
# LOGGING CONFIGURATION
# ======================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger('SideChannelSentinel')

# ======================
# IBM WATSONX CONFIGURATION
# ======================
class WatsonxConfig:
    @staticmethod
    def get_model(model_id, max_tokens=500, temperature=0.2):
        """Initialize IBM Granite model with credentials and retry logic"""
        for attempt in range(3):
            try:
                model = Model(
                    model_id=model_id,
                    credentials={
                        "apikey": os.environ["IBM_API_KEY"],
                        "url": os.environ["IBM_URL"]
                    },
                    project_id=os.environ["IBM_PROJECT_ID"],
                    params={
                        GenParams.DECODING_METHOD: "greedy",
                        GenParams.MAX_NEW_TOKENS: max_tokens,
                        GenParams.MIN_NEW_TOKENS: 50,
                        GenParams.TEMPERATURE: temperature,
                        GenParams.TOP_P: 0.9,
                        GenParams.TOP_K: 50
                    }
                )
                return model
            except (ApiRequestFailure, WMLClientError, requests.RequestException) as e:
                wait_time = (2 ** attempt) + 1
                logger.error(f"IBM API error (attempt {attempt+1}/3): {str(e)}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
            except Exception as e:
                logger.error(f"Unexpected error initializing model: {str(e)}")
                return None
        logger.error("Failed to initialize model after 3 attempts")
        return None

# ======================
# ENHANCED SIGNAL PROCESSING
# ======================
class AdvancedSignalAnalyzer:
    @staticmethod
    def detect_leakage_points(df, sensitivity=7, confidence=0.85):
        """Advanced leakage detection with multiple techniques"""
        logger.info("Starting leakage detection analysis")
        
        # 1. Statistical Analysis with Adaptive Thresholding
        window_size = max(10, int(len(df) * 0.02))  # Dynamic window size
        df['MA'] = df['Power Consumption (mA)'].rolling(window=window_size).mean()
        df['STD'] = df['Power Consumption (mA)'].rolling(window=window_size).std()
        
        # Adaptive threshold based on sensitivity
        sensitivity_multiplier = 0.5 + (sensitivity / 10) * 4.5  # 0.5 to 5.0
        df['Threshold'] = df['STD'].rolling(window=100, min_periods=1).median() * sensitivity_multiplier
        
        # 2. Simulated Correlation Power Analysis (CPA)
        # Generate hypothetical power model (Hamming weight model)
        hypothetical_power = AdvancedSignalAnalyzer.simulate_hamming_model(df)
        df['CPA_Correlation'] = df['Power Consumption (mA)'].rolling(window=window_size).corr(hypothetical_power)
        
        # 3. Combined detection logic
        high_deviation = df[(df['STD'] > df['Threshold']) | 
                            (df['CPA_Correlation'].abs() > confidence)]
        
        leakage_points = []
        if not high_deviation.empty:
            # Group consecutive points
            high_deviation['Group'] = (high_deviation.index.to_series().diff() > 1).cumsum()
            
            for _, group in high_deviation.groupby('Group'):
                peak_idx = group['STD'].idxmax()
                peak_time = df.loc[peak_idx, 'Time (ms)']
                peak_value = df.loc[peak_idx, 'Power Consumption (mA)']
                
                # Calculate feature vector for LLM
                features = {
                    'duration': group['Time (ms)'].max() - group['Time (ms)'].min(),
                    'avg_power': group['Power Consumption (mA)'].mean(),
                    'max_power': group['Power Consumption (mA)'].max(),
                    'cpa_corr': group['CPA_Correlation'].abs().mean(),
                    'std_dev': group['STD'].mean()
                }
                
                # Confidence score based on multiple factors
                conf_score = min(0.95, (features['std_dev'] / df['Threshold'].mean() * 0.4 + 
                                        features['cpa_corr'] * 0.6))
                
                leakage_points.append({
                    'time_ms': peak_time,
                    'power_value': peak_value,
                    'features': features,
                    'confidence': conf_score
                })
        
        logger.info(f"Detected {len(leakage_points)} potential leakage points")
        return leakage_points

    @staticmethod
    def simulate_hamming_model(df):
        """Simulate CPA Hamming weight model for demonstration"""
        # In real CPA, this would be based on intermediate values
        # Here we simulate with a synthetic pattern
        np.random.seed(42)
        base = np.sin(2 * np.pi * df['Time (ms)'] * 5) * 0.3
        spikes = np.zeros(len(df))
        
        # Add spikes at random positions to simulate operation points
        for _ in range(5):
            idx = np.random.randint(0, len(df))
            width = np.random.uniform(0.05, 0.2)
            spike = np.exp(-(df['Time (ms)'] - df.loc[idx, 'Time (ms)'])**2 / (2 * width**2))
            spikes += spike * np.random.uniform(0.5, 1.5)
        
        return base + spikes

    @staticmethod
    def calculate_correlation(leakage_points):
        """Calculate correlation metrics for leakage points"""
        if not leakage_points:
            return 0.0, "Low"
        
        # Weighted average of multiple factors
        total_risk = 0
        weights = {'std_dev': 0.4, 'cpa_corr': 0.5, 'duration': 0.1}
        
        for point in leakage_points:
            features = point['features']
            risk_score = (features['std_dev'] * weights['std_dev'] +
                          features['cpa_corr'] * weights['cpa_corr'] +
                          features['duration'] * weights['duration'])
            total_risk += risk_score * point['confidence']
        
        overall_risk = min(1.0, total_risk / len(leakage_points))
        
        # Determine risk level
        if overall_risk > 0.7:
            risk_level = "High"
        elif overall_risk > 0.5:
            risk_level = "Medium"
        else:
            risk_level = "Low"
            
        return overall_risk, risk_level

# ======================
# ENHANCED GRANITE MODELS INTEGRATION
# ======================
class EnhancedGraniteModels:
    @staticmethod
    def analyze_results(leakage_points, overall_risk, risk_level, model_choice="tiny"):
        """Enhanced analysis with dynamic model selection"""
        model_id_map = {
            "tiny": "ibm/granite-3-2b-instruct",
            "medium": "ibm/granite-3-8b-instruct",
            "instruct": "ibm/granite-13b-instruct-v2"
        }
        
        model_id = model_id_map.get(model_choice.lower(), "ibm/granite-3-2b-instruct")
        logger.info(f"Using model: {model_id} for vulnerability analysis")
        
        try:
            model = WatsonxConfig.get_model(model_id, max_tokens=600)
            if not model:
                return EnhancedGraniteModels.simulate_analysis_summary(leakage_points, overall_risk)
            
            # Prepare detailed leakage points description
            leakage_desc = "\n".join(
                f"- At {point['time_ms']:.2f} ms: "
                f"Power={point['power_value']:.2f}mA, "
                f"Duration={point['features']['duration']:.4f}ms, "
                f"CPA_Corr={point['features']['cpa_corr']:.2f}, "
                f"Confidence={point['confidence']:.2f}"
                for point in leakage_points
            ) if leakage_points else "No significant leakage points detected"
            
            # Enhanced prompt with more context
            prompt = f"""
            [INST] <<SYS>>
            You are a hardware security expert analyzing side-channel power traces. 
            Provide a technical assessment of these findings:

            DETECTION RESULTS:
            {leakage_desc}

            OVERALL RISK: {overall_risk:.2f} ({risk_level})
            
            ANALYSIS TASKS:
            1. Vulnerability summary (1-2 sentences)
            2. Identify cryptographic operations affected by leakage
            3. Assess exploit difficulty (Low/Medium/High)
            4. Business impact assessment
            5. Return response in JSON format:
            {{
                "vulnerability_summary": "Summary text",
                "affected_operations": ["operation1", "operation2"],
                "exploit_difficulty": "Low/Medium/High",
                "business_impact": "Impact explanation",
                "confidence_score": 0.0-1.0
            }}
            <</SYS>>
            [/INST]
            """
            
            # Generate analysis with retry
            response = EnhancedGraniteModels.retry_api_call(model.generate, prompt=prompt)
            analysis = response['results'][0]['generated_text']
            
            # Extract JSON safely
            result = EnhancedGraniteModels.extract_json(analysis)
            result['model_used'] = model_id.split('/')[-1]
            return result
        except Exception as e:
            logger.error(f"Analysis error: {str(e)}")
            return EnhancedGraniteModels.simulate_analysis_summary(leakage_points, overall_risk)
    
    @staticmethod
    def retry_api_call(func, *args, **kwargs):
        """Retry API call with exponential backoff"""
        for attempt in range(3):
            try:
                return func(*args, **kwargs)
            except (ApiRequestFailure, requests.RequestException) as e:
                wait_time = (2 ** attempt) + 1
                logger.warning(f"API call failed (attempt {attempt+1}/3): {str(e)}. Retrying in {wait_time}s...")
                time.sleep(wait_time)
        raise Exception("API call failed after 3 attempts")

    @staticmethod
    def extract_json(text):
        """Robust JSON extraction from LLM response"""
        try:
            # Improved pattern to find JSON objects
            json_match = re.search(r'\{[\s\S]*\}', text)
            if not json_match:
                raise ValueError("No JSON found in response")
                
            json_str = json_match.group(0)
            return json.loads(json_str)
        except (ValueError, json.JSONDecodeError) as e:
            logger.error(f"JSON extraction error: {str(e)}")
            return {
                "vulnerability_summary": "Analysis summary unavailable",
                "affected_operations": [],
                "business_impact": "Unable to generate business impact assessment",
                "exploit_difficulty": "Unknown",
                "confidence_score": 0.0
            }
    
    @staticmethod
    def simulate_analysis_summary(leakage_points, overall_risk):
        """Fallback simulation if API fails"""
        summary = "Significant correlation detected between power consumption and secret data"
        operations = ["S-Box substitution", "Key schedule"]
        
        if overall_risk > 0.7:
            impact = "High risk of key extraction leading to complete system compromise"
            difficulty = "Medium (requires specialized equipment)"
        elif overall_risk > 0.5:
            impact = "Moderate risk of partial key recovery"
            difficulty = "High (requires advanced expertise)"
        else:
            impact = "Low risk in practical scenarios"
            difficulty = "Very High (theoretical risk only)"
        
        return {
            "vulnerability_summary": summary,
            "affected_operations": operations,
            "business_impact": impact,
            "exploit_difficulty": difficulty,
            "confidence_score": min(0.9, overall_risk + 0.2)
        }
    
    @staticmethod
    def generate_countermeasures(analysis, model_choice="instruct"):
        """Generate targeted countermeasures with model choice"""
        model_id_map = {
            "instruct": "ibm/granite-13b-instruct-v2",
            "medium": "ibm/granite-3-8b-instruct",
            "tiny": "ibm/granite-3-2b-instruct"
        }
        
        model_id = model_id_map.get(model_choice.lower(), "ibm/granite-13b-instruct-v2")
        logger.info(f"Using model: {model_id} for countermeasures")
        
        try:
            model = WatsonxConfig.get_model(model_id, max_tokens=800, temperature=0.3)
            if not model:
                return EnhancedGraniteModels.simulate_countermeasures()
            
            # Prepare prompt with targeted operations
            operations = analysis.get('affected_operations', [])
            targeted_ops = f"Specifically target: {', '.join(operations)}" if operations else ""
            
            prompt = f"""
            [INST] <<SYS>>
            As a hardware security specialist, suggest countermeasures for these vulnerabilities:
            
            {analysis.get('vulnerability_summary', 'Vulnerability analysis unavailable')}
            
            Affected Operations: {', '.join(operations) or 'Not specified'}
            Exploit Difficulty: {analysis.get('exploit_difficulty', 'Unknown')}
            
            Requirements:
            1. Provide 5 hardware-focused countermeasures
            2. For each: 
               - Brief description
               - Implementation approach
               - Estimated effectiveness (High/Medium/Low)
            3. Format: 
               "1. [Description] 
                Implementation: [Details] 
                Effectiveness: [Rating]"
            4. Prioritize countermeasures addressing the specific operations mentioned
            {targeted_ops}
            <</SYS>>
            [/INST]
            """
            
            # Generate countermeasures
            response = EnhancedGraniteModels.retry_api_call(model.generate, prompt=prompt)
            text = response['results'][0]['generated_text']
            
            # Parse with enhanced regex
            return EnhancedGraniteModels.parse_countermeasures(text)
        except Exception as e:
            logger.error(f"Countermeasures error: {str(e)}")
            return EnhancedGraniteModels.simulate_countermeasures()
    
    @staticmethod
    def parse_countermeasures(text):
        """Robust parsing of countermeasures with effectiveness"""
        pattern = r"(\d+)\.\s*([^\n]+?)\n\s*Implementation:\s*(.*?)\n\s*Effectiveness:\s*(High|Medium|Low)"
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
        
        countermeasures = []
        implementations = []
        effectiveness = []
        
        for num, desc, impl, eff in matches[:5]:
            countermeasures.append(desc.strip())
            implementations.append(impl.strip())
            effectiveness.append(eff.strip())
        
        # Fill any missing with simulated
        sim_cm, sim_imp, sim_eff, _ = EnhancedGraniteModels.simulate_countermeasures()
        for i in range(5 - len(countermeasures)):
            countermeasures.append(sim_cm[i])
            implementations.append(sim_imp[i])
            effectiveness.append(sim_eff[i])
            
        intro = f"Generated by IBM Granite model"
        return countermeasures, implementations, effectiveness, intro
    
    @staticmethod
    def simulate_countermeasures():
        """Fallback simulation if API fails"""
        countermeasures = [
            "Implement Boolean masking for S-Box operations",
            "Add dynamic noise generation circuits",
            "Use constant-time implementations for all operations",
            "Implement operation shuffling with PRNG",
            "Add random delays between cryptographic operations"
        ]
        
        implementations = [
            "Apply additive/multiplicative masking to intermediate values",
            "Use LFSR-based noise generator with adjustable amplitude",
            "Ensure all branches and memory accesses take fixed time",
            "Randomize processing order using hardware PRNG",
            "Insert delays with Poisson distribution between rounds"
        ]
        
        effectiveness = ["High", "Medium", "High", "Medium", "Low"]
        
        intro = "Simulated countermeasures (API unavailable)"
        
        return countermeasures, implementations, effectiveness, intro

# ======================
# ENHANCED VISUALIZATION
# ======================
def generate_sample_data():
    """Create realistic power trace simulation data with CPA features"""
    np.random.seed(42)
    time_points = np.linspace(0, 10, 1000)
    base_signal = np.sin(2 * np.pi * time_points) * 0.5
    
    # Add cryptographic operation signatures
    operation_points = [
        {"time_ms": 1.5, "magnitude": 1.8, "width": 0.2, "operation": "S-Box substitution"},
        {"time_ms": 3.2, "magnitude": 1.5, "width": 0.15, "operation": "Key schedule"},
        {"time_ms": 5.0, "magnitude": 0.8, "width": 0.1, "operation": "AddRoundKey"},
        {"time_ms": 7.8, "magnitude": 1.2, "width": 0.25, "operation": "Final round"}
    ]
    
    for point in operation_points:
        idx = np.abs(time_points - point['time_ms']).argmin()
        gaussian = point['magnitude'] * np.exp(-(time_points - point['time_ms'])**2 / (2 * point['width']**2))
        base_signal += gaussian
    
    # Add noise and high-frequency components
    base_signal += np.random.normal(0, 0.15, len(time_points))
    base_signal += 0.3 * np.sin(15 * np.pi * time_points)
    base_signal += 0.2 * np.random.random(len(time_points))
    
    return pd.DataFrame({
        "Time (ms)": time_points,
        "Power Consumption (mA)": base_signal
    }), operation_points

def plot_power_traces(df, leakage_points=None, operation_points=None):
    """Enhanced plot with interactive leakage points"""
    fig = go.Figure()
    
    # Main power trace
    fig.add_trace(go.Scatter(
        x=df["Time (ms)"],
        y=df["Power Consumption (mA)"],
        mode='lines',
        name='Power Consumption',
        line=dict(color='#0068c9', width=2),
        fill='tozeroy',
        fillcolor='rgba(0, 104, 201, 0.1)',
        hovertemplate='Time: %{x:.2f} ms<br>Power: %{y:.2f} mA<extra></extra>'
    ))
    
    # Known operation points
    if operation_points:
        for point in operation_points:
            fig.add_vline(
                x=point['time_ms'],
                line=dict(color="#888", width=1.5, dash="dot"),
                annotation_text=point["operation"],
                annotation_position="top right",
                annotation_font_size=10
            )
    
    # Detected leakage points
    if leakage_points:
        leak_x = [point['time_ms'] for point in leakage_points]
        leak_y = [point['power_value'] for point in leakage_points]
        leak_text = [
            f"<b>Leakage Point</b><br>Time: {p['time_ms']:.2f} ms<br>Power: {p['power_value']:.2f} mA"
            f"<br>Confidence: {p['confidence']:.2f}<br>CPA Corr: {p['features']['cpa_corr']:.2f}"
            for p in leakage_points
        ]
        
        fig.add_trace(go.Scatter(
            x=leak_x,
            y=leak_y,
            mode='markers',
            name='Leakage Detected',
            marker=dict(size=12, color='red', symbol='x'),
            hovertemplate='%{text}<extra></extra>',
            text=leak_text
        ))
    
    fig.update_layout(
        title='Power Consumption Analysis',
        xaxis_title='Time (ms)',
        yaxis_title='Power Consumption (mA)',
        template='plotly_white',
        height=450,
        hovermode="x unified",
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    return fig

def risk_meter(value):
    """Create a visual risk meter with improved design"""
    value = min(100, max(0, value))
    level = min(3, int(value // 25))
    colors = ['#00c853', '#ffd600', '#ff9100', '#ff3d00']
    
    html = f"""
    <div style="background: #f8f9fa; border-radius: 10px; padding: 15px; margin: 10px 0; position: relative; box-shadow: 0 2px 4px rgba(0,0,0,0.05);">
        <div style="position: absolute; top: 15px; right: 15px; font-weight: bold; font-size: 24px; color: {colors[level]}">
            {value:.1f}%
        </div>
        <div style="margin-top: 10px;">
            <div style="display: flex; justify-content: space-between; margin-bottom: 5px; font-size: 12px;">
                <span>Low</span>
                <span>Medium</span>
                <span>High</span>
                <span>Critical</span>
            </div>
            <div style="background: #e0e0e0; height: 20px; border-radius: 10px; overflow: hidden;">
                <div style="background: linear-gradient(to right, {', '.join(colors)}); 
                            width: {value}%; height: 100%; 
                            position: relative;">
                </div>
            </div>
        </div>
        <div style="margin-top: 15px; font-size: 14px; color: #666;">
            Risk level indicates likelihood of key extraction
        </div>
    </div>
    """
    return html

def create_download_link(df, filename="power_traces.csv"):
    """Generate a download link for CSV file"""
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    return f'<a href="data:file/csv;base64,{b64}" download="{filename}" style="color: #0068c9; text-decoration: none; border: 1px solid #0068c9; border-radius: 4px; padding: 6px 12px; display: inline-block; margin: 10px 0;">Download CSV File</a>'

# ======================
# STREAMLIT UI
# ======================
def main():
    # Set up environment variables for IBM Watson
    os.environ["IBM_API_KEY"] = "RyIKeWjkIi-jx4F2uerWwUuInh6d9hoIJal8frtzKIMk"
    os.environ["IBM_URL"] = "https://us-south.ml.cloud.ibm.com"
    os.environ["IBM_PROJECT_ID"] = "5d59376b-9172-474f-ac91-c33d1f16bb7b"
    
    # Custom CSS styling
    st.markdown("""
    <style>
        .header {
            background: #0068c9;
            color: white;
            padding: 30px;
            text-align: center;
            border-radius: 8px;
            margin-bottom: 20px;
        }
        .section {
            margin: 20px 0;
            padding: 20px;
            background: white;
            border-radius: 8px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.05);
        }
        .risk-box {
            background: #f0f2f6;
            border-radius: 10px;
            padding: 15px;
            margin-bottom: 20px;
        }
        .risk-details {
            margin-top: 15px;
            background: white;
            padding: 10px;
            border-radius: 5px;
        }
        .vulnerability-summary {
            background: #fff8e1;
            border-left: 4px solid #ffc107;
            padding: 15px;
            margin: 20px 0;
            border-radius: 0 8px 8px 0;
        }
        .countermeasure-box {
            margin-bottom: 15px;
            padding: 15px;
            background: white;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.05);
        }
        .cm-header {
            display: flex;
            align-items: center;
            margin-bottom: 10px;
        }
        .cm-number {
            background: #0068c9;
            color: white;
            width: 28px;
            height: 28px;
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            margin-right: 10px;
            font-weight: bold;
        }
        .cm-title {
            flex-grow: 1;
            font-weight: bold;
            color: #0068c9;
        }
        .cm-effectiveness {
            padding: 4px 10px;
            border-radius: 20px;
            font-size: 0.8em;
            color: white;
        }
        .cm-implementation {
            margin-top: 8px;
            padding-left: 15px;
            border-left: 2px solid #e0e0e0;
            color: #555;
        }
        .progress-step {
            background: #e3f2fd;
            padding: 10px 15px;
            border-radius: 5px;
            margin: 10px 0;
            border-left: 4px solid #0068c9;
        }
        .success-message {
            color: #2e7d32;
            background: #e8f5e9;
            padding: 10px;
            border-radius: 5px;
            border-left: 4px solid #4caf50;
            margin: 10px 0;
        }
        .error-message {
            color: #c62828;
            background: #ffebee;
            padding: 15px;
            border-radius: 5px;
            border-left: 4px solid #f44336;
            margin: 15px 0;
        }
        .summary-box {
            background: #f8f9fa;
            padding: 15px;
            border-radius: 8px;
            border-left: 4px solid #0068c9;
            margin: 15px 0;
        }
        .download-link {
            display: inline-block;
            background: #0068c9;
            color: white;
            text-decoration: none;
            padding: 12px 24px;
            border-radius: 5px;
            font-weight: bold;
            margin: 15px 0;
            transition: background 0.3s;
        }
        .download-link:hover {
            background: #0056b3;
        }
    </style>
    """, unsafe_allow_html=True)
    
    # Initialize session state
    if 'analysis_results' not in st.session_state:
        st.session_state.analysis_results = {}
    if 'report_data' not in st.session_state:
        st.session_state.report_data = {}
    
    # Header
    st.markdown("""
    <div class="header">
        <h1>🔒 SideChannel Sentinel Pro</h1>
        <h3>Advanced Side-Channel Vulnerability Detection</h3>
        <p>Signal Processing + CPA + IBM Granite for Comprehensive Hardware Security</p>
    </div>
    """, unsafe_allow_html=True)
    
    # Data source section
    st.markdown("<h2>Data Source</h2>", unsafe_allow_html=True)
    use_sample_data = st.checkbox('Use sample data for demonstration', value=True)
    uploaded_file = st.file_uploader("Upload CSV file", type=['csv'], 
                                     help="CSV should contain 'Time (ms)' and 'Power Consumption (mA)' columns")
    
    # Analysis configuration
    st.markdown("<h2>Analysis Configuration</h2>", unsafe_allow_html=True)
    col1, col2 = st.columns(2)
    with col1:
        sensitivity = st.slider("Detection Sensitivity:", 1, 10, 5, 
                                help="Higher values detect smaller leaks but may increase false positives")
    with col2:
        confidence = st.slider("Min Confidence Threshold:", 50, 100, 85, 
                               help="Minimum confidence score to consider a leakage point valid")
    
    # FIXED: Corrected selectbox format function
    model_choice = st.selectbox(
        "Analysis Model:", 
        options=[('Granite 3.2B Tiny', 'tiny'), 
                 ('Granite 3.8B Medium', 'medium'),
                 ('Granite 13B Instruct', 'instruct')],
        format_func=lambda x: x[0],  # Use first element of tuple as display text
        index=0
    )
    
    # Extract the internal value from the selected tuple
    if isinstance(model_choice, tuple):
        model_choice = model_choice[1]
    
    # IBM Watsonx Configuration
    st.markdown("""
    <div class="section">
        <h3>IBM Watsonx Configuration</h3>
        <p>Using IBM credentials for Granite models:</p>
        <ul>
            <li><b>Region:</b> us-south</li>
            <li><b>Project ID:</b> 5d59376b-9172-474f-ac91-c33d1f16bb7b</li>
            <li><b>Available Models:</b> Granite 3.2B Tiny, Granite 3.8B Medium, Granite 13B Instruct</li>
        </ul>
    </div>
    """, unsafe_allow_html=True)
    
    # Analyze button
    if st.button("Analyze Power Traces", use_container_width=True, type="primary"):
        st.session_state.analysis_results = {}
        st.session_state.report_data = {}
        
        # Step 1: Process input data
        with st.status("Processing input data...", expanded=True) as status:
            df = None
            operation_points = None
            file_uploaded = False
            
            if uploaded_file is not None:
                try:
                    content = uploaded_file.getvalue().decode('utf-8')
                    df = pd.read_csv(BytesIO(content.encode()))
                    
                    # Validate data types
                    if not pd.api.types.is_numeric_dtype(df['Time (ms)']):
                        raise ValueError("Time column must be numeric")
                    if not pd.api.types.is_numeric_dtype(df['Power Consumption (mA)']):
                        raise ValueError("Power column must be numeric")
                    
                    # Check data ranges
                    if df['Time (ms)'].min() < 0 or df['Time (ms)'].max() > 10000:
                        raise ValueError("Time values out of expected range (0-10000 ms)")
                    if df['Power Consumption (mA)'].abs().max() > 1000:
                        raise ValueError("Power values exceed reasonable range (±1000 mA)")
                    
                    file_uploaded = True
                    st.success("CSV file successfully uploaded and validated!")
                except Exception as e:
                    st.error(f"Data Validation Error: {str(e)}")
                    status.update(label="Data processing failed", state="error", expanded=False)
                    return
            elif use_sample_data:
                df, operation_points = generate_sample_data()
                st.success("Using sample data for analysis")
            
            # Validate data
            if df is None or df.empty:
                st.error("No data available. Please upload a file or use sample data.")
                status.update(label="Data processing failed", state="error", expanded=False)
                return
            
            # Check required columns
            required_columns = ["Time (ms)", "Power Consumption (mA)"]
            if not all(col in df.columns for col in required_columns):
                missing = [col for col in required_columns if col not in df.columns]
                st.error(f"""
                **Missing columns:** {', '.join(missing)}  
                **Required columns:** {', '.join(required_columns)}
                """)
                status.update(label="Data processing failed", state="error", expanded=False)
                return
            
            # Store for report
            st.session_state.report_data = {
                'df': df,
                'operation_points': operation_points,
                'file_uploaded': file_uploaded
            }
            
            # Show data summary
            st.markdown("""
            <div class="summary-box">
                <h4>Data Summary</h4>
                <ul>
                    <li><b>Time Points:</b> {}</li>
                    <li><b>Duration:</b> {:.2f} ms</li>
                    <li><b>Avg Power:</b> {:.2f} mA</li>
                    <li><b>Max Power:</b> {:.2f} mA</li>
                    <li><b>Data Source:</b> {}</li>
                </ul>
            </div>
            """.format(
                len(df),
                df['Time (ms)'].iloc[-1],
                df['Power Consumption (mA)'].mean(),
                df['Power Consumption (mA)'].max(),
                "User Upload" if file_uploaded else "Sample Data"
            ), unsafe_allow_html=True)
            
            # Plot power traces
            st.subheader("Power Trace Data")
            if use_sample_data:
                st.markdown("<p style='color: #666;'>Simulated AES-256 cryptographic operation data</p>", unsafe_allow_html=True)
                st.markdown(create_download_link(df), unsafe_allow_html=True)
            
            fig = plot_power_traces(df, operation_points=operation_points)
            
            # FIX: Add unique key to first plotly_chart
            st.plotly_chart(fig, use_container_width=True, key="initial_trace")
            
            status.update(label="Data processed successfully", state="complete", expanded=False)
        
        # Step 2: Analyze power traces
        with st.status("Analyzing power traces...", expanded=True) as status:
            # Run signal analysis
            leakage_points = AdvancedSignalAnalyzer.detect_leakage_points(
                st.session_state.report_data['df'], 
                sensitivity=sensitivity,
                confidence=confidence/100
            )
            overall_risk, risk_level = AdvancedSignalAnalyzer.calculate_correlation(leakage_points)
            
            # Update plot with leakage points
            st.subheader("Analysis Results")
            fig = plot_power_traces(
                st.session_state.report_data['df'], 
                leakage_points=leakage_points,
                operation_points=st.session_state.report_data.get('operation_points', [])
            )
            
            # FIX: Add unique key to second plotly_chart
            st.plotly_chart(fig, use_container_width=True, key="analysis_trace")
            
            # Use Granite for interpretation
            st.info("Synthesizing results with IBM Granite...")
            analysis = EnhancedGraniteModels.analyze_results(
                leakage_points, 
                overall_risk, 
                risk_level,
                model_choice=model_choice
            )
            
            # Store results for report
            st.session_state.analysis_results = {
                'leakage_points': leakage_points,
                'overall_risk': overall_risk,
                'risk_level': risk_level,
                'analysis': analysis
            }
            
            status.update(label="Analysis completed", state="complete", expanded=False)
        
        # Step 3: Display vulnerability analysis
        st.subheader("Vulnerability Analysis Results")
        st.caption(f"Sensitivity: {sensitivity}/10 | Confidence: {confidence}% | Model: {analysis.get('model_used', 'Granite')}")
        
        # Risk assessment
        risk_score = overall_risk * 100
        risk_color = "#ff3d00" if risk_level == "High" else "#ff9100" if risk_level == "Medium" else "#00c853"
        
        st.markdown(f"""
        <div class="risk-box">
            <h4>Risk Assessment</h4>
            {risk_meter(risk_score)}
            <div class="risk-details">
                <table>
                    <tr>
                        <td><b>Risk Level:</b></td>
                        <td style='color:{risk_color}; font-weight:bold;'>{risk_level}</td>
                        <td><b>Analysis Model:</b></td>
                        <td>{analysis.get('model_used', 'IBM Granite')}</td>
                    </tr>
                    <tr>
                        <td><b>Exploit Difficulty:</b></td>
                        <td>{analysis.get('exploit_difficulty', 'Unknown')}</td>
                        <td><b>Confidence Score:</b></td>
                        <td>{analysis.get('confidence_score', 0.0):.2f}</td>
                    </tr>
                </table>
            </div>
        </div>
        """, unsafe_allow_html=True)
        
        # Vulnerability summary
        st.markdown(f"""
        <div class="vulnerability-summary">
            <h4>Vulnerability Summary</h4>
            <p>{analysis.get("vulnerability_summary", "Summary unavailable")}</p>
        </div>
        """, unsafe_allow_html=True)
        
        # Affected operations
        if analysis.get("affected_operations"):
            st.subheader("Affected Cryptographic Operations")
            for op in analysis["affected_operations"]:
                st.markdown(f"- {op}")
        
        # Business risk
        st.subheader("Business Impact")
        st.markdown(analysis.get('business_impact', 'Risk assessment unavailable'))
        
        # Step 4: Generate countermeasures
        with st.status("Generating security recommendations...", expanded=True) as status:
            st.info("Generating countermeasures using IBM Granite...")
            countermeasures, implementations, effectiveness, intro = EnhancedGraniteModels.generate_countermeasures(
                analysis,
                model_choice=model_choice
            )
            
            # Store for report
            st.session_state.analysis_results['countermeasures'] = countermeasures
            st.session_state.analysis_results['implementations'] = implementations
            st.session_state.analysis_results['effectiveness'] = effectiveness
            
            status.update(label="Countermeasures generated", state="complete", expanded=False)
        
        # Display countermeasures
        st.subheader("Security Recommendations")
        st.caption(f"{intro} | Model: {model_choice}")
        
        for i, (cm, imp, eff) in enumerate(zip(countermeasures, implementations, effectiveness), 1):
            eff_color = "#4caf50" if eff == "High" else "#ff9800" if eff == "Medium" else "#f44336"
            st.markdown(f"""
            <div class="countermeasure-box">
                <div class="cm-header">
                    <span class="cm-number">{i}</span>
                    <span class="cm-title">{cm}</span>
                    <span class="cm-effectiveness" style='background-color:{eff_color};'>{eff}</span>
                </div>
                <div class="cm-implementation">
                    <b>Implementation:</b> {imp}
                </div>
            </div>
            """, unsafe_allow_html=True)
        
        # Implementation roadmap
        st.subheader("Implementation Roadmap")
        st.markdown("""
        <div class="section">
            <table>
                <tr style="background-color: #f5f5f5;">
                    <th>Phase</th>
                    <th>Activities</th>
                    <th>Timeline</th>
                </tr>
                <tr>
                    <td><b>Phase 1</b></td>
                    <td>Implement masking and shuffling countermeasures</td>
                    <td style="text-align:center;">2-4 weeks</td>
                </tr>
                <tr style="background-color: #f9f9f9;">
                    <td><b>Phase 2</b></td>
                    <td>Add noise generation circuits and constant-time implementations</td>
                    <td style="text-align:center;">3-5 weeks</td>
                </tr>
                <tr>
                    <td><b>Phase 3</b></td>
                    <td>Verification and penetration testing</td>
                    <td style="text-align:center;">2-3 weeks</td>
                </tr>
            </table>
            
            <div style="margin-top: 15px; padding: 15px; background: #e3f2fd; border-radius: 5px;">
                <b>Verification Steps:</b>
                <ol>
                    <li>Re-run power analysis after each mitigation</li>
                    <li>Perform correlation coefficient validation</li>
                    <li>Conformal equivalence checking</li>
                    <li>100% coverage testing of vulnerable operations</li>
                    <li>Penetration testing with real-world attack scenarios</li>
                </ol>
            </div>
        </div>
        """, unsafe_allow_html=True)
    
    # Export report button
    if st.session_state.analysis_results:
        if st.button("Export Full Report", use_container_width=True):
            # Generate HTML report
            try:
                html_content = generate_html_report(st.session_state.report_data, st.session_state.analysis_results)
                
                # Create download link
                b64 = base64.b64encode(html_content.encode()).decode()
                url = f'data:text/html;base64,{b64}'
                
                st.markdown(
                    f"<div class='success-message'>Report generated successfully!</div>"
                    f"<a href='{url}' download='sidechannel_report.html' class='download-link'>"
                    "Download Full Report"
                    "</a>", unsafe_allow_html=True)
            except Exception as e:
                st.error(f"Report generation failed: {str(e)}")

def generate_html_report(report_data, analysis_results):
    """Generate comprehensive HTML report"""
    # Create plot image
    fig = plot_power_traces(
        report_data['df'], 
        leakage_points=analysis_results.get('leakage_points', []),
        operation_points=report_data.get('operation_points', [])
    )
    plot_html = fig.to_html(full_html=False, include_plotlyjs='cdn')
    
    # Risk meter
    risk_score = analysis_results.get('overall_risk', 0) * 100
    risk_level = analysis_results.get('risk_level', 'Low')
    
    # Countermeasures table
    cm_table = ""
    for i, (cm, imp, eff) in enumerate(zip(
        analysis_results.get('countermeasures', []),
        analysis_results.get('implementations', []),
        analysis_results.get('effectiveness', [])
    ), 1):
        eff_color = "green" if eff == "High" else "orange" if eff == "Medium" else "red"
        cm_table += f"""
        <tr>
            <td>{i}</td>
            <td>{cm}</td>
            <td>{imp}</td>
            <td style="color:{eff_color}">{eff}</td>
        </tr>
        """
    
    # Create HTML structure
    return f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>SideChannel Sentinel Report</title>
        <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; }}
            .container {{ max-width: 1200px; margin: 0 auto; padding: 20px; }}
            .header {{ background: #0068c9; color: white; padding: 30px; text-align: center; border-radius: 8px; }}
            .section {{ margin: 30px 0; padding: 20px; background: white; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.05); }}
            .risk-meter {{ background: #f8f9fa; padding: 20px; border-radius: 8px; }}
            table {{ width: 100%; border-collapse: collapse; }}
            th, td {{ padding: 12px 15px; text-align: left; border-bottom: 1px solid #ddd; }}
            th {{ background-color: #f5f5f5; }}
            .countermeasure-row:nth-child(even) {{ background-color: #f9f9f9; }}
            .plot-container {{ margin: 20px 0; }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="header">
                <h1>SideChannel Sentinel Report</h1>
                <p>Comprehensive Side-Channel Vulnerability Analysis</p>
                <p>Generated on {time.strftime("%Y-%m-%d %H:%M:%S")}</p>
            </div>
            
            <div class="section">
                <h2>Executive Summary</h2>
                <p><b>Overall Risk:</b> <span style="color:{"#ff3d00" if risk_level=="High" else "#ff9100" if risk_level=="Medium" else "#00c853"}">{risk_level}</span> ({risk_score:.1f}%)</p>
                <p><b>Vulnerability Summary:</b> {analysis_results.get('analysis', {}).get('vulnerability_summary', '')}</p>
                <p><b>Business Impact:</b> {analysis_results.get('analysis', {}).get('business_impact', '')}</p>
            </div>
            
            <div class="section">
                <h2>Power Trace Analysis</h2>
                <div class="plot-container">
                    {plot_html}
                </div>
            </div>
            
            <div class="section">
                <h2>Detailed Vulnerability Assessment</h2>
                <p><b>Affected Operations:</b> {', '.join(analysis_results.get('analysis', {}).get('affected_operations', [])) or 'None detected'}</p>
                <p><b>Exploit Difficulty:</b> {analysis_results.get('analysis', {}).get('exploit_difficulty', 'Unknown')}</p>
                <p><b>Analysis Confidence:</b> {analysis_results.get('analysis', {}).get('confidence_score', 0.0)*100:.1f}%</p>
            </div>
            
            <div class="section">
                <h2>Security Recommendations</h2>
                <table>
                    <tr>
                        <th>#</th>
                        <th>Countermeasure</th>
                        <th>Implementation</th>
                        <th>Effectiveness</th>
                    </tr>
                    {cm_table}
                </table>
            </div>
            
            <div class="section">
                <h2>Implementation Roadmap</h2>
                <table>
                    <tr>
                        <th>Phase</th>
                        <th>Activities</th>
                        <th>Timeline</th>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 1</td>
                        <td>Implement critical countermeasures</td>
                        <td>2-4 weeks</td>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 2</td>
                        <td>Hardware modifications and verification</td>
                        <td>3-5 weeks</td>
                    </tr>
                    <tr class="countermeasure-row">
                        <td>Phase 3</td>
                        <td>Penetration testing and validation</td>
                        <td>2-3 weeks</td>
                    </tr>
                </table>
            </div>
        </div>
    </body>
    </html>
    """

if __name__ == "__main__":
    main()

Overwriting app.py


In [29]:
import subprocess
import threading
import time
import socket
from IPython.display import display, HTML

def find_free_port():
    """Find a free port to run Streamlit on"""
    with socket.socket() as s:
        s.bind(('', 0))
        return s.getsockname()[1]

def run_streamlit(port):
    """Run Streamlit app in a subprocess"""
    command = f"streamlit run app.py --server.port {port} --server.headless true"
    subprocess.run(command.split())

# Find a free port
port = find_free_port()

# Start Streamlit in a separate thread
thread = threading.Thread(target=run_streamlit, args=(port,))
thread.daemon = True
thread.start()

# Wait for the server to start
time.sleep(2)

# Display the app in an iframe
display(HTML(f"""
<iframe src="http://localhost:{port}" width="100%" height="800" style="border:none;"></iframe>
"""))